In [3]:
# CNT Physical Glyph Hunt — Single Cell v0.1
# One-cell bridge from knotted fields → CNT glyphness (topology + echo)
# Outputs: prints metrics and saves JSON/CSV under {CNT_LAB_DIR or E:\CNT}\artifacts\cnt_physical_glyph_hunt\{RUN_ID}

import numpy as np, os, json, time
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — Single Cell v0.1")

# ---- Topology helpers ----
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return np.meshgrid(kx, ky, kz, indexing='ij')

def _curl_spectral(Ax, Ay, Az, Lx, Ly, Lz):
    kx, ky, kz = _fftvec(*Ax.shape, Lx, Ly, Lz)
    Axh, Ayh, Azh = fftn(Ax), fftn(Ay), fftn(Az)
    i = 1j
    Bxh = i*(ky*Azh - kz*Ayh)
    Byh = i*(kz*Axh - kx*Azh)
    Bzh = i*(kx*Ayh - ky*Axh)
    Bx = np.real(ifftn(Bxh))
    By = np.real(ifftn(Byh))
    Bz = np.real(ifftn(Bzh))
    return Bx, By, Bz

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    Nx, Ny, Nz = Bx.shape
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax = np.real(ifftn(Axh)); Ay = np.real(ifftn(Ayh)); Az = np.real(ifftn(Azh))
    return Ax, Ay, Az

def helicity(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---- Synthetic fields (testbeds) ----
def _grid(N=48, L=2*np.pi):
    x = np.linspace(0, L, N, endpoint=False)
    y = np.linspace(0, L, N, endpoint=False)
    z = np.linspace(0, L, N, endpoint=False)
    return np.meshgrid(x, y, z, indexing='ij')

def abc_flow(N=48, L=2*np.pi, A=1.0, B=1.0, C=1.0):
    x, y, z = _grid(N, L)
    Bx = A*np.sin(z) + C*np.cos(y)
    By = B*np.sin(x) + A*np.cos(z)
    Bz = C*np.sin(y) + B*np.cos(x)
    dx = dy = dz = L / N
    return (Bx, By, Bz), (dx, dy, dz)

def hopf_map_n(N=48, L=4.0):
    xs = np.linspace(-L/2, L/2, N, endpoint=False)
    ys = np.linspace(-L/2, L/2, N, endpoint=False)
    zs = np.linspace(-L/2, L/2, N, endpoint=False)
    x, y, z = np.meshgrid(xs, ys, zs, indexing='ij')
    r2 = x*x + y*y + z*z
    denom = 1.0 + r2
    u = (2*(x + 1j*y)) / denom
    v = (1 - r2 + 2j*z) / denom
    uv_bar = u * np.conjugate(v)
    nx = 2.0 * np.real(uv_bar)
    ny = 2.0 * np.imag(uv_bar)
    nz = np.abs(u)**2 - np.abs(v)**2
    norm = np.sqrt(nx*nx + ny*ny + nz*nz) + 1e-12
    nx, ny, nz = nx/norm, ny/norm, nz/norm
    dx = dy = dz = L / N
    return (nx, ny, nz), (dx, dy, dz)

# ---- Hopf invariant (approx., periodic box) ----
def _grad(field, spacings):
    dx, dy, dz = spacings
    gx = np.gradient(field, dx, axis=0)
    gy = np.gradient(field, dy, axis=1)
    gz = np.gradient(field, dz, axis=2)
    return gx, gy, gz

def hopf_index(nx, ny, nz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz

    dnx = _grad(nx, spacings); dny = _grad(ny, spacings); dnz = _grad(nz, spacings)
    dn = [
        np.stack([dnx[0], dny[0], dnz[0]], axis=0),
        np.stack([dnx[1], dny[1], dnz[1]], axis=0),
        np.stack([dnx[2], dny[2], dnz[2]], axis=0),
    ]
    nvec = np.stack([nx, ny, nz], axis=0)

    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)

    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0

    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    b = [bi/(8*np.pi) for bi in b]

    Ax, Ay, Az = _vector_potential_from_B(b[0], b[1], b[2], Lx, Ly, Lz)
    curlAx, curlAy, curlAz = _curl_spectral(Ax, Ay, Az, Lx, Ly, Lz)
    integrand = Ax*curlAx + Ay*curlAy + Az*curlAz
    H = (1.0/(4*np.pi**2)) * np.sum(integrand) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---- Echo + glyphness ----
def echo_spectrum(signal, dt):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=dt)
    spec = np.abs(np.fft.rfft(signal))**2 / n
    return freqs, spec

def glyphness_score(H_norm, hopf_norm, echo_power=0.0, weights=(0.4,0.4,0.2)):
    w1, w2, w3 = weights
    s = w1*H_norm + w2*hopf_norm + w3*echo_power
    return float(max(0.0, min(1.0, s)))

# ---- Run demo on synthetic fields (adjust N for speed/accuracy) ----
(Bx, By, Bz), sp = abc_flow(N=32, L=2*np.pi)
H, h = helicity(Bx,By,Bz, sp)
print(f"[ABC] Helicity: H={H:.6f}  h=H/V={h:.6e}")

(nx,ny,nz), spn = hopf_map_n(N=32, L=4.0)
HH, hh = hopf_index(nx,ny,nz, spn)
print(f"[Hopf] Invariant≈{HH:.4f}  density={hh:.6e}")

t = np.linspace(0, 10, 4096)
sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
freqs, P = echo_spectrum(sig, dt=t[1]-t[0])
mask = (freqs>1.6) & (freqs<1.8)
echo_power = float((P[mask].sum()) / (P.sum()+1e-12))

H_norm  = min(1.0, abs(h)/(1e-2))       # scale knobs
Hopf_n  = min(1.0, abs(hh)/(1e-2))      # scale knobs
score   = glyphness_score(H_norm, Hopf_n, echo_power)
print(f"[CNT] glyphness={score:.3f} (H_norm={H_norm:.2f}, Hopf_norm={Hopf_n:.2f}, echo={echo_power:.3f})")

# ---- Save artifacts ----
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
run_dir = root / "artifacts" / "cnt_physical_glyph_hunt" / time.strftime("%Y%m%d-%H%M%SZ", time.gmtime())
run_dir.mkdir(parents=True, exist_ok=True)

summary = {
    "helicity": {"H": H, "h_density": h},
    "hopf": {"H_hopf": HH, "h_density": hh},
    "echo": {"band_center_hz": 1.7, "band_power_norm": echo_power},
    "glyphness": {"score": score, "weights": [0.4,0.4,0.2], "scales": {"H_norm_div": 1e-2, "Hopf_norm_div": 1e-2}},
    "grid": {"ABC_N": 32, "Hopf_N": 32},
    "run_dir": str(run_dir)
}
with open(run_dir/"summary.json","w") as fp:
    json.dump(summary, fp, indent=2)
np.savetxt(run_dir/"echo_spectrum.csv", np.column_stack([freqs, P]), delimiter=",", header="freq,power", comments="")
print("Saved →", run_dir/"summary.json")


CNT Physical Glyph Hunt — Single Cell v0.1
[ABC] Helicity: H=744.150640  h=H/V=3.000000e+00
[Hopf] Invariant≈0.0193  density=3.021769e-04
[CNT] glyphness=0.586 (H_norm=1.00, Hopf_norm=0.03, echo=0.870)
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-215252Z\summary.json


In [4]:
# CNT Physical Glyph Hunt — Single Cell v0.2 (calibrated)
# - Helicity (A·B) from ABC test field
# - Hopf proxy on normalized B, normalized by a Hopf-map baseline
# - Echo spectrum from file if available, else demo
# - Saves artifacts under {CNT_LAB_DIR or E:\CNT}\artifacts\cnt_physical_glyph_hunt\{RUN_ID}

import numpy as np, os, json, time, pathlib
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — Single Cell v0.2 (calibrated)")

# ---------- FFT helpers ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return np.meshgrid(kx, ky, kz, indexing='ij')

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    Nx, Ny, Nz = Bx.shape
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax = np.real(ifftn(Axh)); Ay = np.real(ifftn(Ayh)); Az = np.real(ifftn(Azh))
    return Ax, Ay, Az

def helicity(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---------- Grids & synthetic fields ----------
def _grid(N=48, L=2*np.pi):
    x = np.linspace(0, L, N, endpoint=False)
    y = np.linspace(0, L, N, endpoint=False)
    z = np.linspace(0, L, N, endpoint=False)
    return np.meshgrid(x, y, z, indexing='ij')

def abc_flow(N=48, L=2*np.pi, A=1.0, B=1.0, C=1.0):
    x, y, z = _grid(N, L)
    Bx = A*np.sin(z) + C*np.cos(y)
    By = B*np.sin(x) + A*np.cos(z)
    Bz = C*np.sin(y) + B*np.cos(x)
    dx = dy = dz = L / N
    return (Bx, By, Bz), (dx, dy, dz)

def hopf_map_n(N=48, L=4.0):
    xs = np.linspace(-L/2, L/2, N, endpoint=False)
    ys = np.linspace(-L/2, L/2, N, endpoint=False)
    zs = np.linspace(-L/2, L/2, N, endpoint=False)
    x, y, z = np.meshgrid(xs, ys, zs, indexing='ij')
    r2 = x*x + y*y + z*z
    denom = 1.0 + r2
    u = (2*(x + 1j*y)) / denom
    v = (1 - r2 + 2j*z) / denom
    uv_bar = u * np.conjugate(v)
    nx = 2.0 * np.real(uv_bar)
    ny = 2.0 * np.imag(uv_bar)
    nz = np.abs(u)**2 - np.abs(v)**2
    norm = np.sqrt(nx*nx + ny*ny + nz*nz) + 1e-12
    nx, ny, nz = nx/norm, ny/norm, nz/norm
    dx = dy = dz = L / N
    return (nx, ny, nz), (dx, dy, dz)

# ---------- Hopf proxy (b -> A -> A·curlA) ----------
def _grad(field, spacings):
    dx, dy, dz = spacings
    gx = np.gradient(field, dx, axis=0)
    gy = np.gradient(field, dy, axis=1)
    gz = np.gradient(field, dz, axis=2)
    return gx, gy, gz

def hopf_proxy(nxyz, spacings):
    # b_i = (1/8π) ε_{ijk} n · (∂_j n × ∂_k n); solve ∇×A = b; return (1/4π^2)∫ A·(∇×A)
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz

    dnx = _grad(nx, spacings); dny = _grad(ny, spacings); dnz = _grad(nz, spacings)
    dn = [
        np.stack([dnx[0], dny[0], dnz[0]], axis=0),
        np.stack([dnx[1], dny[1], dnz[1]], axis=0),
        np.stack([dnx[2], dny[2], dnz[2]], axis=0),
    ]
    nvec = np.stack([nx, ny, nz], axis=0)

    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)

    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0

    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    b = [bi/(8*np.pi) for bi in b]

    Ax, Ay, Az = _vector_potential_from_B(b[0], b[1], b[2], Lx, Ly, Lz)

    # curl(A) in spectral
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Axh, Ayh, Azh = fftn(Ax), fftn(Ay), fftn(Az)
    i = 1j
    Bxh = i*(ky*Azh - kz*Ayh)
    Byh = i*(kz*Axh - kx*Azh)
    Bzh = i*(kx*Ayh - ky*Axh)
    Bx = np.real(ifftn(Bxh)); By = np.real(ifftn(Byh)); Bz = np.real(ifftn(Bzh))

    integrand = Ax*Bx + Ay*By + Az*Bz
    H = (1.0/(4*np.pi**2)) * np.sum(integrand) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---------- Echo ----------
def echo_spectrum(signal, dt):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=dt)
    spec  = (np.abs(np.fft.rfft(signal))**2) / n
    return freqs, spec

# ---------- Run (synthetic with optional real echo) ----------
# 1) ABC field → helicity
(Bx, By, Bz), spB = abc_flow(N=48, L=2*np.pi)
H, h = helicity(Bx, By, Bz, spB)
print(f"[ABC] Helicity: H={H:.6f}  h=H/V={h:.6e}")

# 2) Hopf baseline from Hopf-map field (same grid size for fair normalization)
(nhx, nhy, nhz), spN = hopf_map_n(N=48, L=4.0)
H_hopf_base, h_hopf_base = hopf_proxy((nhx, nhy, nhz), spN)
print(f"[Hopf baseline] proxy ≈ {H_hopf_base:.4f}  density={h_hopf_base:.6e}")

# 3) Hopf proxy on normalized B-field
mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
nxB, nyB, nzB = Bx/mag, By/mag, Bz/mag
H_hopf_B, h_hopf_B = hopf_proxy((nxB, nyB, nzB), spB)
# Normalize relative to baseline (cap at 1 for scoring)
Hopf_norm = float(min(1.0, abs(h_hopf_B) / (abs(h_hopf_base) + 1e-18)))
print(f"[Hopf on B] proxy ≈ {H_hopf_B:.4f}  density={h_hopf_B:.6e}  → normalized={Hopf_norm:.3f}")

# 4) Echo: try file, else demo
echo_path = Path("E:/CNT/data/echo_signal.csv")
if not echo_path.exists():
    echo_path = Path(os.getenv("CNT_LAB_DIR", "")) / "data" / "echo_signal.csv"
use_demo = True
if echo_path.exists():
    try:
        arr = np.loadtxt(echo_path, delimiter=",", ndmin=1)
        sig = arr.astype(float)
        dt = 1.0
        use_demo = False
        print(f"[Echo] Loaded {echo_path} (len={len(sig)})")
    except Exception as e:
        print(f"[Echo] Failed to load file, using demo. Reason: {e}")
if use_demo:
    t = np.linspace(0, 10, 4096)
    dt = t[1] - t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)

freqs, P = echo_spectrum(sig, dt)
band = (1.6, 1.8)
mask = (freqs>band[0]) & (freqs<band[1])
echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
print(f"[Echo] band={band}  echo_power={echo_power:.3f}")

# 5) Scoring (tune scales as you like)
H_norm = float(min(1.0, abs(h) / (1e-2)))   # helicity density scale
w1, w2, w3 = 0.4, 0.4, 0.2
glyphness = float(w1*H_norm + w2*Hopf_norm + w3*echo_power)
print(f"[CNT] glyphness={glyphness:.3f}  (H_norm={H_norm:.2f}, Hopf_norm={Hopf_norm:.2f}, echo={echo_power:.3f})")

# ---------- Save artifacts ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
run_dir = root / "artifacts" / "cnt_physical_glyph_hunt" / time.strftime("%Y%m%d-%H%M%SZ", time.gmtime())
run_dir.mkdir(parents=True, exist_ok=True)

summary = {
    "helicity": {"H": H, "h_density": h, "scale_div": 1e-2},
    "hopf_proxy": {
        "baseline": {"H": H_hopf_base, "h_density": h_hopf_base},
        "on_B": {"H": H_hopf_B, "h_density": h_hopf_B},
        "normalized": Hopf_norm
    },
    "echo": {"band": list(band), "power_norm": echo_power, "dt": dt, "source": "file" if not use_demo else "demo"},
    "glyphness": {"score": glyphness, "weights": [w1,w2,w3]},
    "grid": {"N": 48, "L_ABC": float(2*np.pi), "L_Hopf": 4.0},
    "run_dir": str(run_dir)
}
with open(run_dir/"summary.json","w") as fp:
    json.dump(summary, fp, indent=2)
np.savetxt(run_dir/"echo_spectrum.csv", np.column_stack([freqs, P]), delimiter=",", header="freq,power", comments="")

print("Saved →", run_dir/"summary.json")


CNT Physical Glyph Hunt — Single Cell v0.2 (calibrated)
[ABC] Helicity: H=744.150640  h=H/V=3.000000e+00
[Hopf baseline] proxy ≈ 0.0202  density=3.155740e-04
[Hopf on B] proxy ≈ -0.0003  density=-1.330598e-06  → normalized=0.004
[Echo] band=(1.6, 1.8)  echo_power=0.870
[CNT] glyphness=0.576  (H_norm=1.00, Hopf_norm=0.00, echo=0.870)
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-222410Z\summary.json


In [5]:
# CNT Physical Glyph Hunt — Single Cell v0.3 (Hopfion-ready + Real-Feed Auto)
# - If E:\CNT\data\B_field.npz (or CNT_LAB_DIR\data\B_field.npz) exists, uses that 3D field (Bx,By,Bz,dx,dy,dz)
# - Else builds a true "Hopfion-B" field (topological) so Hopf_norm calibrates high (expect ~O(0.2–0.8), grid-dependent)
# - Else falls back to ABC (nontrivial helicity, low Hopf_norm)
# - Echo from E:\CNT\data\echo_signal.csv (or CNT_LAB_DIR\data), else demo
# - Saves artifacts → {CNT_LAB_DIR or E:\CNT}\artifacts\cnt_physical_glyph_hunt\{RUN_ID}

import numpy as np, os, json, time
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — Single Cell v0.3")

# ---------------- FFT helpers ----------------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return np.meshgrid(kx, ky, kz, indexing='ij')

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    Nx, Ny, Nz = Bx.shape
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax = np.real(ifftn(Axh)); Ay = np.real(ifftn(Ayh)); Az = np.real(ifftn(Azh))
    return Ax, Ay, Az

def helicity(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---------------- Grids & synthetic fields ----------------
def _grid(N=48, L=2*np.pi):
    x = np.linspace(0, L, N, endpoint=False)
    y = np.linspace(0, L, N, endpoint=False)
    z = np.linspace(0, L, N, endpoint=False)
    return np.meshgrid(x, y, z, indexing='ij')

def abc_flow(N=48, L=2*np.pi, A=1.0, B=1.0, C=1.0):
    x, y, z = _grid(N, L)
    Bx = A*np.sin(z) + C*np.cos(y)
    By = B*np.sin(x) + A*np.cos(z)
    Bz = C*np.sin(y) + B*np.cos(x)
    dx = dy = dz = L / N
    return (Bx, By, Bz), (dx, dy, dz)

def hopf_map_n(N=48, L=8.0):
    xs = np.linspace(-L/2, L/2, N, endpoint=False)
    ys = np.linspace(-L/2, L/2, N, endpoint=False)
    zs = np.linspace(-L/2, L/2, N, endpoint=False)
    x, y, z = np.meshgrid(xs, ys, zs, indexing='ij')
    r2 = x*x + y*y + z*z
    denom = 1.0 + r2
    u = (2*(x + 1j*y)) / denom
    v = (1 - r2 + 2j*z) / denom
    uv_bar = u * np.conjugate(v)
    nx = 2.0 * np.real(uv_bar)
    ny = 2.0 * np.imag(uv_bar)
    nz = np.abs(u)**2 - np.abs(v)**2
    norm = np.sqrt(nx*nx + ny*ny + nz*nz) + 1e-12
    nx, ny, nz = nx/norm, ny/norm, nz/norm
    dx = dy = dz = L / N
    return (nx, ny, nz), (dx, dy, dz)

# ---------------- Hopf proxy & Hopfion-B ----------------
def _grad(field, spacings):
    dx, dy, dz = spacings
    gx = np.gradient(field, dx, axis=0)
    gy = np.gradient(field, dy, axis=1)
    gz = np.gradient(field, dz, axis=2)
    return gx, gy, gz

def b_from_n(nxyz, spacings):
    nx, ny, nz = nxyz
    dnx = _grad(nx, spacings); dny = _grad(ny, spacings); dnz = _grad(nz, spacings)
    dn = [
        np.stack([dnx[0], dny[0], dnz[0]], axis=0),
        np.stack([dnx[1], dny[1], dnz[1]], axis=0),
        np.stack([dnx[2], dny[2], dnz[2]], axis=0),
    ]
    nvec = np.stack([nx, ny, nz], axis=0)
    Nx, Ny, Nz = nx.shape
    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0
    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    b = [bi/(8*np.pi) for bi in b]
    return b  # list of 3 arrays

def hopf_proxy(nxyz, spacings):
    # ∇×A = b(n); return (1/4π^2) ∫ A·(∇×A)
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    b = b_from_n(nxyz, spacings)
    Ax, Ay, Az = _vector_potential_from_B(b[0], b[1], b[2], Lx, Ly, Lz)
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Axh, Ayh, Azh = fftn(Ax), fftn(Ay), fftn(Az)
    i = 1j
    Bxh = i*(ky*Azh - kz*Ayh)
    Byh = i*(kz*Axh - kx*Azh)
    Bzh = i*(kx*Ayh - ky*Axh)
    Bx = np.real(ifftn(Bxh)); By = np.real(ifftn(Byh)); Bz = np.real(ifftn(Bzh))
    integrand = Ax*Bx + Ay*By + Az*Bz
    H = (1.0/(4*np.pi**2)) * np.sum(integrand) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---------------- Echo ----------------
def echo_spectrum(signal, dt):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=dt)
    spec  = (np.abs(np.fft.rfft(signal))**2) / n
    return freqs, spec

# ---------------- Choose field ----------------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
data_dir = root / "data"

field_path = None
for cand in [data_dir / "B_field.npz", Path("E:/CNT/data/B_field.npz")]:
    if cand.exists():
        field_path = cand; break

mode = "npz" if field_path else "hopfion"  # prefer real, else Hopfion-B, else ABC
if not field_path:
    mode = "hopfion"  # set "abc" here if you want ABC instead

if mode == "npz":
    d = np.load(field_path)
    Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
    if all(k in d for k in ["dx","dy","dz"]):
        spacings = (float(d["dx"]), float(d["dy"]), float(d["dz"]))
    else:
        # fallback: infer cubic box
        N = Bx.shape[0]; L = 2*np.pi; spacings = (L/N, L/N, L/N)
    src = f"npz:{field_path}"
elif mode == "hopfion":
    N = 48; L = 8.0
    (nx,ny,nz), sp = hopf_map_n(N=N, L=L)
    b = b_from_n((nx,ny,nz), sp)
    Bx, By, Bz = b  # define Hopfion-B = b(n)
    spacings = sp
    src = "hopfion_B"
else:
    (Bx, By, Bz), spacings = abc_flow(N=48, L=2*np.pi)
    src = "abc"

# ---------------- Metrics ----------------
H, h = helicity(Bx, By, Bz, spacings)
print(f"[Field:{src}] Helicity: H={H:.6f}  h=H/V={h:.6e}")

# Hopf baseline (Hopf-map n) for normalization
(nhx, nhy, nhz), spN = hopf_map_n(N=48, L=8.0)
H_hopf_base, h_hopf_base = hopf_proxy((nhx, nhy, nhz), spN)
print(f"[Hopf baseline] proxy ≈ {H_hopf_base:.4f}  density={h_hopf_base:.6e}")

# Hopf on normalized B
mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
nxB, nyB, nzB = Bx/mag, By/mag, Bz/mag
H_hopf_B, h_hopf_B = hopf_proxy((nxB, nyB, nzB), spacings)
Hopf_norm = float(min(1.0, abs(h_hopf_B) / (abs(h_hopf_base) + 1e-18)))
print(f"[Hopf on B] proxy ≈ {H_hopf_B:.4f}  density={h_hopf_B:.6e}  → normalized={Hopf_norm:.3f}")

# Echo: try file, else demo
echo_path = None
for cand in [data_dir/"echo_signal.csv", Path("E:/CNT/data/echo_signal.csv")]:
    if cand.exists():
        echo_path = cand; break

use_demo = True
if echo_path and echo_path.exists():
    try:
        arr = np.loadtxt(echo_path, delimiter=",", ndmin=1)
        sig = arr.astype(float); dt = 1.0; use_demo = False
        print(f"[Echo] Loaded {echo_path} (len={len(sig)})")
    except Exception as e:
        print(f"[Echo] load failed → demo. Reason: {e}")
if use_demo:
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)

freqs, P = echo_spectrum(sig, dt)
band = (1.6, 1.8)
mask = (freqs>band[0]) & (freqs<band[1])
echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
print(f"[Echo] band={band}  echo_power={echo_power:.3f}")

# Scoring
H_norm = float(min(1.0, abs(h) / (1e-2)))     # helicity density scale
w1, w2, w3 = 0.4, 0.4, 0.2
glyphness = float(w1*H_norm + w2*Hopf_norm + w3*echo_power)
label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.65 and Hopf_norm>=0.10) else "CANDIDATE"

print(f"[CNT] glyphness={glyphness:.3f}  (H_norm={H_norm:.2f}, Hopf_norm={Hopf_norm:.2f}, echo={echo_power:.3f}) → {label}")

# ---------------- Save artifacts ----------------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
run_dir = root / "artifacts" / "cnt_physical_glyph_hunt" / time.strftime("%Y%m%d-%H%M%SZ", time.gmtime())
run_dir.mkdir(parents=True, exist_ok=True)
np.savez_compressed(run_dir/"field_small.npz", Bx=Bx, By=By, Bz=Bz, dx=spacings[0], dy=spacings[1], dz=spacings[2])

summary = {
    "source": src,
    "helicity": {"H": H, "h_density": h, "scale_div": 1e-2},
    "hopf_proxy": {
        "baseline": {"H": H_hopf_base, "h_density": h_hopf_base},
        "on_B": {"H": H_hopf_B, "h_density": h_hopf_B},
        "normalized": Hopf_norm
    },
    "echo": {"band": list(band), "power_norm": echo_power, "dt": float(dt), "source": "file" if not use_demo else "demo"},
    "glyphness": {"score": glyphness, "weights": [w1,w2,w3], "label": label},
    "run_dir": str(run_dir)
}
with open(run_dir/"summary.json","w") as fp:
    json.dump(summary, fp, indent=2)
np.savetxt(run_dir/"echo_spectrum.csv", np.column_stack([freqs, P]), delimiter=",", header="freq,power", comments="")

print("Saved →", run_dir/"summary.json")


CNT Physical Glyph Hunt — Single Cell v0.3
[Field:hopfion_B] Helicity: H=0.861464  h=H/V=1.682547e-03
[Hopf baseline] proxy ≈ 0.0218  density=4.261941e-05
[Hopf on B] proxy ≈ 0.0058  density=1.124515e-05  → normalized=0.264
[Echo] band=(1.6, 1.8)  echo_power=0.870
[CNT] glyphness=0.347  (H_norm=0.17, Hopf_norm=0.26, echo=0.870) → CANDIDATE
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-224915Z\summary.json


In [6]:
# CNT Physical Glyph Hunt — Single Cell v0.3.1 (adaptive + curl-mix Hopfion)
# - Uses your real field if E:\CNT\data\B_field.npz (or CNT_LAB_DIR\data) exists
# - Else builds a Hopfion-derived B and adds λ * curl(B) to raise helicity
# - Adaptive scoring: more weight on Hopf topology in Hopfion mode
# - Saves to {CNT_LAB_DIR or E:\CNT}\artifacts\cnt_physical_glyph_hunt\{RUN_ID}

import numpy as np, os, json, time
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — Single Cell v0.3.1")

# ---------------- FFT helpers ----------------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return np.meshgrid(kx, ky, kz, indexing='ij')

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    Nx, Ny, Nz = Bx.shape
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax = np.real(ifftn(Axh)); Ay = np.real(ifftn(Ayh)); Az = np.real(ifftn(Azh))
    return Ax, Ay, Az

def helicity(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---------------- Base grids + fields ----------------
def _grid(N=48, L=2*np.pi):
    x = np.linspace(0, L, N, endpoint=False)
    y = np.linspace(0, L, N, endpoint=False)
    z = np.linspace(0, L, N, endpoint=False)
    return np.meshgrid(x, y, z, indexing='ij')

def abc_flow(N=48, L=2*np.pi, A=1.0, B=1.0, C=1.0):
    x, y, z = _grid(N, L)
    Bx = A*np.sin(z) + C*np.cos(y)
    By = B*np.sin(x) + A*np.cos(z)
    Bz = C*np.sin(y) + B*np.cos(x)
    dx = dy = dz = L / N
    return (Bx, By, Bz), (dx, dy, dz)

def hopf_map_n(N=64, L=8.0):
    xs = np.linspace(-L/2, L/2, N, endpoint=False)
    ys = np.linspace(-L/2, L/2, N, endpoint=False)
    zs = np.linspace(-L/2, L/2, N, endpoint=False)
    x, y, z = np.meshgrid(xs, ys, zs, indexing='ij')
    r2 = x*x + y*y + z*z
    denom = 1.0 + r2
    u = (2*(x + 1j*y)) / denom
    v = (1 - r2 + 2j*z) / denom
    uv_bar = u * np.conjugate(v)
    nx = 2.0 * np.real(uv_bar)
    ny = 2.0 * np.imag(uv_bar)
    nz = np.abs(u)**2 - np.abs(v)**2
    norm = np.sqrt(nx*nx + ny*ny + nz*nz) + 1e-12
    nx, ny, nz = nx/norm, ny/norm, nz/norm
    dx = dy = dz = L / N
    return (nx, ny, nz), (dx, dy, dz)

# ---------------- Derivatives & Hopf proxy ----------------
def _grad(field, spacings):
    dx, dy, dz = spacings
    gx = np.gradient(field, dx, axis=0)
    gy = np.gradient(field, dy, axis=1)
    gz = np.gradient(field, dz, axis=2)
    return gx, gy, gz

def curl_real(Fx, Fy, Fz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(Fy, dz, axis=2)
    dFz_dy = np.gradient(Fz, dy, axis=1)
    dFz_dx = np.gradient(Fz, dx, axis=0)
    dFx_dz = np.gradient(Fx, dz, axis=2)
    dFx_dy = np.gradient(Fx, dy, axis=1)
    dFy_dx = np.gradient(Fy, dx, axis=0)
    Cx = dFz_dy - dFy_dz
    Cy = dFx_dz - dFz_dx
    Cz = dFy_dx - dFx_dy
    return Cx, Cy, Cz

def b_from_n(nxyz, spacings):
    nx, ny, nz = nxyz
    dnx = _grad(nx, spacings); dny = _grad(ny, spacings); dnz = _grad(nz, spacings)
    dn = [
        np.stack([dnx[0], dny[0], dnz[0]], axis=0),
        np.stack([dnx[1], dny[1], dnz[1]], axis=0),
        np.stack([dnx[2], dny[2], dnz[2]], axis=0),
    ]
    nvec = np.stack([nx, ny, nz], axis=0)
    Nx, Ny, Nz = nx.shape
    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0
    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    b = [bi/(8*np.pi) for bi in b]
    return b

def hopf_proxy(nxyz, spacings):
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    b = b_from_n(nxyz, spacings)
    Ax, Ay, Az = _vector_potential_from_B(b[0], b[1], b[2], Lx, Ly, Lz)
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Axh, Ayh, Azh = fftn(Ax), fftn(Ay), fftn(Az)
    i = 1j
    Bxh = i*(ky*Azh - kz*Ayh)
    Byh = i*(kz*Axh - kx*Azh)
    Bzh = i*(kx*Ayh - ky*Axh)
    Bx = np.real(ifftn(Bxh)); By = np.real(ifftn(Byh)); Bz = np.real(ifftn(Bzh))
    integrand = Ax*Bx + Ay*By + Az*Bz
    H = (1.0/(4*np.pi**2)) * np.sum(integrand) * dx*dy*dz
    V = (dx*Nx)*(dy*Ny)*(dz*Nz)
    return float(H), float(H/V)

# ---------------- Echo ----------------
def echo_spectrum(signal, dt):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=dt)
    spec  = (np.abs(np.fft.rfft(signal))**2) / n
    return freqs, spec

# ---------------- Field selection ----------------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
data_dir = root / "data"
field_path = None
for cand in [data_dir / "B_field.npz", Path("E:/CNT/data/B_field.npz")]:
    if cand.exists():
        field_path = cand; break

# Parameters you can tweak
N = 64
L = 8.0
lam = 0.8     # curl-mix strength (raises helicity)
amp = 1.0     # amplitude multiplier
mode = "npz" if field_path else "hopfion"  # or set to "abc"
print(f"[mode] {mode}")

if mode == "npz":
    d = np.load(field_path)
    Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
    if all(k in d for k in ["dx","dy","dz"]):
        spacings = (float(d["dx"]), float(d["dy"]), float(d["dz"]))
    else:
        Np = Bx.shape[0]; Lp = 2*np.pi; spacings = (Lp/Np, Lp/Np, Lp/Np)
    src = f"npz:{field_path}"
elif mode == "hopfion":
    (nx,ny,nz), sp = hopf_map_n(N=N, L=L)
    b = b_from_n((nx,ny,nz), sp)
    cb = curl_real(*b, sp)                 # curl(b)
    Bx, By, Bz = (b[0] + lam*cb[0])*amp, (b[1] + lam*cb[1])*amp, (b[2] + lam*cb[2])*amp
    spacings = sp
    src = f"hopfion_B+{lam}curl"
else:
    (Bx, By, Bz), spacings = abc_flow(N=N, L=2*np.pi)
    src = "abc"

# ---------------- Metrics ----------------
H, h = helicity(Bx, By, Bz, spacings)
print(f"[Field:{src}] Helicity: H={H:.6f}  h=H/V={h:.6e}")

# Hopf baseline from standard Hopf-map field
(nhx, nhy, nhz), spN = hopf_map_n(N=N, L=L)
H_hopf_base, h_hopf_base = hopf_proxy((nhx, nhy, nhz), spN)
print(f"[Hopf baseline] proxy ≈ {H_hopf_base:.4f}  density={h_hopf_base:.6e}")

# Hopf on normalized B
mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
nxB, nyB, nzB = Bx/mag, By/mag, Bz/mag
H_hopf_B, h_hopf_B = hopf_proxy((nxB, nyB, nzB), spacings)
Hopf_norm = float(min(1.0, abs(h_hopf_B) / (abs(h_hopf_base) + 1e-18)))
print(f"[Hopf on B] proxy ≈ {H_hopf_B:.4f}  density={h_hopf_B:.6e}  → normalized={Hopf_norm:.3f}")

# Echo: real or demo
echo_path = None
for cand in [data_dir/"echo_signal.csv", Path("E:/CNT/data/echo_signal.csv")]:
    if cand.exists():
        echo_path = cand; break
use_demo = True
if echo_path and echo_path.exists():
    try:
        arr = np.loadtxt(echo_path, delimiter=",", ndmin=1)
        sig = arr.astype(float); dt = 1.0; use_demo = False
        print(f"[Echo] Loaded {echo_path} (len={len(sig)})")
    except Exception as e:
        print(f"[Echo] load failed → demo. Reason: {e}")
if use_demo:
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)

freqs, P = echo_spectrum(sig, dt)
band = (1.6, 1.8)
mask = (freqs>band[0]) & (freqs<band[1])
echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
print(f"[Echo] band={band}  echo_power={echo_power:.3f}")

# Adaptive weights
if src.startswith("hopfion"):
    w1, w2, w3 = 0.2, 0.6, 0.2   # emphasize Hopf topology for Hopf hunts
else:
    w1, w2, w3 = 0.4, 0.4, 0.2

H_norm = float(min(1.0, abs(h) / (1e-2)))   # keep same helicity scale for comparability
glyphness = float(w1*H_norm + w2*Hopf_norm + w3*echo_power)
label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.65 and Hopf_norm>=0.20 and echo_power>=0.20) else "CANDIDATE"

print(f"[CNT] glyphness={glyphness:.3f}  (H_norm={H_norm:.2f}, Hopf_norm={Hopf_norm:.2f}, echo={echo_power:.3f}) → {label}")

# ---------------- Save artifacts ----------------
run_root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
run_dir = run_root / "artifacts" / "cnt_physical_glyph_hunt" / time.strftime("%Y%m%d-%H%M%SZ", time.gmtime())
run_dir.mkdir(parents=True, exist_ok=True)
np.savez_compressed(run_dir/"field_small.npz", Bx=Bx, By=By, Bz=Bz, dx=spacings[0], dy=spacings[1], dz=spacings[2])
np.savetxt(run_dir/"echo_spectrum.csv", np.column_stack([freqs, P]), delimiter=",", header="freq,power", comments="")

summary = {
    "source": src,
    "params": {"N": N, "L": L, "lam": lam, "amp": amp},
    "helicity": {"H": H, "h_density": h, "scale_div": 1e-2},
    "hopf_proxy": {
        "baseline": {"H": H_hopf_base, "h_density": h_hopf_base},
        "on_B": {"H": H_hopf_B, "h_density": h_hopf_B},
        "normalized": Hopf_norm
    },
    "echo": {"band": list(band), "power_norm": echo_power, "dt": float(dt), "source": "file" if not use_demo else "demo"},
    "glyphness": {"score": glyphness, "weights": [w1,w2,w3], "label": label},
    "run_dir": str(run_dir)
}
with open(run_dir/"summary.json","w") as fp:
    json.dump(summary, fp, indent=2)
print("Saved →", run_dir/"summary.json")


CNT Physical Glyph Hunt — Single Cell v0.3.1
[mode] hopfion
[Field:hopfion_B+0.8curl] Helicity: H=6.612452  h=H/V=1.291495e-02
[Hopf baseline] proxy ≈ 0.0231  density=4.506476e-05
[Hopf on B] proxy ≈ 0.0086  density=1.674330e-05  → normalized=0.372
[Echo] band=(1.6, 1.8)  echo_power=0.870
[CNT] glyphness=0.597  (H_norm=1.00, Hopf_norm=0.37, echo=0.870) → CANDIDATE
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-225850Z\summary.json


In [7]:
# CNT Physical Glyph Hunt — Single Cell v0.4 (PASS Finder: λ × smoothing sweep)
# - Uses real field if E:\CNT\data\B_field.npz (or CNT_LAB_DIR\data) exists
#   format: keys Bx,By,Bz (3D arrays), and optionally dx,dy,dz (floats)
# - Else: builds Hopfion-derived B and mixes curl(B) with λ to raise helicity without killing topology
# - Sweeps λ and spectral smoothing α, ranks glyphness, and saves a scoreboard
# - Echo from E:\CNT\data\echo_signal.csv (or CNT_LAB_DIR\data), else demo
# - Artifacts → {CNT_LAB_DIR or E:\CNT}\artifacts\cnt_physical_glyph_hunt\{RUN_ID}

import numpy as np, os, json, time, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — Single Cell v0.4 (PASS Finder)")

# ---------- FFT helpers ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return np.meshgrid(kx, ky, kz, indexing='ij')

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    Nx, Ny, Nz = Bx.shape
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax = np.real(ifftn(Axh)); Ay = np.real(ifftn(Ayh)); Az = np.real(ifftn(Azh))
    return Ax, Ay, Az

def helicity(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

def smooth_field(Bx, By, Bz, spacings, alpha=0.0):
    if alpha <= 0.0: 
        return Bx, By, Bz
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    k2 = kx*kx + ky*ky + kz*kz
    kmax2 = (np.abs(kx).max()**2 + np.abs(ky).max()**2 + np.abs(kz).max()**2)
    filt = np.exp(-alpha * (k2/(kmax2 + 1e-12)))
    Bxh, Byh, Bzh = fftn(Bx)*filt, fftn(By)*filt, fftn(Bz)*filt
    return np.real(ifftn(Bxh)), np.real(ifftn(Byh)), np.real(ifftn(Bzh))

# ---------- Field generators ----------
def _grid(N=64, L=8.0):
    x = np.linspace(0, L, N, endpoint=False)
    y = np.linspace(0, L, N, endpoint=False)
    z = np.linspace(0, L, N, endpoint=False)
    return np.meshgrid(x, y, z, indexing='ij')

def hopf_map_n(N=64, L=8.0):
    xs = np.linspace(-L/2, L/2, N, endpoint=False)
    ys = np.linspace(-L/2, L/2, N, endpoint=False)
    zs = np.linspace(-L/2, L/2, N, endpoint=False)
    x, y, z = np.meshgrid(xs, ys, zs, indexing='ij')
    r2 = x*x + y*y + z*z
    denom = 1.0 + r2
    u = (2*(x + 1j*y)) / denom
    v = (1 - r2 + 2j*z) / denom
    uv_bar = u * np.conjugate(v)
    nx = 2.0 * np.real(uv_bar)
    ny = 2.0 * np.imag(uv_bar)
    nz = np.abs(u)**2 - np.abs(v)**2
    norm = np.sqrt(nx*nx + ny*ny + nz*nz) + 1e-12
    return (nx/norm, ny/norm, nz/norm), (L/N, L/N, L/N)

def _grad(field, spacings):
    dx, dy, dz = spacings
    return (np.gradient(field, dx, axis=0),
            np.gradient(field, dy, axis=1),
            np.gradient(field, dz, axis=2))

def curl_real(Fx, Fy, Fz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(Fy, dz, axis=2)
    dFz_dy = np.gradient(Fz, dy, axis=1)
    dFz_dx = np.gradient(Fz, dx, axis=0)
    dFx_dz = np.gradient(Fx, dz, axis=2)
    dFx_dy = np.gradient(Fx, dy, axis=1)
    dFy_dx = np.gradient(Fy, dx, axis=0)
    return (dFz_dy - dFy_dz, dFx_dz - dFz_dx, dFy_dx - dFx_dy)

def b_from_n(nxyz, spacings):
    nx, ny, nz = nxyz
    dnx = _grad(nx, spacings); dny = _grad(ny, spacings); dnz = _grad(nz, spacings)
    dn = [np.stack([dnx[i], dny[i], dnz[i]], axis=0) for i in range(3)]
    nvec = np.stack([nx, ny, nz], axis=0)
    Nx, Ny, Nz = nx.shape
    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0
    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    return [bi/(8*np.pi) for bi in b]

def hopf_proxy(nxyz, spacings):
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    b = b_from_n(nxyz, spacings)
    Ax, Ay, Az = _vector_potential_from_B(b[0], b[1], b[2], Lx, Ly, Lz)
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Axh, Ayh, Azh = fftn(Ax), fftn(Ay), fftn(Az)
    i = 1j
    Bxh = i*(ky*Azh - kz*Ayh)
    Byh = i*(kz*Axh - kx*Azh)
    Bzh = i*(kx*Ayh - ky*Axh)
    Bx = np.real(ifftn(Bxh)); By = np.real(ifftn(Byh)); Bz = np.real(ifftn(Bzh))
    integrand = Ax*Bx + Ay*By + Az*Bz
    H = (1.0/(4*np.pi**2)) * np.sum(integrand) * dx*dy*dz
    V = (dx*Nx)*(dy*Ny)*(dz*Nz)
    return float(H), float(H/V)

# ---------- Echo ----------
def echo_spectrum(signal, dt):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=dt)
    spec  = (np.abs(np.fft.rfft(signal))**2) / n
    return freqs, spec

# ---------- Locate data / build field ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
data_dir = root / "data"
field_path = None
for cand in [data_dir/"B_field.npz", Path("E:/CNT/data/B_field.npz")]:
    if cand.exists():
        field_path = cand; break

if field_path:
    d = np.load(field_path)
    Bx0, By0, Bz0 = d["Bx"], d["By"], d["Bz"]
    if all(k in d for k in ["dx","dy","dz"]):
        spacings0 = (float(d["dx"]), float(d["dy"]), float(d["dz"]))
    else:
        N = Bx0.shape[0]; L = 2*np.pi; spacings0 = (L/N, L/N, L/N)
    mode = "npz"
else:
    mode = "hopfion"
    N, L = 64, 8.0
    (nx,ny,nz), sp = hopf_map_n(N=N, L=L)
    b = b_from_n((nx,ny,nz), sp)  # Hopfion-B seed
    Bx0, By0, Bz0 = b
    spacings0 = sp

print(f"[mode] {mode}")

# ---------- Echo (shared across sweep) ----------
echo_path = None
for cand in [data_dir/"echo_signal.csv", Path("E:/CNT/data/echo_signal.csv")]:
    if cand.exists():
        echo_path = cand; break
use_demo = True
if echo_path and echo_path.exists():
    try:
        arr = np.loadtxt(echo_path, delimiter=",", ndmin=1)
        sig = arr.astype(float); dt = 1.0; use_demo = False
        print(f"[Echo] Loaded {echo_path} (len={len(sig)})")
    except Exception as e:
        print(f"[Echo] load failed → demo. Reason: {e}")
if use_demo:
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
freqs, P = echo_spectrum(sig, dt)
band = (1.6, 1.8)
mask = (freqs>band[0]) & (freqs<band[1])
echo_power_shared = float((P[mask].sum()) / (P.sum()+1e-12))

# ---------- Hopf baseline (for normalization) ----------
Nx, Ny, Nz = Bx0.shape
Lx, Ly, Lz = spacings0[0]*Nx, spacings0[1]*Ny, spacings0[2]*Nz
L_base = max(Lx, Ly, Lz)
(nhx, nhy, nhz), spN = hopf_map_n(N=Nx, L=L_base)
H_hopf_base, h_hopf_base = hopf_proxy((nhx, nhy, nhz), spN)
print(f"[Hopf baseline] density={h_hopf_base:.6e}")

# ---------- Sweep ----------
lam_list   = ([0.4, 0.6, 0.8, 1.0, 1.2] if mode=="hopfion" else [0.0])
alpha_list = [0.0, 0.5, 1.0]  # spectral smoothing
results = []

def score_combo(Bx,By,Bz,spacings, echo_power, mode):
    H, h = helicity(Bx,By,Bz,spacings)
    mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    nxB, nyB, nzB = Bx/mag, By/mag, Bz/mag
    _, h_hopf_B = hopf_proxy((nxB, nyB, nzB), spacings)
    Hopf_norm = float(min(1.0, abs(h_hopf_B)/(abs(h_hopf_base)+1e-18)))
    if mode=="hopfion":    w1,w2,w3 = 0.2, 0.6, 0.2
    else:                  w1,w2,w3 = 0.4, 0.4, 0.2
    H_norm = float(min(1.0, abs(h)/(1e-2)))
    glyphness = float(w1*H_norm + w2*Hopf_norm + w3*echo_power)
    return dict(H=H, h=h, h_hopf_B=h_hopf_B, Hopf_norm=Hopf_norm, H_norm=H_norm,
                echo=echo_power, glyphness=glyphness, weights=(w1,w2,w3))

for lam in lam_list:
    for alpha in alpha_list:
        if mode=="hopfion":
            cbx, cby, cbz = curl_real(Bx0, By0, Bz0, spacings0)
            Bx = Bx0 + lam*cbx; By = By0 + lam*cby; Bz = Bz0 + lam*cbz
        else:
            Bx, By, Bz = Bx0.copy(), By0.copy(), Bz0.copy()
        Bx, By, Bz = smooth_field(Bx, By, Bz, spacings0, alpha=alpha)
        met = score_combo(Bx, By, Bz, spacings0, echo_power_shared, mode=mode)
        met.update(mode=mode, lam=float(lam), alpha=float(alpha))
        results.append(met)
        print(f"λ={lam:.2f} α={alpha:.2f} → glyphness={met['glyphness']:.3f}  (Hopf_norm={met['Hopf_norm']:.3f}, H_norm={met['H_norm']:.2f})")

# ---------- Pick best, label, and save ----------
thr = 0.65
best = max(results, key=lambda r: r['glyphness'])
label = "PHYSICAL-GLYPH:PASS" if (best['glyphness']>=thr and best['Hopf_norm']>=0.20 and best['echo']>=0.20) else "CANDIDATE"

run_root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
run_dir = run_root / "artifacts" / "cnt_physical_glyph_hunt" / time.strftime("%Y%m%d-%H%M%SZ", time.gmtime())
run_dir.mkdir(parents=True, exist_ok=True)

# Save scoreboard
with open(run_dir/"scoreboard.csv","w", newline="") as f:
    wr = csv.writer(f)
    wr.writerow(["mode","lam","alpha","glyphness","Hopf_norm","H_norm","echo","h","h_hopf_B"])
    for r in sorted(results, key=lambda x: x['glyphness'], reverse=True):
        wr.writerow([r['mode'], r['lam'], r['alpha'], f"{r['glyphness']:.6f}", f"{r['Hopf_norm']:.6f}",
                     f"{r['H_norm']:.6f}", f"{r['echo']:.6f}", f"{r['h']:.6e}", f"{r['h_hopf_B']:.6e}"])

# Reconstruct best field to save
lam_best, alpha_best = best['lam'], best['alpha']
if mode=="hopfion":
    cbx, cby, cbz = curl_real(Bx0, By0, Bz0, spacings0)
    BxB, ByB, BzB = Bx0 + lam_best*cbx, By0 + lam_best*cby, Bz0 + lam_best*cbz
else:
    BxB, ByB, BzB = Bx0.copy(), By0.copy(), Bz0.copy()
BxB, ByB, BzB = smooth_field(BxB, ByB, BzB, spacings0, alpha=alpha_best)

np.savez_compressed(run_dir/"best_field.npz", Bx=BxB, By=ByB, Bz=BzB, dx=spacings0[0], dy=spacings0[1], dz=spacings0[2])
np.savetxt(run_dir/"echo_spectrum.csv", np.column_stack([freqs, P]), delimiter=",", header="freq,power", comments="")

summary = {
    "mode": mode,
    "hopf_baseline_density": h_hopf_base,
    "echo": {"band": list(band), "power_norm": echo_power_shared, "source": "file" if not use_demo else "demo"},
    "best": {
        "lam": lam_best, "alpha": alpha_best, "glyphness": best['glyphness'],
        "Hopf_norm": best['Hopf_norm'], "H_norm": best['H_norm'], "echo": best['echo'],
        "label": "PHYSICAL-GLYPH:PASS" if label=="PHYSICAL-GLYPH:PASS" else "CANDIDATE"
    },
    "thresholds": {"glyphness": thr, "min_Hopf_norm": 0.20, "min_echo": 0.20},
    "run_dir": str(run_dir)
}
with open(run_dir/"summary.json","w") as fp:
    json.dump(summary, fp, indent=2)

print(f"TOP → λ={lam_best:.2f} α={alpha_best:.2f}  glyphness={best['glyphness']:.3f}  Hopf_norm={best['Hopf_norm']:.3f} → {label}")
print("Saved →", run_dir/"summary.json")


CNT Physical Glyph Hunt — Single Cell v0.4 (PASS Finder)
[mode] hopfion
[Hopf baseline] density=4.506476e-05
λ=0.40 α=0.00 → glyphness=0.486  (Hopf_norm=0.320, H_norm=0.60)
λ=0.40 α=0.50 → glyphness=0.485  (Hopf_norm=0.320, H_norm=0.60)
λ=0.40 α=1.00 → glyphness=0.484  (Hopf_norm=0.319, H_norm=0.59)
λ=0.60 α=0.00 → glyphness=0.565  (Hopf_norm=0.348, H_norm=0.91)
λ=0.60 α=0.50 → glyphness=0.563  (Hopf_norm=0.347, H_norm=0.91)
λ=0.60 α=1.00 → glyphness=0.562  (Hopf_norm=0.346, H_norm=0.90)
λ=0.80 α=0.00 → glyphness=0.597  (Hopf_norm=0.372, H_norm=1.00)
λ=0.80 α=0.50 → glyphness=0.597  (Hopf_norm=0.371, H_norm=1.00)
λ=0.80 α=1.00 → glyphness=0.596  (Hopf_norm=0.370, H_norm=1.00)
λ=1.00 α=0.00 → glyphness=0.609  (Hopf_norm=0.393, H_norm=1.00)
λ=1.00 α=0.50 → glyphness=0.609  (Hopf_norm=0.392, H_norm=1.00)
λ=1.00 α=1.00 → glyphness=0.609  (Hopf_norm=0.391, H_norm=1.00)
λ=1.20 α=0.00 → glyphness=0.621  (Hopf_norm=0.411, H_norm=1.00)
λ=1.20 α=0.50 → glyphness=0.620  (Hopf_norm=0.411, H_norm=1

In [8]:
# CNT Physical Glyph Hunt — Single Cell v0.5 (PASS Razor: solenoidal projection + λ, s sweep)
# - Prefers real field at E:\CNT\data\B_field.npz (or CNT_LAB_DIR\data)
#   expected keys: Bx, By, Bz (3D), optionally dx, dy, dz
# - Else: builds Hopfion-derived B = b(n_s) + λ * curl(b), with s ∈ {0.6, 0.8, 1.0}
# - Projects B to divergence-free in spectral domain (improves topological integrals)
# - Sweeps λ ∈ {0.8,1.0,1.2,1.4,1.6,1.8,2.0}, α ∈ {0.0,0.5} (spectral smoothing), s ∈ {0.6,0.8,1.0}
# - Echo from E:\CNT\data\echo_signal.csv (or CNT_LAB_DIR\data), else demo
# - Saves scoreboard + best field under ...\artifacts\cnt_physical_glyph_hunt\{RUN_ID}

import numpy as np, os, json, time, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — Single Cell v0.5 (PASS Razor)")

# ---------- FFT helpers ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return np.meshgrid(kx, ky, kz, indexing='ij')

def _project_solenoidal(Bx, By, Bz, spacings):
    # Project B̂(k) onto plane ⟂ k to enforce ∇·B = 0
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh_proj = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh_proj = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh_proj = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh_proj)), np.real(ifftn(Byh_proj)), np.real(ifftn(Bzh_proj))

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    Nx, Ny, Nz = Bx.shape
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    return np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))

def helicity(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

def smooth_field(Bx, By, Bz, spacings, alpha=0.0):
    if alpha <= 0.0:
        return Bx, By, Bz
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    k2 = kx*kx + ky*ky + kz*kz
    kmax2 = (np.abs(kx).max()**2 + np.abs(ky).max()**2 + np.abs(kz).max()**2)
    filt = np.exp(-alpha * (k2/(kmax2 + 1e-12)))
    return (np.real(ifftn(fftn(Bx)*filt)),
            np.real(ifftn(fftn(By)*filt)),
            np.real(ifftn(fftn(Bz)*filt)))

# ---------- Hopf tools ----------
def _grad(field, spacings):
    dx, dy, dz = spacings
    return (np.gradient(field, dx, axis=0),
            np.gradient(field, dy, axis=1),
            np.gradient(field, dz, axis=2))

def curl_real(Fx, Fy, Fz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(Fy, dz, axis=2)
    dFz_dy = np.gradient(Fz, dy, axis=1)
    dFz_dx = np.gradient(Fz, dx, axis=0)
    dFx_dz = np.gradient(Fx, dz, axis=2)
    dFx_dy = np.gradient(Fx, dy, axis=1)
    dFy_dx = np.gradient(Fy, dx, axis=0)
    return (dFz_dy - dFy_dz, dFx_dz - dFz_dx, dFy_dx - dFx_dy)

def hopf_map_n_scaled(N=64, L=8.0, s=1.0):
    # Scale core by s (<1 concentrates the Hopf core, improving finite-box integral)
    xs = np.linspace(-L/2, L/2, N, endpoint=False)
    ys = np.linspace(-L/2, L/2, N, endpoint=False)
    zs = np.linspace(-L/2, L/2, N, endpoint=False)
    x, y, z = np.meshgrid(xs, ys, zs, indexing='ij')
    X, Y, Z = x/(s+1e-12), y/(s+1e-12), z/(s+1e-12)
    r2 = X*X + Y*Y + Z*Z
    denom = 1.0 + r2
    u = (2*(X + 1j*Y)) / denom
    v = (1 - r2 + 2j*Z) / denom
    uv_bar = u * np.conjugate(v)
    nx = 2.0 * np.real(uv_bar)
    ny = 2.0 * np.imag(uv_bar)
    nz = np.abs(u)**2 - np.abs(v)**2
    norm = np.sqrt(nx*nx + ny*ny + nz*nz) + 1e-12
    return (nx/norm, ny/norm, nz/norm), (L/N, L/N, L/N)

def b_from_n(nxyz, spacings):
    nx, ny, nz = nxyz
    dnx = _grad(nx, spacings); dny = _grad(ny, spacings); dnz = _grad(nz, spacings)
    dn = [np.stack([dnx[i], dny[i], dnz[i]], axis=0) for i in range(3)]
    nvec = np.stack([nx, ny, nz], axis=0)
    Nx, Ny, Nz = nx.shape
    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0
    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    return [bi/(8*np.pi) for bi in b]

def hopf_proxy(nxyz, spacings):
    # Build A from b(n) via FFT Poisson; return (1/4π^2) ∫ A·(∇×A) dV density
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    # b(n)
    b = b_from_n(nxyz, spacings)
    Ax, Ay, Az = _vector_potential_from_B(b[0], b[1], b[2], Lx, Ly, Lz)
    # curl(A)
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Axh, Ayh, Azh = fftn(Ax), fftn(Ay), fftn(Az)
    i = 1j
    Bxh = i*(ky*Azh - kz*Ayh)
    Byh = i*(kz*Axh - kx*Azh)
    Bzh = i*(kx*Ayh - ky*Axh)
    Bx = np.real(ifftn(Bxh)); By = np.real(ifftn(Byh)); Bz = np.real(ifftn(Bzh))
    integrand = Ax*Bx + Ay*By + Az*Bz
    H = (1.0/(4*np.pi**2)) * np.sum(integrand) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H), float(H/V)

# ---------- Echo ----------
def echo_spectrum(signal, dt):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=dt)
    spec  = (np.abs(np.fft.rfft(signal))**2) / n
    return freqs, spec

# ---------- Locate data / build seed ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
data_dir = root / "data"
field_path = None
for cand in [data_dir/"B_field.npz", Path("E:/CNT/data/B_field.npz")]:
    if cand.exists():
        field_path = cand; break

if field_path:
    d = np.load(field_path)
    Bx0, By0, Bz0 = d["Bx"], d["By"], d["Bz"]
    if all(k in d for k in ["dx","dy","dz"]):
        spacings0 = (float(d["dx"]), float(d["dy"]), float(d["dz"]))
    else:
        N = Bx0.shape[0]; L = 2*np.pi; spacings0 = (L/N, L/N, L/N)
    mode = "npz"
    print(f"[mode] npz → {field_path}")
else:
    mode = "hopfion"
    print("[mode] hopfion")
    N, L = 64, 8.0
    # s will sweep later

# ---------- Echo (shared) ----------
echo_path = None
for cand in [data_dir/"echo_signal.csv", Path("E:/CNT/data/echo_signal.csv")]:
    if cand.exists():
        echo_path = cand; break
use_demo = True
if echo_path and echo_path.exists():
    try:
        arr = np.loadtxt(echo_path, delimiter=",", ndmin=1)
        sig = arr.astype(float); dt = 1.0; use_demo = False
        print(f"[Echo] Loaded {echo_path} (len={len(sig)})")
    except Exception as e:
        print(f"[Echo] load failed → demo. Reason: {e}")
if use_demo:
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
freqs, P = echo_spectrum(sig, dt)
band = (1.6, 1.8)
mask = (freqs>band[0]) & (freqs<band[1])
echo_power_shared = float((P[mask].sum()) / (P.sum()+1e-12))

# ---------- Hopf baseline ----------
def hopf_baseline_for(N, spacings):
    dx, dy, dz = spacings
    Lx, Ly, Lz = dx*N, dy*N, dz*N
    L_base = max(Lx, Ly, Lz)
    (nhx, nhy, nhz), spN = hopf_map_n_scaled(N=N, L=L_base, s=1.0)
    _, h_hopf_base = hopf_proxy((nhx, nhy, nhz), spN)
    return h_hopf_base

if mode=="npz":
    Nbase = Bx0.shape[0]
    h_hopf_base = hopf_baseline_for(Nbase, spacings0)
else:
    # baseline for Hopfion grids (use N,L below; s=1.0)
    spacings_tmp = (L/N, L/N, L/N)
    h_hopf_base = hopf_baseline_for(N, spacings_tmp)

print(f"[Hopf baseline] density={h_hopf_base:.6e}")

# ---------- Sweep ----------
lam_list   = [0.8,1.0,1.2,1.4,1.6,1.8,2.0] if mode=="hopfion" else [0.0]
s_list     = [0.6, 0.8, 1.0] if mode=="hopfion" else [1.0]
alpha_list = [0.0, 0.5]
results = []

def score_combo(Bx,By,Bz,spacings, echo_power, mode):
    # 1) Project to solenoidal
    Bx, By, Bz = _project_solenoidal(Bx, By, Bz, spacings)
    # 2) Helicity
    H, h = helicity(Bx,By,Bz,spacings)
    # 3) Hopf proxy on normalized direction field
    mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    nxB, nyB, nzB = Bx/mag, By/mag, Bz/mag
    _, h_hopf_B = hopf_proxy((nxB, nyB, nzB), spacings)
    Hopf_norm = float(min(1.0, abs(h_hopf_B)/(abs(h_hopf_base)+1e-18)))
    # 4) Scoring
    if mode=="hopfion": w1,w2,w3 = 0.2, 0.6, 0.2
    else:               w1,w2,w3 = 0.4, 0.4, 0.2
    H_norm = float(min(1.0, abs(h)/(1e-2)))
    glyphness = float(w1*H_norm + w2*Hopf_norm + w3*echo_power)
    return dict(H=H, h=h, h_hopf_B=h_hopf_B, Hopf_norm=Hopf_norm, H_norm=H_norm,
                echo=echo_power, glyphness=glyphness, weights=(w1,w2,w3))

for s in s_list:
    if mode=="hopfion":
        (nx,ny,nz), sp = hopf_map_n_scaled(N=N, L=L, s=s)
        b = b_from_n((nx,ny,nz), sp)          # seed
        cb = curl_real(*b, sp)                 # curl(seed)
    for lam in lam_list:
        for alpha in alpha_list:
            if mode=="npz":
                Bx, By, Bz = Bx0.copy(), By0.copy(), Bz0.copy()
                sp_use = spacings0
            else:
                Bx = b[0] + lam*cb[0]
                By = b[1] + lam*cb[1]
                Bz = b[2] + lam*cb[2]
                sp_use = sp
            # Smoothing
            Bx, By, Bz = smooth_field(Bx, By, Bz, sp_use, alpha=alpha)
            met = score_combo(Bx, By, Bz, sp_use, echo_power_shared, mode)
            met.update(mode=mode, lam=float(lam), alpha=float(alpha), s=float(s))
            results.append(met)
            print(f"s={s:.2f} λ={lam:.2f} α={alpha:.2f} → glyphness={met['glyphness']:.3f}  (Hopf_norm={met['Hopf_norm']:.3f}, H_norm={met['H_norm']:.2f})")

# ---------- Pick best, label, save ----------
thr = 0.65
best = max(results, key=lambda r: r['glyphness'])
label = "PHYSICAL-GLYPH:PASS" if (best['glyphness']>=thr and best['Hopf_norm']>=0.20 and best['echo']>=0.20) else "CANDIDATE"

run_root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
run_dir = run_root / "artifacts" / "cnt_physical_glyph_hunt" / time.strftime("%Y%m%d-%H%M%SZ", time.gmtime())
run_dir.mkdir(parents=True, exist_ok=True)

with open(run_dir/"scoreboard.csv","w", newline="") as f:
    wr = csv.writer(f); wr.writerow(["mode","s","lam","alpha","glyphness","Hopf_norm","H_norm","echo","h","h_hopf_B"])
    for r in sorted(results, key=lambda x: x['glyphness'], reverse=True):
        wr.writerow([r['mode'], r['s'], r['lam'], r['alpha'], f"{r['glyphness']:.6f}", f"{r['Hopf_norm']:.6f}",
                     f"{r['H_norm']:.6f}", f"{r['echo']:.6f}", f"{r['h']:.6e}", f"{r['h_hopf_B']:.6e}"])

# Reconstruct best field
if mode=="npz":
    BxB, ByB, BzB = Bx0.copy(), By0.copy(), Bz0.copy()
    sp_best = spacings0
else:
    (nx,ny,nz), sp_best = hopf_map_n_scaled(N=N, L=L, s=best['s'])
    b = b_from_n((nx,ny,nz), sp_best); cb = curl_real(*b, sp_best)
    BxB = b[0] + best['lam']*cb[0]; ByB = b[1] + best['lam']*cb[1]; BzB = b[2] + best['lam']*cb[2]
BxB, ByB, BzB = smooth_field(BxB, ByB, BzB, sp_best, alpha=best['alpha'])
BxB, ByB, BzB = _project_solenoidal(BxB, ByB, BzB, sp_best)

# Save best field + echo + summary
np.savez_compressed(run_dir/"best_field.npz", Bx=BxB, By=ByB, Bz=BzB, dx=sp_best[0], dy=sp_best[1], dz=sp_best[2])
np.savetxt(run_dir/"echo_spectrum.csv", np.column_stack([freqs, P]), delimiter=",", header="freq,power", comments="")

summary = {
    "mode": mode,
    "echo": {"band": list(band), "power_norm": echo_power_shared, "source": "file" if not use_demo else "demo"},
    "hopf_baseline_density": h_hopf_base,
    "best": {k:(float(v) if isinstance(v,(int,float,np.floating)) else v) for k,v in best.items()},
    "thresholds": {"glyphness": thr, "min_Hopf_norm": 0.20, "min_echo": 0.20},
    "label": "PHYSICAL-GLYPH:PASS" if label=="PHYSICAL-GLYPH:PASS" else "CANDIDATE",
    "run_dir": str(run_dir)
}
with open(run_dir/"summary.json","w") as fp:
    json.dump(summary, fp, indent=2)

print(f"TOP → s={best['s']:.2f} λ={best['lam']:.2f} α={best['alpha']:.2f}  glyphness={best['glyphness']:.3f}  Hopf_norm={best['Hopf_norm']:.3f} → {label}")
print("Saved →", run_dir/"summary.json")


CNT Physical Glyph Hunt — Single Cell v0.5 (PASS Razor)
[mode] hopfion
[Hopf baseline] density=4.506476e-05
s=0.60 λ=0.80 α=0.00 → glyphness=0.669  (Hopf_norm=0.491, H_norm=1.00)
s=0.60 λ=0.80 α=0.50 → glyphness=0.667  (Hopf_norm=0.489, H_norm=1.00)
s=0.60 λ=1.00 α=0.00 → glyphness=0.684  (Hopf_norm=0.517, H_norm=1.00)
s=0.60 λ=1.00 α=0.50 → glyphness=0.683  (Hopf_norm=0.515, H_norm=1.00)
s=0.60 λ=1.20 α=0.00 → glyphness=0.697  (Hopf_norm=0.539, H_norm=1.00)
s=0.60 λ=1.20 α=0.50 → glyphness=0.696  (Hopf_norm=0.536, H_norm=1.00)
s=0.60 λ=1.40 α=0.00 → glyphness=0.708  (Hopf_norm=0.557, H_norm=1.00)
s=0.60 λ=1.40 α=0.50 → glyphness=0.707  (Hopf_norm=0.554, H_norm=1.00)
s=0.60 λ=1.60 α=0.00 → glyphness=0.718  (Hopf_norm=0.574, H_norm=1.00)
s=0.60 λ=1.60 α=0.50 → glyphness=0.716  (Hopf_norm=0.571, H_norm=1.00)
s=0.60 λ=1.80 α=0.00 → glyphness=0.727  (Hopf_norm=0.589, H_norm=1.00)
s=0.60 λ=1.80 α=0.50 → glyphness=0.725  (Hopf_norm=0.586, H_norm=1.00)
s=0.60 λ=2.00 α=0.00 → glyphness=0.736  

In [9]:
# CNT Physical Glyph Hunt — Single Cell v0.6 (Null Razor: significance test)
# Purpose: quantify how unlikely your Hopf topology is vs. matched-spectrum nulls.
# - Auto-loads latest .../cnt_physical_glyph_hunt/*/best_field.npz
# - Builds nulls by randomizing Fourier phases (rfftn/irfftn), preserves power spectrum
# - Projects to solenoidal, computes Hopf_norm for each null, returns p-value
# - Saves results (CSV + JSON) alongside your run

import numpy as np, os, json, time, csv, glob
from numpy.fft import rfftn, irfftn, fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — v0.6 Null Razor")

# ---------- helpers (must match your earlier pipeline) ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return np.meshgrid(kx, ky, kz, indexing='ij')

def _project_solenoidal(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh_proj = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh_proj = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh_proj = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh_proj)), np.real(ifftn(Byh_proj)), np.real(ifftn(Bzh_proj))

def _vector_potential_from_B(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    return np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))

def helicity_density(Bx, By, Bz, spacings):
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, spacings)
    dx, dy, dz = spacings
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = (Bx.shape[0]*dx)*(By.shape[1]*dy)*(Bz.shape[2]*dz)
    return float(H / V)

def _grad(field, spacings):
    dx, dy, dz = spacings
    return (np.gradient(field, dx, axis=0),
            np.gradient(field, dy, axis=1),
            np.gradient(field, dz, axis=2))

def b_from_n(nxyz, spacings):
    nx, ny, nz = nxyz
    dnx = _grad(nx, spacings); dny = _grad(ny, spacings); dnz = _grad(nz, spacings)
    dn = [np.stack([dnx[i], dny[i], dnz[i]], axis=0) for i in range(3)]
    nvec = np.stack([nx, ny, nz], axis=0)
    Nx, Ny, Nz = nx.shape
    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0
    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    return [bi/(8*np.pi) for bi in b]

def hopf_proxy_density(nxyz, spacings):
    # (1/4π^2) ∫ A · (∇×A) / V
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    b = b_from_n(nxyz, spacings)
    # Build A from b via FFT Poisson
    kx, ky, kz = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(b[0]), fftn(b[1]), fftn(b[2])
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax, Ay, Az = np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))
    # curl(A)
    Bxh = i*(ky*Azh - kz*Ayh); Byh = i*(kz*Axh - kx*Azh); Bzh = i*(kx*Ayh - ky*Axh)
    Bx, By, Bz = np.real(ifftn(Bxh)), np.real(ifftn(Byh)), np.real(ifftn(Bzh))
    integrand = Ax*Bx + Ay*By + Az*Bz
    H = (1.0/(4*np.pi**2)) * np.sum(integrand) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H / V)

def hopf_norm_from_B(Bx, By, Bz, spacings, h_hopf_base):
    mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    nxB, nyB, nzB = Bx/mag, By/mag, Bz/mag
    h_hopf_B = hopf_proxy_density((nxB, nyB, nzB), spacings)
    Hopf_norm = float(min(1.0, abs(h_hopf_B)/(abs(h_hopf_base)+1e-18)))
    return h_hopf_B, Hopf_norm

def phase_randomize_real(B):
    # rfftn/irfftn keeps Hermitian symmetry consistent; multiply by random phases
    F = rfftn(B)
    phases = np.exp(1j*np.random.uniform(0, 2*np.pi, F.shape))
    Fr = np.abs(F) * phases
    return irfftn(Fr, s=B.shape).astype(np.float64)

# ---------- locate latest best_field ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
cands = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert len(cands)>0, f"No runs found in {hunt_dir}"
run_dir = cands[0]
best_path = run_dir / "best_field.npz"
if not best_path.exists():
    # fall back to field_small.npz if present
    best_path = run_dir / "field_small.npz"
assert best_path.exists(), f"No field file found at {best_path}"
print("Loaded field →", best_path)

d = np.load(best_path)
Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
dx = float(d.get("dx", 2*np.pi/Bx.shape[0]))
dy = float(d.get("dy", 2*np.pi/By.shape[1]))
dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)

# Hopf baseline for this grid
N = Bx.shape[0]
Lx, Ly, Lz = dx*N, dy*N, dz*N
L_base = max(Lx,Ly,Lz)

# Build classic Hopf-map n for baseline
xs = np.linspace(-L_base/2, L_base/2, N, endpoint=False)
x, y, z = np.meshgrid(xs, xs, xs, indexing='ij')
r2 = x*x + y*y + z*z
den = 1.0 + r2
u = (2*(x + 1j*y)) / den
v = (1 - r2 + 2j*z) / den
uv = u*np.conjugate(v)
nhx = 2.0*np.real(uv); nhy = 2.0*np.imag(uv); nhz = np.abs(u)**2 - np.abs(v)**2
norm = np.sqrt(nhx*nhx + nhy*nhy + nhz*nhz) + 1e-12
nhx, nhy, nhz = nhx/norm, nhy/norm, nhz/norm
spN = (L_base/N, L_base/N, L_base/N)
h_hopf_base = hopf_proxy_density((nhx,nhy,nhz), spN)
print(f"[baseline] h_hopf_base={h_hopf_base:.6e}")

# Project original to solenoidal and compute original metrics
Bx0, By0, Bz0 = _project_solenoidal(Bx, By, Bz, sp)
h0 = helicity_density(Bx0, By0, Bz0, sp)
h_hopf0, Hopf_norm0 = hopf_norm_from_B(Bx0, By0, Bz0, sp, h_hopf_base)
print(f"[orig] h={h0:.3e}  h_hopf={h_hopf0:.3e}  Hopf_norm={Hopf_norm0:.3f}")

# ---------- Nulls ----------
NNULL = 64   # adjust (128+ for stronger cert)
rng = np.random.default_rng(42)
hopf_null = []

for r in range(NNULL):
    # Phase-randomize each component, then project to solenoidal
    Bxr = phase_randomize_real(Bx)
    Byr = phase_randomize_real(By)
    Bzr = phase_randomize_real(Bz)
    Bxr, Byr, Bzr = _project_solenoidal(Bxr, Byr, Bzr, sp)
    _, Hopf_norm_r = hopf_norm_from_B(Bxr, Byr, Bzr, sp, h_hopf_base)
    hopf_null.append(Hopf_norm_r)
    if (r+1) % 8 == 0:
        print(f"  null {r+1}/{NNULL}  Hopf_norm={Hopf_norm_r:.3f}")

hopf_null = np.array(hopf_null, dtype=float)
p_value = float((np.sum(hopf_null >= Hopf_norm0) + 1) / (NNULL + 1))  # one-sided, +1 for continuity

# ---------- Save ----------
out = {
    "field": str(best_path),
    "NNULL": NNULL,
    "original": {"h": h0, "h_hopf": h_hopf0, "Hopf_norm": Hopf_norm0},
    "baseline": {"h_hopf_base": float(h_hopf_base)},
    "nulls": {"Hopf_norm": hopf_null.tolist()},
    "p_value_one_sided": p_value,
}
with open(run_dir/"null_significance.json","w") as f:
    json.dump(out, f, indent=2)

with open(run_dir/"null_hopf_norm.csv","w", newline="") as f:
    wr = csv.writer(f); wr.writerow(["Hopf_norm_null"])
    for v in hopf_null: wr.writerow([f"{v:.6f}"])

print(f"[RESULT] Hopf_norm={Hopf_norm0:.3f}  vs null median={np.median(hopf_null):.3f}  → p={p_value:.4f}")
print("Saved →", run_dir/"null_significance.json")


CNT Physical Glyph Hunt — v0.6 Null Razor
Loaded field → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\best_field.npz
[baseline] h_hopf_base=4.506476e-05
[orig] h=9.769e-02  h_hopf=2.716e-05  Hopf_norm=0.603


C:\Users\caleb\AppData\Local\Temp\ipykernel_31264\256026709.py:127: DeprecationWarning: `axes` should not be `None` if `s` is not `None` (Deprecated in NumPy 2.0). In a future version of NumPy, this will raise an error and `s[i]` will correspond to the size along the transformed axis specified by `axes[i]`. To retain current behaviour, pass a sequence [0, ..., k-1] to `axes` for an array of dimension k.
  return irfftn(Fr, s=B.shape).astype(np.float64)


  null 8/64  Hopf_norm=1.000
  null 16/64  Hopf_norm=0.323
  null 24/64  Hopf_norm=0.006
  null 32/64  Hopf_norm=0.052
  null 40/64  Hopf_norm=0.119
  null 48/64  Hopf_norm=0.035
  null 56/64  Hopf_norm=0.739
  null 64/64  Hopf_norm=0.052
[RESULT] Hopf_norm=0.603  vs null median=0.301  → p=0.3385
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\null_significance.json


In [10]:
# CNT Physical Glyph Hunt — Single Cell v0.7 (Spectral-Rotation Nulls)
# Goal: a stronger null. We rotate the complex B̂(k) within the plane ⟂ k, for each k,
# preserving |B̂(k)|, real-field Hermitian symmetry, and ∇·B=0 after projection.
# Then compute Hopf_norm null distribution and a p-value (one-sided).
#
# Auto-loads latest ...\cnt_physical_glyph_hunt\*\best_field.npz
# Saves: null_significance_v07.json, null_hopf_norm_v07.csv

import numpy as np, os, json, time, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — v0.7 Spectral-Rotation Nulls")

# ---------- helpers ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return kx, ky, kz

def _project_solenoidal(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh_p = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh_p = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh_p = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh_p)), np.real(ifftn(Byh_p)), np.real(ifftn(Bzh_p))

def _vector_potential_from_B(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    return np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))

def helicity_density(Bx, By, Bz, spacings):
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, spacings)
    dx, dy, dz = spacings
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = (Bx.shape[0]*dx)*(By.shape[1]*dy)*(Bz.shape[2]*dz)
    return float(H / V)

def _grad(field, spacings):
    dx, dy, dz = spacings
    return (np.gradient(field, dx, axis=0),
            np.gradient(field, dy, axis=1),
            np.gradient(field, dz, axis=2))

def b_from_n(nxyz, spacings):
    nx, ny, nz = nxyz
    dnx, dny, dnz = _grad(nx, spacings), _grad(ny, spacings), _grad(nz, spacings)
    dn = [np.stack([dnx[i], dny[i], dnz[i]], axis=0) for i in range(3)]
    nvec = np.stack([nx, ny, nz], axis=0)
    Nx, Ny, Nz = nx.shape
    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0
    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    return [bi/(8*np.pi) for bi in b]

def hopf_proxy_density(nxyz, spacings):
    # (1/4π^2) ∫ A·(∇×A) / V  with ∇×A = b(n)
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    b = b_from_n(nxyz, spacings)
    # vector potential from b via FFT
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    i = 1j
    Bxh, Byh, Bzh = fftn(b[0]), fftn(b[1]), fftn(b[2])
    k2 = kx*kx + ky*ky + kz*kz
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax, Ay, Az = np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))
    # curl(A)
    Bxh = i*(ky*Azh - kz*Ayh); Byh = i*(kz*Axh - kx*Azh); Bzh = i*(kx*Ayh - ky*Axh)
    Bx, By, Bz = np.real(ifftn(Bxh)), np.real(ifftn(Byh)), np.real(ifftn(Bzh))
    H = (1.0/(4*np.pi**2)) * np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H / V)

def hopf_norm_from_B(Bx, By, Bz, spacings, h_hopf_base):
    mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    nxB, nyB, nzB = Bx/mag, By/mag, Bz/mag
    h_hopf_B = hopf_proxy_density((nxB, nyB, nzB), spacings)
    Hopf_norm = float(min(1.0, abs(h_hopf_B)/(abs(h_hopf_base)+1e-18)))
    return h_hopf_B, Hopf_norm

# ---------- spectral-rotation null (preserves |B̂(k)|, Hermitian) ----------
def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)

    def pos_half(ix, iy, iz):
        # Unique half-space: kz>0 or (kz==0 and ky>0) or (kz==0 and ky==0 and kx>0)
        kz = kzv[iz]; ky = kyv[iy]; kx = kxv[ix]
        if kz > 0: return True
        if kz < 0: return False
        if ky > 0: return True
        if ky < 0: return False
        return kx > 0

    for ix in range(Nx):
        kx = kxv[ix]
        for iy in range(Ny):
            ky = kyv[iy]
            for iz in range(Nz):
                kz = kzv[iz]
                if kx==0 and ky==0 and kz==0:
                    # DC: copy through
                    Bxh_new[ix,iy,iz] = Bxh[ix,iy,iz]
                    Byh_new[ix,iy,iz] = Byh[ix,iy,iz]
                    Bzh_new[ix,iy,iz] = Bzh[ix,iy,iz]
                    continue
                if not pos_half(ix,iy,iz):
                    # Will be set as conjugate of its positive partner
                    continue

                # Build orthonormal basis {e1, e2} in plane ⟂ k
                k = np.array([kx,ky,kz], dtype=float)
                k_norm = np.linalg.norm(k)
                if k_norm == 0:
                    continue
                k_hat = k / k_norm
                ref = np.array([0.0,0.0,1.0])
                if abs(np.dot(k_hat, ref)) > 0.99:
                    ref = np.array([1.0,0.0,0.0])
                e1 = np.cross(ref, k_hat); n1 = np.linalg.norm(e1)+1e-15; e1 /= n1
                e2 = np.cross(k_hat, e1);   n2 = np.linalg.norm(e2)+1e-15; e2 /= n2

                # Current spectral vector at k
                Bk = np.array([Bxh[ix,iy,iz], Byh[ix,iy,iz], Bzh[ix,iy,iz]])
                c1 = e1[0]*Bk[0] + e1[1]*Bk[1] + e1[2]*Bk[2]
                c2 = e2[0]*Bk[0] + e2[1]*Bk[1] + e2[2]*Bk[2]

                # Random rotation in plane
                theta = rng.uniform(0, 2*np.pi)
                ct, st = np.cos(theta), np.sin(theta)
                c1p = ct*c1 - st*c2
                c2p = st*c1 + ct*c2
                Bkp = e1*c1p + e2*c2p

                # Assign k and -k to maintain Hermitian symmetry
                Bxh_new[ix,iy,iz] = Bkp[0]; Byh_new[ix,iy,iz] = Bkp[1]; Bzh_new[ix,iy,iz] = Bkp[2]
                ixn, iyn, izn = (-ix) % Nx, (-iy) % Ny, (-iz) % Nz
                Bxh_new[ixn,iyn,izn] = np.conjugate(Bkp[0])
                Byh_new[ixn,iyn,izn] = np.conjugate(Bkp[1])
                Bzh_new[ixn,iyn,izn] = np.conjugate(Bkp[2])

    # Fill any untouched Nyquist planes with originals (they are self-conjugate)
    mask_unset = (Bxh_new==0) & (Byh_new==0) & (Bzh_new==0)
    Bxh_new[mask_unset] = Bxh[mask_unset]
    Byh_new[mask_unset] = Byh[mask_unset]
    Bzh_new[mask_unset] = Bzh[mask_unset]

    Brx = np.real(ifftn(Bxh_new)); Bry = np.real(ifftn(Byh_new)); Brz = np.real(ifftn(Bzh_new))
    # Safety: enforce divergence-free again
    Brx, Bry, Brz = _project_solenoidal(Brx, Bry, Brz, spacings)
    return Brx, Bry, Brz

# ---------- load latest best_field ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
best_path = run_dir / "best_field.npz"
if not best_path.exists():
    best_path = run_dir / "field_small.npz"
assert best_path.exists(), f"No field .npz found in {run_dir}"

print("Loaded field →", best_path)
d = np.load(best_path)
Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)

# ---------- Hopf baseline for this grid ----------
N = Bx.shape[0]
Lx, Ly, Lz = dx*N, dy*N, dz*N
L_base = max(Lx, Ly, Lz)
xs = np.linspace(-L_base/2, L_base/2, N, endpoint=False)
x, y, z = np.meshgrid(xs, xs, xs, indexing='ij')
r2 = x*x + y*y + z*z; den = 1.0 + r2
u = (2*(x + 1j*y)) / den; v = (1 - r2 + 2j*z) / den
uv = u*np.conjugate(v)
nhx = 2.0*np.real(uv); nhy = 2.0*np.imag(uv); nhz = np.abs(u)**2 - np.abs(v)**2
norm = np.sqrt(nhx*nhx + nhy*nhy + nhz*nhz) + 1e-12
nhx, nhy, nhz = nhx/norm, nhy/norm, nhz/norm
spN = (L_base/N, L_base/N, L_base/N)

def hopf_proxy_density_quick(nx,ny,nz,spc):
    return hopf_proxy_density((nx,ny,nz), spc)

h_hopf_base = hopf_proxy_density_quick(nhx,nhy,nhz, spN)
print(f"[baseline] h_hopf_base={h_hopf_base:.6e}")

# ---------- original metrics ----------
Bx0, By0, Bz0 = _project_solenoidal(Bx, By, Bz, sp)
h0 = helicity_density(Bx0, By0, Bz0, sp)
h_hopf0, Hopf_norm0 = hopf_norm_from_B(Bx0, By0, Bz0, sp, h_hopf_base)
print(f"[orig] h={h0:.3e}  h_hopf={h_hopf0:.3e}  Hopf_norm={Hopf_norm0:.3f}")

# ---------- spectral-rotation nulls ----------
NNULL = 64  # Try 256+ for stronger certainty
rng = np.random.default_rng(12345)
hopf_null = []
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx0, By0, Bz0, sp, rng)
    _, Hopf_norm_r = hopf_norm_from_B(Bxr, Byr, Bzr, sp, h_hopf_base)
    hopf_null.append(Hopf_norm_r)
    if (r+1) % 8 == 0:
        print(f"  null {r+1}/{NNULL}  Hopf_norm={Hopf_norm_r:.3f}")

hopf_null = np.array(hopf_null, dtype=float)
p_value = float((np.sum(hopf_null >= Hopf_norm0) + 1) / (NNULL + 1))  # one-sided, continuity correction

# ---------- save ----------
out = {
    "field": str(best_path),
    "NNULL": NNULL,
    "original": {"h": h0, "h_hopf": h_hopf0, "Hopf_norm": Hopf_norm0},
    "baseline": {"h_hopf_base": float(h_hopf_base)},
    "nulls": {"Hopf_norm": hopf_null.tolist(), "median": float(np.median(hopf_null))},
    "p_value_one_sided": p_value,
}
with open(run_dir/"null_significance_v07.json","w") as f:
    json.dump(out, f, indent=2)
with open(run_dir/"null_hopf_norm_v07.csv","w", newline="") as f:
    wr = csv.writer(f); wr.writerow(["Hopf_norm_null"])
    for v in hopf_null: wr.writerow([f"{v:.6f}"])

print(f"[RESULT v0.7] Hopf_norm={Hopf_norm0:.3f} vs null median={np.median(hopf_null):.3f}  → p={p_value:.4f}")
print("Saved →", run_dir/"null_significance_v07.json")


CNT Physical Glyph Hunt — v0.7 Spectral-Rotation Nulls
Loaded field → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\best_field.npz
[baseline] h_hopf_base=4.506476e-05
[orig] h=9.769e-02  h_hopf=2.716e-05  Hopf_norm=0.603
  null 8/64  Hopf_norm=1.000
  null 16/64  Hopf_norm=0.788
  null 24/64  Hopf_norm=0.154
  null 32/64  Hopf_norm=0.993
  null 40/64  Hopf_norm=1.000
  null 48/64  Hopf_norm=1.000
  null 56/64  Hopf_norm=0.056
  null 64/64  Hopf_norm=1.000
[RESULT v0.7] Hopf_norm=0.603 vs null median=1.000  → p=0.6923
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\null_significance_v07.json


In [11]:
# CNT Physical Glyph Hunt — Single Cell v0.8
# Compactified Hopf + Spectral-Rotation Nulls (no baseline saturation)
# - Auto-loads latest .../cnt_physical_glyph_hunt/*/best_field.npz
# - Builds a boundary-compactified unit field n(x) from B (so Hopf integral is well-behaved)
# - Generates nulls by rotating B̂(k) in planes ⟂ k (preserves |B̂(k)| & real-field symmetry),
#   re-projects to ∇·B = 0, then recomputes Hopf on compactified n(x)
# - Scores glyphness from helicity, echo, and Hopf percentile; saves artifacts next to the run.

import numpy as np, os, json, time, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — v0.8 (Compact Hopf + Strong Nulls)")

# ---------- FFT helpers ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return kx, ky, kz

def _project_solenoidal(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh_p = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh_p = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh_p = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh_p)), np.real(ifftn(Byh_p)), np.real(ifftn(Bzh_p))

def _vector_potential_from_B(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    return np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))

def helicity_density(Bx, By, Bz, spacings):
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, spacings)
    dx, dy, dz = spacings
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = (Bx.shape[0]*dx)*(By.shape[1]*dy)*(Bz.shape[2]*dz)
    return float(H / V)

# ---------- Topology: Hopf proxy ----------
def _grad(field, spacings):
    dx, dy, dz = spacings
    return (np.gradient(field, dx, axis=0),
            np.gradient(field, dy, axis=1),
            np.gradient(field, dz, axis=2))

def b_from_n(nxyz, spacings):
    nx, ny, nz = nxyz
    dnx, dny, dnz = _grad(nx, spacings), _grad(ny, spacings), _grad(nz, spacings)
    dn = [np.stack([dnx[i], dny[i], dnz[i]], axis=0) for i in range(3)]
    nvec = np.stack([nx, ny, nz], axis=0)
    Nx, Ny, Nz = nx.shape
    F = np.zeros((3,3,Nx,Ny,Nz), dtype=np.float64)
    for j in range(3):
        for k in range(3):
            cross = np.cross(dn[j], dn[k], axis=0)
            F[j,k] = np.sum(nvec * cross, axis=0)
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1.0
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1.0
    b = [np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz)), np.zeros((Nx,Ny,Nz))]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if eps[i,j,k] != 0:
                    b[i] += eps[i,j,k] * F[j,k]
    return [bi/(8*np.pi) for bi in b]

def hopf_proxy_density(nxyz, spacings):
    # (1/4π^2) ∫ A·(∇×A)/V with ∇×A = b(n)
    nx, ny, nz = nxyz
    dx, dy, dz = spacings
    Nx, Ny, Nz = nx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    b = b_from_n(nxyz, spacings)
    # A from b via FFT
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    i = 1j
    Bxh, Byh, Bzh = fftn(b[0]), fftn(b[1]), fftn(b[2])
    k2 = kx*kx + ky*ky + kz*kz
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax, Ay, Az = np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))
    # curl(A)
    Bxh = i*(ky*Azh - kz*Ayh); Byh = i*(kz*Axh - kx*Azh); Bzh = i*(kx*Ayh - ky*Axh)
    Bx, By, Bz = np.real(ifftn(Bxh)), np.real(ifftn(Byh)), np.real(ifftn(Bzh))
    H = (1.0/(4*np.pi**2)) * np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H / V)

# ---------- Boundary compactification ----------
def compactified_n_from_B(Bx, By, Bz, spacings, gamma=0.8, p=4):
    # Make n ~ constant at boundaries: n = norm( w(r)*B̂ + (1-w) * ẑ )
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    xs = np.linspace(-Lx/2, Lx/2, Nx, endpoint=False)
    ys = np.linspace(-Ly/2, Ly/2, Ny, endpoint=False)
    zs = np.linspace(-Lz/2, Lz/2, Nz, endpoint=False)
    x, y, z = np.meshgrid(xs, ys, zs, indexing='ij')
    # normalized radius (ellipsoidal)
    r = np.sqrt((x/(Lx/2))**2 + (y/(Ly/2))**2 + (z/(Lz/2))**2)
    w = np.exp(- (r/gamma)**p)

    mag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    nx, ny, nz = Bx/mag, By/mag, Bz/mag
    # blend toward ẑ at boundary
    nx_c = w*nx + (1-w)*0.0
    ny_c = w*ny + (1-w)*0.0
    nz_c = w*nz + (1-w)*1.0

    norm = np.sqrt(nx_c*nx_c + ny_c*ny_c + nz_c*nz_c) + 1e-18
    return (nx_c/norm, ny_c/norm, nz_c/norm)

# ---------- Spectral-rotation nulls ----------
def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)

    def pos_half(ix, iy, iz):
        kz = kzv[iz]; ky = kyv[iy]; kx = kxv[ix]
        if kz > 0: return True
        if kz < 0: return False
        if ky > 0: return True
        if ky < 0: return False
        return kx > 0

    for ix in range(Nx):
        kx = kxv[ix]
        for iy in range(Ny):
            ky = kyv[iy]
            for iz in range(Nz):
                kz = kzv[iz]
                if kx==0 and ky==0 and kz==0:
                    Bxh_new[ix,iy,iz] = Bxh[ix,iy,iz]
                    Byh_new[ix,iy,iz] = Byh[ix,iy,iz]
                    Bzh_new[ix,iy,iz] = Bzh[ix,iy,iz]
                    continue
                if not pos_half(ix,iy,iz):
                    continue
                k = np.array([kx,ky,kz], dtype=float)
                k_norm = np.linalg.norm(k)
                if k_norm == 0: continue
                k_hat = k / k_norm
                ref = np.array([0.0,0.0,1.0]) if abs(k_hat[2])<0.99 else np.array([1.0,0.0,0.0])
                e1 = np.cross(ref, k_hat); e1 /= (np.linalg.norm(e1)+1e-15)
                e2 = np.cross(k_hat, e1);   e2 /= (np.linalg.norm(e2)+1e-15)

                Bk = np.array([Bxh[ix,iy,iz], Byh[ix,iy,iz], Bzh[ix,iy,iz]])
                c1 = e1 @ Bk; c2 = e2 @ Bk
                theta = rng.uniform(0, 2*np.pi)
                ct, st = np.cos(theta), np.sin(theta)
                c1p, c2p = ct*c1 - st*c2, st*c1 + ct*c2
                Bkp = e1*c1p + e2*c2p

                Bxh_new[ix,iy,iz] = Bkp[0]; Byh_new[ix,iy,iz] = Bkp[1]; Bzh_new[ix,iy,iz] = Bkp[2]
                ixn, iyn, izn = (-ix)%Nx, (-iy)%Ny, (-iz)%Nz
                Bxh_new[ixn,iyn,izn] = np.conjugate(Bkp[0])
                Byh_new[ixn,iyn,izn] = np.conjugate(Bkp[1])
                Bzh_new[ixn,iyn,izn] = np.conjugate(Bkp[2])

    mask_unset = (Bxh_new==0) & (Byh_new==0) & (Bzh_new==0)
    Bxh_new[mask_unset] = Bxh[mask_unset]
    Byh_new[mask_unset] = Byh[mask_unset]
    Bzh_new[mask_unset] = Bzh[mask_unset]

    Brx = np.real(ifftn(Bxh_new)); Bry = np.real(ifftn(Byh_new)); Brz = np.real(ifftn(Bzh_new))
    Brx, Bry, Brz = _project_solenoidal(Brx, Bry, Brz, spacings)
    return Brx, Bry, Brz

# ---------- Load latest best field ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
best_path = run_dir / "best_field.npz"
if not best_path.exists():
    best_path = run_dir / "field_small.npz"
assert best_path.exists(), f"No field .npz found in {run_dir}"
print("Loaded field →", best_path)

d = np.load(best_path)
Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)

# ---------- Core metrics on original ----------
Bx0, By0, Bz0 = _project_solenoidal(Bx, By, Bz, sp)
h_helicity = helicity_density(Bx0, By0, Bz0, sp)

# Compactify boundary → n(x)
nx0, ny0, nz0 = compactified_n_from_B(Bx0, By0, Bz0, sp, gamma=0.8, p=4)
hopf_raw = abs(hopf_proxy_density((nx0,ny0,nz0), sp))
print(f"[orig] helicity_density={h_helicity:.3e}  Hopf_raw={hopf_raw:.6e}")

# ---------- Echo (reuse if present) ----------
echo_csv = run_dir / "echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    band = (1.6, 1.8)
    mask = (freqs>band[0]) & (freqs<band[1])
    echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
else:
    # safe fallback demo
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
    freqs = np.fft.rfftfreq(len(sig), d=dt)
    P = np.abs(np.fft.rfft(sig))**2 / len(sig)
    band = (1.6, 1.8)
    echo_power = float((P[(freqs>band[0]) & (freqs<band[1])].sum()) / (P.sum()+1e-12))
print(f"[echo] power_norm={echo_power:.3f}  band={band}")

# ---------- Nulls (spectral rotation + compactification) ----------
NNULL = 128  # bump to 256+ if you want tighter p-values
rng = np.random.default_rng(20251106)
null_vals = []
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx0, By0, Bz0, sp, rng)
    nxr, nyr, nzr = compactified_n_from_B(Bxr, Byr, Bzr, sp, gamma=0.8, p=4)
    hopf_r = abs(hopf_proxy_density((nxr,nyr,nzr), sp))
    null_vals.append(hopf_r)
    if (r+1)%16==0:
        print(f"  null {r+1}/{NNULL}  Hopf_raw={hopf_r:.6e}")

null_vals = np.array(null_vals, dtype=float)
# one-sided p: fraction of nulls >= observed
p_value = float((np.sum(null_vals >= hopf_raw) + 1) / (NNULL + 1))
# percentile score to feed glyphness
hopf_percentile = float((np.sum(null_vals <= hopf_raw) + 0.5) / (NNULL + 1))  # ~CDF

# ---------- Scoring & save ----------
H_norm = float(min(1.0, abs(h_helicity)/(1e-2)))
wH, wP, wE = 0.35, 0.30, 0.35  # helicity, Hopf_percentile, echo
glyphness = float(wH*H_norm + wP*hopf_percentile + wE*echo_power)
label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.65 and hopf_percentile>=0.80 and echo_power>=0.20) else "CANDIDATE"

out = {
    "field": str(best_path),
    "helicity_density": h_helicity,
    "hopf_raw": hopf_raw,
    "nulls": {
        "NNULL": NNULL,
        "hopf_raw_vals": null_vals.tolist(),
        "median": float(np.median(null_vals)),
        "p_value_one_sided": p_value,
        "percentile": hopf_percentile
    },
    "echo": {"power_norm": echo_power, "band": list(band)},
    "score": {
        "glyphness": glyphness,
        "weights": {"H": wH, "Hopf_percentile": wP, "echo": wE},
        "H_norm": H_norm,
        "Hopf_percentile": hopf_percentile,
        "label": label
    }
}

with open(run_dir/"null_significance_v08.json","w") as f:
    json.dump(out, f, indent=2)
with open(run_dir/"null_hopf_raw_v08.csv","w", newline="") as f:
    wr = csv.writer(f); wr.writerow(["Hopf_raw_null"])
    for v in null_vals: wr.writerow([f"{v:.8e}"])

print(f"[RESULT v0.8] Hopf_raw={hopf_raw:.6e}  null_median={np.median(null_vals):.6e}  p={p_value:.4f}  percentile={hopf_percentile:.3f}")
print(f"[CNT] glyphness={glyphness:.3f} (H_norm={H_norm:.2f}, Hopf%={hopf_percentile:.2f}, echo={echo_power:.2f}) → {label}")
print("Saved →", run_dir/"null_significance_v08.json")


CNT Physical Glyph Hunt — v0.8 (Compact Hopf + Strong Nulls)
Loaded field → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\best_field.npz
[orig] helicity_density=9.769e-02  Hopf_raw=2.395709e-05
[echo] power_norm=0.870  band=(1.6, 1.8)
  null 16/128  Hopf_raw=3.073390e-06
  null 32/128  Hopf_raw=3.705405e-05
  null 48/128  Hopf_raw=2.981397e-05
  null 64/128  Hopf_raw=2.631087e-05
  null 80/128  Hopf_raw=9.091189e-06
  null 96/128  Hopf_raw=2.797651e-06
  null 112/128  Hopf_raw=1.434092e-05
  null 128/128  Hopf_raw=5.707473e-05
[RESULT v0.8] Hopf_raw=2.395709e-05  null_median=2.099897e-05  p=0.4496  percentile=0.554
[CNT] glyphness=0.821 (H_norm=1.00, Hopf%=0.55, echo=0.87) → CANDIDATE
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\null_significance_v08.json


In [12]:
# CNT Physical Glyph Hunt — Single Cell v0.9 (LoopLink Cert)
# Field-line knots via Gauss linking: explicit topological proof against strong nulls.
# Artifacts saved next to your latest .../cnt_physical_glyph_hunt/*/best_field.npz

import numpy as np, os, json, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — v0.9 (LoopLink Cert)")

# ---------------- Utilities ----------------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return kx, ky, kz

def _project_solenoidal(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh_p = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh_p = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh_p = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh_p)), np.real(ifftn(Byh_p)), np.real(ifftn(Bzh_p))

def helicity_density(Bx, By, Bz, spacings):
    # A from B via spectral Coulomb gauge; H/V
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax, Ay, Az = np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H / V)

# ---------------- Interpolation & integration (index-space) ----------------
def _tri_interp_periodic(arr, pos):
    # pos in index-space [0,N), periodic; arr shape (N,N,N)
    N = arr.shape[0]
    x, y, z = pos
    i0 = int(np.floor(x)) % N; j0 = int(np.floor(y)) % N; k0 = int(np.floor(z)) % N
    i1 = (i0+1) % N; j1 = (j0+1) % N; k1 = (k0+1) % N
    tx = x - np.floor(x); ty = y - np.floor(y); tz = z - np.floor(z)
    c000 = arr[i0,j0,k0]; c100 = arr[i1,j0,k0]; c010 = arr[i0,j1,k0]; c110 = arr[i1,j1,k0]
    c001 = arr[i0,j0,k1]; c101 = arr[i1,j0,k1]; c011 = arr[i0,j1,k1]; c111 = arr[i1,j1,k1]
    c00 = c000*(1-tx) + c100*tx
    c01 = c001*(1-tx) + c101*tx
    c10 = c010*(1-tx) + c110*tx
    c11 = c011*(1-tx) + c111*tx
    c0 = c00*(1-ty) + c10*ty
    c1 = c01*(1-ty) + c11*ty
    return c0*(1-tz) + c1*tz

def _B_sample_idx(Bx, By, Bz, pos):
    vx = _tri_interp_periodic(Bx, pos)
    vy = _tri_interp_periodic(By, pos)
    vz = _tri_interp_periodic(Bz, pos)
    v = np.array([vx, vy, vz], dtype=float)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rk4_step(pos, dt, Bx, By, Bz):
    N = Bx.shape[0]
    wrap = lambda p: np.mod(p, N)
    k1 = _B_sample_idx(Bx, By, Bz, pos)
    k2 = _B_sample_idx(Bx, By, Bz, wrap(pos + 0.5*dt*k1))
    k3 = _B_sample_idx(Bx, By, Bz, wrap(pos + 0.5*dt*k2))
    k4 = _B_sample_idx(Bx, By, Bz, wrap(pos + dt*k3))
    new = wrap(pos + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4))
    return new

def _min_image_delta(a, b, box):
    # a,b: (...,3) in index-space, box=(N,N,N)
    d = a - b
    d = (d + box/2.0) % box - box/2.0
    return d

def _trace_loop(seed, Bx, By, Bz, dt=0.3, steps=3000, min_close=300, close_tol=0.8):
    N = Bx.shape[0]
    box = np.array([N,N,N], dtype=float)
    pos = np.array(seed, dtype=float)
    start = pos.copy()
    pts = [pos.copy()]
    for t in range(1, steps+1):
        pos = _rk4_step(pos, dt, Bx, By, Bz)
        if t % 2 == 0:
            pts.append(pos.copy())
        if t > min_close:
            d = np.linalg.norm(_min_image_delta(pos, start, box))
            if d < close_tol:
                # make it closed
                pts.append(start.copy())
                P = np.asarray(pts)
                # dedup/decimate a bit
                return P
    return None  # open line

def _find_closed_loops(Bx, By, Bz, TARGET=8, grid_div=4, jitter=0.15, dt=0.3, steps=3000, min_close=300):
    N = Bx.shape[0]; box = np.array([N,N,N], dtype=float)
    # seed grid
    coords = np.linspace(0, N, grid_div, endpoint=False) + N/(2*grid_div)
    seeds = []
    for xi in coords:
        for yi in coords:
            for zi in coords:
                seeds.append([xi, yi, zi])
    rng = np.random.default_rng(17)
    rng.shuffle(seeds)
    loops = []
    centers = []
    for s in seeds:
        if len(loops) >= TARGET: break
        s = np.array(s) + rng.normal(0, jitter, size=3)
        s = np.mod(s, N)
        L = _trace_loop(s, Bx, By, Bz, dt=dt, steps=steps, min_close=min_close)
        if L is None: continue
        # uniqueness by center
        c = np.mean(L[:-1], axis=0)
        if centers:
            dd = [np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers]
            if np.min(dd) < 2.0:  # too similar
                continue
        loops.append(L)
        centers.append(c)
    return loops

# ---------------- Gauss linking integral ----------------
def _gauss_link(A, B, box):
    # A, B: (m,3)/(n,3) closed polylines in index-space; use minimal image between segment midpoints.
    # Return real-valued Lk (we'll round to nearest integer).
    A = np.asarray(A); B = np.asarray(B)
    a_mid = 0.5*(A[1:] + A[:-1]);  b_mid = 0.5*(B[1:] + B[:-1])
    da = A[1:] - A[:-1];           db = B[1:] - B[:-1]
    # broadcast
    Da = da[:,None,:]                 # (ma,1,3)
    Db = db[None,:,:]                 # (1,mb,3)
    Ra = a_mid[:,None,:]              # (ma,1,3)
    Rb = b_mid[None,:,:]              # (1,mb,3)
    dR = _min_image_delta(Ra, Rb, box)  # (ma,mb,3)
    cross = np.cross(Da, Db)          # (ma,mb,3)
    num = np.einsum('ijk,ijk->ij', cross, dR)  # (ma,mb)
    denom = (np.linalg.norm(dR, axis=2)**3 + 1e-12)
    s = np.sum(num / denom) / (4*np.pi)
    return float(s)

def total_linking_metric(loops, box):
    # sum of |rounded Lk| over all unordered pairs
    if len(loops) < 2: return 0.0, 0, []
    Lks = []
    for i in range(len(loops)):
        for j in range(i+1, len(loops)):
            lk = _gauss_link(loops[i], loops[j], box)
            Lks.append(lk)
    Lks = np.array(Lks, dtype=float)
    Lk_int = np.rint(Lks)  # nearest integer
    total_abs = float(np.sum(np.abs(Lk_int)))
    nonzero = int(np.sum(np.abs(Lk_int) >= 1))
    return total_abs, nonzero, Lks.tolist()

# ---------------- Spectral-rotation nulls ----------------
def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)
    def pos_half(ix, iy, iz):
        kz = kzv[iz]; ky = kyv[iy]; kx = kxv[ix]
        if kz > 0: return True
        if kz < 0: return False
        if ky > 0: return True
        if ky < 0: return False
        return kx > 0
    for ix in range(Nx):
        kx = kxv[ix]
        for iy in range(Ny):
            ky = kyv[iy]
            for iz in range(Nz):
                kz = kzv[iz]
                if kx==0 and ky==0 and kz==0:
                    Bxh_new[ix,iy,iz] = Bxh[ix,iy,iz]
                    Byh_new[ix,iy,iz] = Byh[ix,iy,iz]
                    Bzh_new[ix,iy,iz] = Bzh[ix,iy,iz]
                    continue
                if not pos_half(ix,iy,iz):
                    continue
                k = np.array([kx,ky,kz], dtype=float)
                k_norm = np.linalg.norm(k)
                if k_norm == 0: continue
                k_hat = k / k_norm
                ref = np.array([0.0,0.0,1.0]) if abs(k_hat[2])<0.99 else np.array([1.0,0.0,0.0])
                e1 = np.cross(ref, k_hat); e1 /= (np.linalg.norm(e1)+1e-15)
                e2 = np.cross(k_hat, e1);   e2 /= (np.linalg.norm(e2)+1e-15)
                Bk = np.array([Bxh[ix,iy,iz], Byh[ix,iy,iz], Bzh[ix,iy,iz]])
                c1 = e1 @ Bk; c2 = e2 @ Bk
                theta = rng.uniform(0, 2*np.pi)
                ct, st = np.cos(theta), np.sin(theta)
                c1p, c2p = ct*c1 - st*c2, st*c1 + ct*c2
                Bkp = e1*c1p + e2*c2p
                Bxh_new[ix,iy,iz] = Bkp[0]; Byh_new[ix,iy,iz] = Bkp[1]; Bzh_new[ix,iy,iz] = Bkp[2]
                ixn, iyn, izn = (-ix)%Nx, (-iy)%Ny, (-iz)%Nz
                Bxh_new[ixn,iyn,izn] = np.conjugate(Bkp[0])
                Byh_new[ixn,iyn,izn] = np.conjugate(Bkp[1])
                Bzh_new[ixn,iyn,izn] = np.conjugate(Bkp[2])
    mask_unset = (Bxh_new==0) & (Byh_new==0) & (Bzh_new==0)
    Bxh_new[mask_unset] = Bxh[mask_unset]
    Byh_new[mask_unset] = Byh[mask_unset]
    Bzh_new[mask_unset] = Bzh[mask_unset]
    Brx = np.real(ifftn(Bxh_new)); Bry = np.real(ifftn(Byh_new)); Brz = np.real(ifftn(Bzh_new))
    return _project_solenoidal(Brx, Bry, Brz, spacings)

# ---------------- Load field ----------------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
best_path = run_dir / "best_field.npz"
if not best_path.exists():
    best_path = run_dir / "field_small.npz"
assert best_path.exists(), f"No field file found in {run_dir}"
print("Loaded field →", best_path)

d = np.load(best_path)
Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)
Bx, By, Bz = _project_solenoidal(Bx, By, Bz, sp)

# Helicity (sanity)
h_H = helicity_density(Bx, By, Bz, sp)
print(f"[orig] helicity_density={h_H:.3e}")

# ---------------- Trace loops & linking on original ----------------
# Tunables (speed/robustness)
TARGET_LOOPS = 8     # raise to 10–12 for stronger lock
DT = 0.3
STEPS = 3000
MIN_CLOSE = 300
CLOSE_TOL = 0.8

loops = _find_closed_loops(Bx, By, Bz, TARGET=TARGET_LOOPS, grid_div=4, jitter=0.15,
                           dt=DT, steps=STEPS, min_close=MIN_CLOSE)
box = np.array([Bx.shape[0]]*3, dtype=float)
total_abs, nonzero, Lks = total_linking_metric(loops, box)
print(f"[orig] loops={len(loops)}  total|Lk|={total_abs:.0f}  linked_pairs={nonzero}")

# ---------------- Nulls (spectral-rotation) ----------------
NNULL = 16            # ↑ 32/64 for stronger stats
rng = np.random.default_rng(909)
null_vals = []
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx, By, Bz, sp, rng)
    loops_r = _find_closed_loops(Bxr, Byr, Bzr, TARGET=len(loops), grid_div=4, jitter=0.15,
                                 dt=DT, steps=STEPS, min_close=MIN_CLOSE)
    t_abs_r, nz_r, _ = total_linking_metric(loops_r, box)
    null_vals.append(t_abs_r)
    if (r+1) % 4 == 0:
        print(f"  null {r+1}/{NNULL}  loops={len(loops_r)}  total|Lk|={t_abs_r:.0f}")

null_vals = np.array(null_vals, dtype=float)
p_value = float((np.sum(null_vals >= total_abs) + 1) / (NNULL + 1))
link_percentile = float((np.sum(null_vals <= total_abs) + 0.5) / (NNULL + 1))
print(f"[LoopLink] total|Lk|={total_abs:.0f}  null_median={np.median(null_vals):.1f}  p={p_value:.4f}  percentile={link_percentile:.3f}")

# ---------------- Echo (reuse your spectrum if present) ----------------
echo_csv = run_dir / "echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    band = (1.6, 1.8)
    mask = (freqs>band[0]) & (freqs<band[1])
    echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
else:
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
    freqs = np.fft.rfftfreq(len(sig), d=dt)
    P = np.abs(np.fft.rfft(sig))**2 / len(sig)
    band = (1.6, 1.8)
    echo_power = float((P[(freqs>band[0]) & (freqs<band[1])].sum()) / (P.sum()+1e-12))

# ---------------- Glyphness & save ----------------
# We reward explicit linking more now (it’s the hard topology).
wH, wL, wE = 0.25, 0.50, 0.25
H_norm = float(min(1.0, abs(h_H)/(1e-2)))
glyphness = float(wH*H_norm + wL*link_percentile + wE*echo_power)
label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.70 and link_percentile>=0.90 and echo_power>=0.20) else "CANDIDATE"

out = {
    "helicity_density": h_H,
    "loops_found": len(loops),
    "linking": {
        "total_abs_Lk": total_abs,
        "linked_pairs": nonzero,
        "Lk_raw_pairs": Lks
    },
    "nulls": {
        "NNULL": NNULL,
        "total_abs_Lk_nulls": null_vals.tolist(),
        "median": float(np.median(null_vals)),
        "p_value_one_sided": p_value,
        "percentile": link_percentile
    },
    "echo": {"power_norm": echo_power, "band": list(band)},
    "score": {
        "glyphness": glyphness,
        "weights": {"helicity": wH, "link_percentile": wL, "echo": wE},
        "H_norm": H_norm,
        "link_percentile": link_percentile,
        "label": label
    }
}
with open(run_dir/"link_significance_v09.json","w") as f:
    json.dump(out, f, indent=2)
with open(run_dir/"link_nulls_v09.csv","w", newline="") as f:
    wr = csv.writer(f); wr.writerow(["total_abs_Lk_null"])
    for v in null_vals: wr.writerow([f"{v:.6f}"])
print(f"[CNT] glyphness={glyphness:.3f} (H={H_norm:.2f}, Link%={link_percentile:.2f}, Echo={echo_power:.2f}) → {label}")
print("Saved →", run_dir/"link_significance_v09.json")


CNT Physical Glyph Hunt — v0.9 (LoopLink Cert)
Loaded field → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\best_field.npz
[orig] helicity_density=9.769e-02
[orig] loops=1  total|Lk|=0  linked_pairs=0
  null 4/16  loops=0  total|Lk|=0
  null 8/16  loops=0  total|Lk|=0
  null 12/16  loops=0  total|Lk|=0
  null 16/16  loops=0  total|Lk|=0
[LoopLink] total|Lk|=0  null_median=0.0  p=1.0000  percentile=0.971
[CNT] glyphness=0.953 (H=1.00, Link%=0.97, Echo=0.87) → PHYSICAL-GLYPH:PASS
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\link_significance_v09.json


In [13]:
# CNT Physical Glyph Hunt — Single Cell v0.9.3 (Non-Degenerate Gate + Loop Harvest)
# Purpose: avoid "degenerate PASS" when only a few/no loops are found.
# - Hunts more closed loops (multi-dt, forward/backward integration, denser seeds)
# - Requires loops>=MIN_LOOPS and linked_pairs>=1 to trust linking
# - Uses spectral-rotation, divergence-free nulls for linking significance
# - Reuses your saved echo_spectrum.csv for final glyphness

import numpy as np, os, json, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — v0.9.3 (Non-Degenerate Gate)")

# ---------------- FFT & projection ----------------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return kx, ky, kz

def _project_solenoidal(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh_p = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh_p = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh_p = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh_p)), np.real(ifftn(Byh_p)), np.real(ifftn(Bzh_p))

def helicity_density(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
    Ax, Ay, Az = np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))
    H = np.sum(Ax*Bx + Ay*By + Az*Bz) * dx*dy*dz
    V = Lx*Ly*Lz
    return float(H / V)

# ---------------- Streamline integration ----------------
def _tri_interp_periodic(arr, pos):
    N = arr.shape[0]
    x, y, z = pos
    i0 = int(np.floor(x)) % N; j0 = int(np.floor(y)) % N; k0 = int(np.floor(z)) % N
    i1 = (i0+1) % N; j1 = (j0+1) % N; k1 = (k0+1) % N
    tx = x - np.floor(x); ty = y - np.floor(y); tz = z - np.floor(z)
    c000 = arr[i0,j0,k0]; c100 = arr[i1,j0,k0]; c010 = arr[i0,j1,k0]; c110 = arr[i1,j1,k0]
    c001 = arr[i0,j0,k1]; c101 = arr[i1,j0,k1]; c011 = arr[i0,j1,k1]; c111 = arr[i1,j1,k1]
    c00 = c000*(1-tx) + c100*tx
    c01 = c001*(1-tx) + c101*tx
    c10 = c010*(1-tx) + c110*tx
    c11 = c011*(1-tx) + c111*tx
    c0 = c00*(1-ty) + c10*ty
    c1 = c01*(1-ty) + c11*ty
    return c0*(1-tz) + c1*tz

def _B_dir(Bx, By, Bz, pos):
    v = np.array([_tri_interp_periodic(Bx,pos),
                  _tri_interp_periodic(By,pos),
                  _tri_interp_periodic(Bz,pos)], dtype=float)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rk4_step(pos, dt, Bx, By, Bz):
    N = Bx.shape[0]
    wrap = lambda p: np.mod(p, N)
    k1 = _B_dir(Bx,By,Bz,pos)
    k2 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k1))
    k3 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k2))
    k4 = _B_dir(Bx,By,Bz,wrap(pos + dt*k3))
    return wrap(pos + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4))

def _min_image_delta(a, b, box):
    d = a - b
    return (d + box/2.0) % box - box/2.0

def _trace_loop(seed, Bx, By, Bz, dt=0.3, steps=6000, min_close=600, close_tol=0.8):
    N = Bx.shape[0]; box = np.array([N,N,N], float)
    pos = np.array(seed, float); start = pos.copy()
    pts = [pos.copy()]
    for t in range(1, steps+1):
        pos = _rk4_step(pos, dt, Bx, By, Bz)
        if t % 2 == 0: pts.append(pos.copy())
        if t > min_close:
            d = np.linalg.norm(_min_image_delta(pos, start, box))
            if d < close_tol:
                pts.append(start.copy())
                return np.asarray(pts)
    return None

def _find_closed_loops(Bx, By, Bz, TARGET=12, grid_div=6, jitter=0.2, dt_list=(0.2,0.3,0.4), steps=6000, min_close=600, close_tol=0.9):
    N = Bx.shape[0]; box = np.array([N,N,N], float)
    coords = np.linspace(0, N, grid_div, endpoint=False) + N/(2*grid_div)
    seeds = [[xi,yi,zi] for xi in coords for yi in coords for zi in coords]
    rng = np.random.default_rng(101); rng.shuffle(seeds)
    loops, centers = [], []
    for sign in (1.0, -1.0):
        for dt in dt_list:
            for s in seeds:
                if len(loops) >= TARGET: break
                seed = (np.array(s) + rng.normal(0, jitter, 3)) % N
                L = _trace_loop(seed, Bx, By, Bz, dt=sign*dt, steps=steps, min_close=min_close, close_tol=close_tol)
                if L is None: continue
                c = np.mean(L[:-1], axis=0)
                if centers:
                    dd = [np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers]
                    if np.min(dd) < 1.5:  # discourage near-duplicates
                        continue
                loops.append(L); centers.append(c)
            if len(loops) >= TARGET: break
        if len(loops) >= TARGET: break
    return loops

# ---------------- Gauss linking ----------------
def _gauss_link(A, B, box):
    A = np.asarray(A); B = np.asarray(B)
    a_mid = 0.5*(A[1:] + A[:-1]);  b_mid = 0.5*(B[1:] + B[:-1])
    da = A[1:] - A[:-1];           db = B[1:] - B[:-1]
    Da = da[:,None,:]; Db = db[None,:,:]
    Ra = a_mid[:,None,:]; Rb = b_mid[None,:,:]
    dR = _min_image_delta(Ra, Rb, box)
    cross = np.cross(Da, Db)
    num = np.einsum('ijk,ijk->ij', cross, dR)
    denom = (np.linalg.norm(dR, axis=2)**3 + 1e-12)
    return float(np.sum(num/denom) / (4*np.pi))

def total_linking_metric(loops, box):
    if len(loops) < 2: return 0.0, 0, []
    Lks = []
    for i in range(len(loops)):
        for j in range(i+1, len(loops)):
            Lks.append(_gauss_link(loops[i], loops[j], box))
    Lks = np.array(Lks, float)
    Lk_int = np.rint(Lks)
    total_abs = float(np.sum(np.abs(Lk_int)))
    nonzero = int(np.sum(np.abs(Lk_int) >= 1))
    return total_abs, nonzero, Lks.tolist()

# ---------------- Spectral-rotation nulls ----------------
def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)
    def pos_half(ix, iy, iz):
        kz = kzv[iz]; ky = kyv[iy]; kx = kxv[ix]
        if kz > 0: return True
        if kz < 0: return False
        if ky > 0: return True
        if ky < 0: return False
        return kx > 0
    for ix in range(Nx):
        kx = kxv[ix]
        for iy in range(Ny):
            ky = kyv[iy]
            for iz in range(Nz):
                kz = kzv[iz]
                if kx==0 and ky==0 and kz==0:
                    Bxh_new[ix,iy,iz] = Bxh[ix,iy,iz]; Byh_new[ix,iy,iz] = Byh[ix,iy,iz]; Bzh_new[ix,iy,iz] = Bzh[ix,iy,iz]; continue
                if not pos_half(ix,iy,iz): continue
                k = np.array([kx,ky,kz], float); kn = np.linalg.norm(k)
                if kn==0: continue
                k_hat = k/kn
                ref = np.array([0,0,1.0]) if abs(k_hat[2])<0.99 else np.array([1.0,0,0])
                e1 = np.cross(ref, k_hat); e1 /= (np.linalg.norm(e1)+1e-15)
                e2 = np.cross(k_hat, e1);   e2 /= (np.linalg.norm(e2)+1e-15)
                Bk = np.array([Bxh[ix,iy,iz], Byh[ix,iy,iz], Bzh[ix,iy,iz]])
                c1, c2 = e1 @ Bk, e2 @ Bk
                th = rng.uniform(0, 2*np.pi); ct, st = np.cos(th), np.sin(th)
                Bkp = e1*(ct*c1 - st*c2) + e2*(st*c1 + ct*c2)
                Bxh_new[ix,iy,iz] = Bkp[0]; Byh_new[ix,iy,iz] = Bkp[1]; Bzh_new[ix,iy,iz] = Bkp[2]
                ixn, iyn, izn = (-ix)%Nx, (-iy)%Ny, (-iz)%Nz
                Bxh_new[ixn,iyn,izn] = np.conjugate(Bkp[0]); Byh_new[ixn,iyn,izn] = np.conjugate(Bkp[1]); Bzh_new[ixn,iyn,izn] = np.conjugate(Bkp[2])
    mask = (Bxh_new==0) & (Byh_new==0) & (Bzh_new==0)
    Bxh_new[mask] = Bxh[mask]; Byh_new[mask] = Byh[mask]; Bzh_new[mask] = Bzh[mask]
    Brx = np.real(ifftn(Bxh_new)); Bry = np.real(ifftn(Byh_new)); Brz = np.real(ifftn(Bzh_new))
    return _project_solenoidal(Brx, Bry, Brz, spacings)

# ---------------- Load field ----------------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
best_path = run_dir / "best_field.npz"
if not best_path.exists():
    best_path = run_dir / "field_small.npz"
assert best_path.exists(), f"No field file found in {run_dir}"
print("Loaded field →", best_path)

d = np.load(best_path)
Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)

# Project & helicity
Bx, By, Bz = _project_solenoidal(Bx, By, Bz, sp)
h_H = helicity_density(Bx, By, Bz, sp)
print(f"[orig] helicity_density={h_H:.3e}")

# ---------------- Loop harvest ----------------
TARGET_LOOPS   = 12
DT_LIST        = (0.2, 0.3, 0.4)
STEPS          = 6000
MIN_CLOSE      = 600
CLOSE_TOL      = 0.9
GRID_DIV       = 6

loops = _find_closed_loops(Bx, By, Bz, TARGET=TARGET_LOOPS, grid_div=GRID_DIV, jitter=0.2,
                           dt_list=DT_LIST, steps=STEPS, min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
box = np.array([Bx.shape[0]]*3, float)
total_abs, linked_pairs, Lks = total_linking_metric(loops, box)
print(f"[orig] loops={len(loops)}  total|Lk|={total_abs:.0f}  linked_pairs={linked_pairs}")

# ---------------- Non-degenerate gate ----------------
MIN_LOOPS       = 4
REQUIRE_LINKED  = 1   # ≥1 linked pair
trusted = (len(loops) >= MIN_LOOPS and linked_pairs >= REQUIRE_LINKED)

# ---------------- Nulls for linking ----------------
NNULL = 16  # increase to 32/64 for stronger stats
rng = np.random.default_rng(3049)
null_vals = []
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx, By, Bz, sp, rng)
    loops_r = _find_closed_loops(Bxr, Byr, Bzr, TARGET=min(len(loops), TARGET_LOOPS),
                                 grid_div=GRID_DIV, jitter=0.2,
                                 dt_list=DT_LIST, steps=STEPS, min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
    t_abs_r, nz_r, _ = total_linking_metric(loops_r, box)
    null_vals.append(t_abs_r)
    if (r+1) % 4 == 0:
        print(f"  null {r+1}/{NNULL}  loops={len(loops_r)}  total|Lk|={t_abs_r:.0f}")

null_vals = np.array(null_vals, float)
p_link = float((np.sum(null_vals >= total_abs) + 1)/(NNULL+1))
link_percentile = float((np.sum(null_vals <= total_abs) + 0.5)/(NNULL+1))
print(f"[Linking] total|Lk|={total_abs:.0f}  null_median={np.median(null_vals):.1f}  p={p_link:.4f}  percentile={link_percentile:.3f}")

# ---------------- Echo reuse ----------------
echo_csv = run_dir / "echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    band = (1.6, 1.8)
    mask = (freqs>band[0]) & (freqs<band[1])
    echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
else:
    # demo fallback
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
    freqs = np.fft.rfftfreq(len(sig), d=dt); P = np.abs(np.fft.rfft(sig))**2 / len(sig)
    band = (1.6, 1.8); echo_power = float((P[(freqs>band[0]) & (freqs<band[1])].sum()) / (P.sum()+1e-12))

# ---------------- Final glyphness & label ----------------
# If not trusted, we DO NOT use link_percentile to PASS; we call it INCONCLUSIVE and report.
wH, wL, wE = 0.25, 0.50, 0.25
H_norm = float(min(1.0, abs(h_H)/(1e-2)))
if trusted:
    glyphness = float(wH*H_norm + wL*link_percentile + wE*echo_power)
    label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.70 and link_percentile>=0.90 and echo_power>=0.20) else "CANDIDATE"
else:
    glyphness = float(0.5*H_norm + 0.5*echo_power)  # report a conservative composite without linking
    label = "INCONCLUSIVE (need ≥2 linked loops)"

# ---------------- Save ----------------
out = {
    "helicity_density": h_H,
    "loops_found": len(loops),
    "linking": {"total_abs_Lk": total_abs, "linked_pairs": linked_pairs, "pairs_raw": Lks},
    "nulls": {"NNULL": NNULL, "total_abs_Lk_nulls": null_vals.tolist(), "median": float(np.median(null_vals)),
              "p_value_one_sided": p_link, "percentile": link_percentile},
    "echo": {"power_norm": echo_power, "band": list(band)},
    "gate": {"min_loops": MIN_LOOPS, "require_linked_pairs": REQUIRE_LINKED, "trusted": bool(trusted)},
    "score": {"glyphness": glyphness, "weights": {"H": wH, "Link%": wL, "Echo": wE} if trusted else {"H":0.5,"Echo":0.5},
              "H_norm": H_norm, "link_percentile": link_percentile if trusted else None, "label": label}
}
with open(run_dir/"link_significance_v093.json","w") as f:
    json.dump(out, f, indent=2)

print(f"[CNT] glyphness={glyphness:.3f}  (H={H_norm:.2f}, Link%={'{:.2f}'.format(link_percentile) if trusted else '—'}, Echo={echo_power:.2f}) → {label}")
print("Saved →", run_dir/"link_significance_v093.json")


CNT Physical Glyph Hunt — v0.9.3 (Non-Degenerate Gate)
Loaded field → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\best_field.npz
[orig] helicity_density=9.769e-02
[orig] loops=4  total|Lk|=57  linked_pairs=6
  null 4/16  loops=4  total|Lk|=9
  null 8/16  loops=4  total|Lk|=48
  null 12/16  loops=4  total|Lk|=90
  null 16/16  loops=4  total|Lk|=101
[Linking] total|Lk|=57  null_median=46.0  p=0.4706  percentile=0.559
[CNT] glyphness=0.747  (H=1.00, Link%=0.56, Echo=0.87) → CANDIDATE
Saved → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\link_significance_v093.json


In [14]:
# CNT Physical Glyph Hunt — Single Cell v0.10 (Beltrami-Linked Cert)
# Filters loop-for-loop by local Beltrami coherence to suppress spurious linking in nulls.
# Saves: link_beltrami_v010.json, link_beltrami_nulls_v010.csv next to your latest run.

import numpy as np, os, json, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — v0.10 (Beltrami-Linked Cert)")

# ---------- FFT + projection ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return kx, ky, kz

def _project_solenoidal(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2  = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh)), np.real(ifftn(Byh)), np.real(ifftn(Bzh))

def helicity_density(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    i = 1j
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    k2 = kx*kx + ky*ky + kz*kz
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i*np.where(k2!=0, cx/k2, 0.0)
        Ayh = i*np.where(k2!=0, cy/k2, 0.0)
        Azh = i*np.where(k2!=0, cz/k2, 0.0)
    Ax, Ay, Az = np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))
    H = np.sum(Ax*Bx + Ay*By + Az*Bz)*dx*dy*dz
    V = (dx*Nx)*(dy*Ny)*(dz*Nz)
    return float(H/V)

# ---------- Derivatives / curl / Beltrami ----------
def _grad(f, spacings):
    dx, dy, dz = spacings
    return (np.gradient(f, dx, axis=0),
            np.gradient(f, dy, axis=1),
            np.gradient(f, dz, axis=2))

def curl_real(Fx, Fy, Fz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(Fy, dz, axis=2)
    dFz_dy = np.gradient(Fz, dy, axis=1)
    dFz_dx = np.gradient(Fz, dx, axis=0)
    dFx_dz = np.gradient(Fx, dz, axis=2)
    dFx_dy = np.gradient(Fx, dy, axis=1)
    dFy_dx = np.gradient(Fy, dx, axis=0)
    return (dFz_dy - dFy_dz, dFx_dz - dFz_dx, dFy_dx - dFx_dy)

def beltrami_alpha(Bx, By, Bz, spacings):
    Cx, Cy, Cz = curl_real(Bx, By, Bz, spacings)
    Bmag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    Cmag = np.sqrt(Cx*Cx + Cy*Cy + Cz*Cz) + 1e-12
    cos = (Bx*Cx + By*Cy + Bz*Cz) / (Bmag*Cmag)
    return np.clip(cos, -1.0, 1.0)  # α(x) = cos angle(B, curl B)

# ---------- Interp + streamline loops ----------
def _tri_interp_periodic(arr, pos):
    N = arr.shape[0]
    x, y, z = pos
    i0 = int(np.floor(x)) % N; j0 = int(np.floor(y)) % N; k0 = int(np.floor(z)) % N
    i1 = (i0+1) % N; j1 = (j0+1) % N; k1 = (k0+1) % N
    tx = x - np.floor(x); ty = y - np.floor(y); tz = z - np.floor(z)
    c000 = arr[i0,j0,k0]; c100 = arr[i1,j0,k0]; c010 = arr[i0,j1,k0]; c110 = arr[i1,j1,k0]
    c001 = arr[i0,j0,k1]; c101 = arr[i1,j0,k1]; c011 = arr[i0,j1,k1]; c111 = arr[i1,j1,k1]
    c00 = c000*(1-tx) + c100*tx
    c01 = c001*(1-tx) + c101*tx
    c10 = c010*(1-tx) + c110*tx
    c11 = c011*(1-tx) + c111*tx
    c0 = c00*(1-ty) + c10*ty
    c1 = c01*(1-ty) + c11*ty
    return c0*(1-tz) + c1*tz

def _B_dir(Bx, By, Bz, pos):
    v = np.array([_tri_interp_periodic(Bx,pos),
                  _tri_interp_periodic(By,pos),
                  _tri_interp_periodic(Bz,pos)], float)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rk4_step(pos, dt, Bx, By, Bz):
    N = Bx.shape[0]; wrap = lambda p: np.mod(p, N)
    k1 = _B_dir(Bx,By,Bz,pos)
    k2 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k1))
    k3 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k2))
    k4 = _B_dir(Bx,By,Bz,wrap(pos + dt*k3))
    return wrap(pos + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4))

def _min_image_delta(a, b, box):
    d = a - b
    return (d + box/2.0) % box - box/2.0

def _trace_loop(seed, Bx, By, Bz, dt=0.3, steps=6000, min_close=600, close_tol=0.9):
    N = Bx.shape[0]; box = np.array([N,N,N], float)
    pos = np.array(seed, float); start = pos.copy()
    pts = [pos.copy()]
    for t in range(1, steps+1):
        pos = _rk4_step(pos, dt, Bx, By, Bz)
        if t % 2 == 0: pts.append(pos.copy())
        if t > min_close:
            d = np.linalg.norm(_min_image_delta(pos, start, box))
            if d < close_tol:
                pts.append(start.copy())
                return np.asarray(pts)
    return None

def _find_closed_loops(Bx, By, Bz, TARGET=12, grid_div=6, jitter=0.2, dt_list=(0.2,0.3,0.4), steps=6000, min_close=600, close_tol=0.9):
    N = Bx.shape[0]; box = np.array([N,N,N], float)
    coords = np.linspace(0, N, grid_div, endpoint=False) + N/(2*grid_div)
    seeds = [[xi,yi,zi] for xi in coords for yi in coords for zi in coords]
    rng = np.random.default_rng(101); rng.shuffle(seeds)
    loops, centers = [], []
    for sign in (1.0, -1.0):
        for dt in dt_list:
            for s in seeds:
                if len(loops) >= TARGET: break
                seed = (np.array(s) + rng.normal(0, jitter, 3)) % N
                L = _trace_loop(seed, Bx, By, Bz, dt=sign*dt, steps=steps, min_close=min_close, close_tol=close_tol)
                if L is None: continue
                c = np.mean(L[:-1], axis=0)
                if centers:
                    dd = [np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers]
                    if np.min(dd) < 1.5:  # avoid near-duplicates
                        continue
                loops.append(L); centers.append(c)
            if len(loops) >= TARGET: break
        if len(loops) >= TARGET: break
    return loops

# ---------- Loop Beltrami coherence ----------
def loop_beltrami_fraction(L, alpha_field):
    N = alpha_field.shape[0]
    vals = []
    for p in L[:-1]:
        vals.append(_tri_interp_periodic(alpha_field, p))
    vals = np.array(vals, float)
    return float(np.mean(vals > 0.7))  # fraction of samples where α>0.7

def filter_loops_beltrami(loops, alpha_field, min_frac=0.6):
    keep = []
    for L in loops:
        f = loop_beltrami_fraction(L, alpha_field)
        if f >= min_frac:
            keep.append(L)
    return keep

# ---------- Gauss linking ----------
def _gauss_link(A, B, box):
    A = np.asarray(A); B = np.asarray(B)
    a_mid = 0.5*(A[1:] + A[:-1]);  b_mid = 0.5*(B[1:] + B[:-1])
    da = A[1:] - A[:-1];           db = B[1:] - B[:-1]
    Da = da[:,None,:]; Db = db[None,:,:]
    Ra = a_mid[:,None,:]; Rb = b_mid[None,:,:]
    dR = _min_image_delta(Ra, Rb, box)
    cross = np.cross(Da, Db)
    num = np.einsum('ijk,ijk->ij', cross, dR)
    denom = (np.linalg.norm(dR, axis=2)**3 + 1e-12)
    return float(np.sum(num/denom)/(4*np.pi))

def total_linking_metric(loops, box):
    if len(loops) < 2: return 0.0, 0, []
    Lks = []
    for i in range(len(loops)):
        for j in range(i+1, len(loops)):
            Lks.append(_gauss_link(loops[i], loops[j], box))
    Lks = np.array(Lks, float)
    Lk_int = np.rint(Lks)
    total_abs = float(np.sum(np.abs(Lk_int)))
    nonzero = int(np.sum(np.abs(Lk_int) >= 1))
    return total_abs, nonzero, Lks.tolist()

# ---------- Spectral-rotation nulls ----------
def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)
    def pos_half(ix, iy, iz):
        kz = kzv[iz]; ky = kyv[iy]; kx = kxv[ix]
        if kz>0:return True
        if kz<0:return False
        if ky>0:return True
        if ky<0:return False
        return kx>0
    for ix in range(Nx):
        kx = kxv[ix]
        for iy in range(Ny):
            ky = kyv[iy]
            for iz in range(Nz):
                kz = kzv[iz]
                if kx==0 and ky==0 and kz==0:
                    Bxh_new[ix,iy,iz]=Bxh[ix,iy,iz]; Byh_new[ix,iy,iz]=Byh[ix,iy,iz]; Bzh_new[ix,iy,iz]=Bzh[ix,iy,iz]; continue
                if not pos_half(ix,iy,iz): continue
                k = np.array([kx,ky,kz], float); kn = np.linalg.norm(k)
                if kn==0: continue
                k_hat = k/kn
                ref = np.array([0,0,1.0]) if abs(k_hat[2])<0.99 else np.array([1.0,0,0])
                e1 = np.cross(ref, k_hat); e1 /= (np.linalg.norm(e1)+1e-15)
                e2 = np.cross(k_hat, e1);   e2 /= (np.linalg.norm(e2)+1e-15)
                Bk = np.array([Bxh[ix,iy,iz], Byh[ix,iy,iz], Bzh[ix,iy,iz]])
                c1, c2 = e1 @ Bk, e2 @ Bk
                th = rng.uniform(0, 2*np.pi); ct, st = np.cos(th), np.sin(th)
                Bkp = e1*(ct*c1 - st*c2) + e2*(st*c1 + ct*c2)
                Bxh_new[ix,iy,iz]=Bkp[0]; Byh_new[ix,iy,iz]=Bkp[1]; Bzh_new[ix,iy,iz]=Bkp[2]
                ixn,iyn,izn = (-ix)%Nx, (-iy)%Ny, (-iz)%Nz
                Bxh_new[ixn,iyn,izn]=np.conjugate(Bkp[0]); Byh_new[ixn,iyn,izn]=np.conjugate(Bkp[1]); Bzh_new[ixn,iyn,izn]=np.conjugate(Bkp[2])
    mask=(Bxh_new==0)&(Byh_new==0)&(Bzh_new==0)
    Bxh_new[mask]=Bxh[mask]; Byh_new[mask]=Byh[mask]; Bzh_new[mask]=Bzh[mask]
    Brx = np.real(ifftn(Bxh_new)); Bry = np.real(ifftn(Byh_new)); Brz = np.real(ifftn(Bzh_new))
    return _project_solenoidal(Brx, Bry, Brz, spacings)

# ---------- Load field & precompute ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
best_path = run_dir/"best_field.npz"
if not best_path.exists():
    best_path = run_dir/"field_small.npz"
assert best_path.exists(), f"No field .npz found in {run_dir}"
print("Loaded field →", best_path)

d = np.load(best_path)
Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)
Bx, By, Bz = _project_solenoidal(Bx, By, Bz, sp)
h_H = helicity_density(Bx, By, Bz, sp)
print(f"[orig] helicity_density={h_H:.3e}")

alpha = beltrami_alpha(Bx, By, Bz, sp)

# ---------- Harvest + filter ----------
TARGET_LOOPS   = 14
DT_LIST        = (0.2, 0.3, 0.4)
GRID_DIV       = 7
STEPS          = 7000
MIN_CLOSE      = 700
CLOSE_TOL      = 1.0
BELTRAMI_THR   = 0.6   # fraction of points with α>0.7 must exceed this
print(f"[params] TARGET_LOOPS={TARGET_LOOPS} GRID_DIV={GRID_DIV} BELTRAMI_THR={BELTRAMI_THR}")

loops = _find_closed_loops(Bx, By, Bz, TARGET=TARGET_LOOPS, grid_div=GRID_DIV,
                           jitter=0.2, dt_list=DT_LIST, steps=STEPS,
                           min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
box = np.array([Bx.shape[0]]*3, float)
loops_coh = filter_loops_beltrami(loops, alpha, min_frac=BELTRAMI_THR)
total_abs, linked_pairs, _ = total_linking_metric(loops_coh, box)
print(f"[orig] loops={len(loops)}  coherent={len(loops_coh)}  total|Lk|={total_abs:.0f}  linked_pairs={linked_pairs}")

# ---------- Nulls with same coherence gate ----------
NNULL = 16
rng = np.random.default_rng(777)
null_vals = []
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx, By, Bz, sp, rng)
    ar = beltrami_alpha(Bxr, Byr, Bzr, sp)
    loops_r = _find_closed_loops(Bxr, Byr, Bzr, TARGET=min(len(loops), TARGET_LOOPS),
                                 grid_div=GRID_DIV, jitter=0.2, dt_list=DT_LIST,
                                 steps=STEPS, min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
    loops_r_coh = filter_loops_beltrami(loops_r, ar, min_frac=BELTRAMI_THR)
    t_abs_r, nz_r, _ = total_linking_metric(loops_r_coh, box)
    null_vals.append(t_abs_r)
    if (r+1)%4==0:
        print(f"  null {r+1}/{NNULL}  loops={len(loops_r)} coh={len(loops_r_coh)} total|Lk|={t_abs_r:.0f}")

null_vals = np.array(null_vals, float)
p_link = float((np.sum(null_vals >= total_abs) + 1)/(NNULL+1))
link_percentile = float((np.sum(null_vals <= total_abs) + 0.5)/(NNULL+1))
print(f"[BeltramiLink] total|Lk|={total_abs:.0f}  null_median={np.median(null_vals):.1f}  p={p_link:.4f}  percentile={link_percentile:.3f}")

# ---------- Echo reuse ----------
echo_csv = run_dir/"echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    band = (1.6, 1.8)
    mask = (freqs>band[0]) & (freqs<band[1])
    echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
else:
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
    freqs = np.fft.rfftfreq(len(sig), d=dt); P = np.abs(np.fft.rfft(sig))**2 / len(sig)
    band = (1.6, 1.8); echo_power = float((P[(freqs>band[0]) & (freqs<band[1])].sum()) / (P.sum()+1e-12))

# ---------- Final glyphness & save ----------
MIN_LOOPS, REQUIRE_LINKED = 4, 1
trusted = (len(loops_coh) >= MIN_LOOPS and linked_pairs >= REQUIRE_LINKED)

wH, wL, wE = 0.25, 0.50, 0.25
H_norm = float(min(1.0, abs(h_H)/(1e-2)))
if trusted:
    glyphness = float(wH*H_norm + wL*link_percentile + wE*echo_power)
    label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.70 and link_percentile>=0.90 and echo_power>=0.20) else "CANDIDATE"
else:
    glyphness = float(0.5*H_norm + 0.5*echo_power)
    label = "INCONCLUSIVE (need coherent links)"

out = {
    "helicity_density": h_H,
    "loops_total": len(loops), "loops_coherent": len(loops_coh),
    "beltrami_filter": {"alpha_thresh": 0.7, "min_fraction": BELTRAMI_THR},
    "linking": {"total_abs_Lk": total_abs, "linked_pairs": linked_pairs},
    "nulls": {"NNULL": NNULL, "total_abs_Lk_nulls": null_vals.tolist(), "median": float(np.median(null_vals)),
              "p_value_one_sided": p_link, "percentile": link_percentile},
    "echo": {"power_norm": echo_power, "band": list(band)},
    "score": {"glyphness": glyphness, "weights": {"H":wH,"Link%":wL,"Echo":wE} if trusted else {"H":0.5,"Echo":0.5},
              "H_norm": H_norm, "link_percentile": link_percentile if trusted else None, "label": label}
}
with open(run_dir/"link_beltrami_v010.json","w") as f: json.dump(out, f, indent=2)
with open(run_dir/"link_beltrami_nulls_v010.csv","w", newline="") as f:
    wr = csv.writer(f); wr.writerow(["total_abs_Lk_null"])
    for v in null_vals: wr.writerow([f"{v:.6f}"])

print(f"[CNT] glyphness={glyphness:.3f} (H={H_norm:.2f}, Link%={link_percentile:.2f}*, Echo={echo_power:.2f}) → {label}")
print("Saved →", run_dir/"link_beltrami_v010.json")


CNT Physical Glyph Hunt — v0.10 (Beltrami-Linked Cert)
Loaded field → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\best_field.npz
[orig] helicity_density=9.769e-02
[params] TARGET_LOOPS=14 GRID_DIV=7 BELTRAMI_THR=0.6


KeyboardInterrupt: 

In [15]:
# CNT GPU Bootstrap — RTX 4070 (CuPy/cuFFT hybrid)  vGPU-1.0
# Overrides: _project_solenoidal, _vector_potential_from_B, helicity_density, spectral_rotation_null
# Strategy: GPU for all spectral heavy steps; keep streamline/loop tracing on CPU.

import sys, subprocess, importlib, math
import numpy as np

# --- 1) Bring CuPy online (CUDA 12.x wheel fits RTX 4070 drivers) ---
try:
    import cupy as cp
except Exception:
    print("[setup] Installing CuPy for CUDA 12.x…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "cupy-cuda12x"])
    import cupy as cp

try:
    ndev = cp.cuda.runtime.getDeviceCount()
    dev = cp.cuda.Device(0)
    dev.use()
    props = cp.cuda.runtime.getDeviceProperties(0)
    name = props["name"].decode() if isinstance(props["name"], bytes) else props["name"]
    print(f"[GPU] {name} • SMs={props['multiProcessorCount']} • mem={props['totalGlobalMem']/1e9:.1f} GB  • CuPy={cp.__version__}")
    HAS_GPU = True
except Exception as e:
    print("[GPU] No CUDA device or driver issue:", e)
    HAS_GPU = False

# --- 2) Small helpers ---
_DTYPE = np.float32  # use FP32 on GPU for speed; accurate enough for our stats

def _ensure_cp(x):
    return cp.asarray(x, dtype=_DTYPE)

def _ensure_np(x):
    return cp.asnumpy(x)

# Build k-grids (GPU)
def _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = cp.fft.fftfreq(Nx, d=Lx/Nx) * (2*cp.pi)
    ky = cp.fft.fftfreq(Ny, d=Ly/Ny) * (2*cp.pi)
    kz = cp.fft.fftfreq(Nz, d=Lz/Nz) * (2*cp.pi)
    return cp.meshgrid(kx, ky, kz, indexing='ij')

# --- 3) GPU overrides (fall back to CPU if needed) ---
def _project_solenoidal(Bx, By, Bz, spacings):
    """
    Project B onto the divergence-free subspace in Fourier domain (Coulomb gauge).
    Accepts numpy, returns numpy; runs on GPU if available.
    """
    if not HAS_GPU:
        # CPU fallback (numpy FFT)
        dx, dy, dz = spacings
        Nx, Ny, Nz = Bx.shape
        Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
        kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
        ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
        kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
        kx, ky, kz = np.meshgrid(kx, ky, kz, indexing='ij')
        Bxh, Byh, Bzh = np.fft.fftn(Bx), np.fft.fftn(By), np.fft.fftn(Bz)
        k2 = kx*kx + ky*ky + kz*kz
        dot = kx*Bxh + ky*Byh + kz*Bzh
        Bxh = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
        return (np.fft.ifftn(Bxh).real.astype(_DTYPE),
                np.fft.ifftn(Byh).real.astype(_DTYPE),
                np.fft.ifftn(Bzh).real.astype(_DTYPE))
    # GPU path
    dx, dy, dz = spacings
    Bxg, Byg, Bzg = _ensure_cp(Bx), _ensure_cp(By), _ensure_cp(Bz)
    Nx, Ny, Nz = Bxg.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kx, ky, kz = _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = cp.fft.fftn(Bxg), cp.fft.fftn(Byg), cp.fft.fftn(Bzg)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    Bxh = cp.where(k2!=0, Bxh - kx*dot/k2, 0.0)
    Byh = cp.where(k2!=0, Byh - ky*dot/k2, 0.0)
    Bzh = cp.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    Brx = cp.fft.ifftn(Bxh).real.astype(_DTYPE)
    Bry = cp.fft.ifftn(Byh).real.astype(_DTYPE)
    Brz = cp.fft.ifftn(Bzh).real.astype(_DTYPE)
    return _ensure_np(Brx), _ensure_np(Bry), _ensure_np(Brz)

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    """
    Solve ∇×A = B (Coulomb gauge) in Fourier domain. Accepts numpy; returns numpy. GPU-backed.
    """
    if not HAS_GPU:
        kx = np.fft.fftfreq(Bx.shape[0], d=Lx/Bx.shape[0]) * 2*np.pi
        ky = np.fft.fftfreq(By.shape[1], d=Ly/By.shape[1]) * 2*np.pi
        kz = np.fft.fftfreq(Bz.shape[2], d=Lz/Bz.shape[2]) * 2*np.pi
        kx, ky, kz = np.meshgrid(kx, ky, kz, indexing='ij')
        Bxh, Byh, Bzh = np.fft.fftn(Bx), np.fft.fftn(By), np.fft.fftn(Bz)
        k2 = kx*kx + ky*ky + kz*kz
        i = 1j
        cx = ky*Bzh - kz*Byh
        cy = kz*Bxh - kx*Bzh
        cz = kx*Byh - ky*Bxh
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
        return (np.fft.ifftn(Axh).real.astype(_DTYPE),
                np.fft.ifftn(Ayh).real.astype(_DTYPE),
                np.fft.ifftn(Azh).real.astype(_DTYPE))
    # GPU path
    Bxg, Byg, Bzg = _ensure_cp(Bx), _ensure_cp(By), _ensure_cp(Bz)
    Nx, Ny, Nz = Bxg.shape
    kx, ky, kz = _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = cp.fft.fftn(Bxg), cp.fft.fftn(Byg), cp.fft.fftn(Bzg)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    Axh = i * cp.where(k2!=0, cx/k2, 0.0)
    Ayh = i * cp.where(k2!=0, cy/k2, 0.0)
    Azh = i * cp.where(k2!=0, cz/k2, 0.0)
    Ax = cp.fft.ifftn(Axh).real.astype(_DTYPE)
    Ay = cp.fft.ifftn(Ayh).real.astype(_DTYPE)
    Az = cp.fft.ifftn(Azh).real.astype(_DTYPE)
    return _ensure_np(Ax), _ensure_np(Ay), _ensure_np(Az)

def helicity_density(Bx, By, Bz, spacings):
    """
    H/V with A from ∇×A=B. Accepts numpy; returns float. GPU-backed.
    """
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = float((Ax*Bx + Ay*By + Az*Bz).sum() * dx*dy*dz)
    V = Lx*Ly*Lz
    return H / V

def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    """
    Strong null: rotate B̂(k) within the plane ⟂k for half the spectrum,
    mirror-conjugate to enforce Hermitian, then project to solenoidal.
    Accepts numpy; returns numpy. GPU-backed & vectorized.
    """
    dx, dy, dz = spacings
    if not HAS_GPU:
        # fall back to your earlier CPU spectral_rotation_null (if defined)
        # (or keep phase-scramble as a last resort)
        from numpy.fft import fftn, ifftn, fftfreq
        Nx, Ny, Nz = Bx.shape
        Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
        kxv, kyv, kzv = fftfreq(Nx, d=Lx/Nx)*2*np.pi, fftfreq(Ny, d=Ly/Ny)*2*np.pi, fftfreq(Nz, d=Lz/Nz)*2*np.pi
        kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
        Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
        k2 = kx*kx + ky*ky + kz*kz
        pos = (kz>0) | ((kz==0)&(ky>0)) | ((kz==0)&(ky==0)&(kx>0))
        # Build orthonormal basis in ⟂k
        kn = np.sqrt(k2)+1e-15
        kxh, kyh, kzh = kx/kn, ky/kn, kz/kn
        use_x = (np.abs(kxh) < 0.9)
        ax = np.where(use_x, 1.0, 0.0); ay = np.where(use_x, 0.0, 1.0); az = 0.0
        e1x = ay*kzh - az*kyh; e1y = az*kxh - ax*kzh; e1z = ax*kyh - ay*kxh
        n1 = np.sqrt(e1x*e1x + e1y*e1y + e1z*e1z)+1e-15; e1x/=n1; e1y/=n1; e1z/=n1
        e2x = kyh*e1z - kzh*e1y; e2y = kzh*e1x - kxh*e1z; e2z = kxh*e1y - kyh*e1x
        c1 = e1x*Bxh + e1y*Byh + e1z*Bzh
        c2 = e2x*Bxh + e2y*Byh + e2z*Bzh
        theta = rng.uniform(0, 2*np.pi, size=Bxh.shape)
        ct, st = np.cos(theta), np.sin(theta)
        c1p = ct*c1 - st*c2
        c2p = st*c1 + ct*c2
        Bxh_rot = e1x*c1p + e2x*c2p
        Byh_rot = e1y*c1p + e2y*c2p
        Bzh_rot = e1z*c1p + e2z*c2p
        # Build Hermitian array by pos + flipped conjugate
        zero = (k2==0)
        Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)
        Bxh_new[pos] = Bxh_rot[pos]; Byh_new[pos] = Byh_rot[pos]; Bzh_new[pos] = Bzh_rot[pos]
        Bxh_new += np.conj(Bxh_new[::-1, ::-1, ::-1])
        Byh_new += np.conj(Byh_new[::-1, ::-1, ::-1])
        Bzh_new += np.conj(Bzh_new[::-1, ::-1, ::-1])
        Bxh_new[zero] = Bxh[zero]; Byh_new[zero] = Byh[zero]; Bzh_new[zero] = Bzh[zero]
        Brx = np.fft.ifftn(Bxh_new).real.astype(_DTYPE)
        Bry = np.fft.ifftn(Byh_new).real.astype(_DTYPE)
        Brz = np.fft.ifftn(Bzh_new).real.astype(_DTYPE)
        return _project_solenoidal(Brx, Bry, Brz, spacings)
    # GPU path
    Bxg, Byg, Bzg = _ensure_cp(Bx), _ensure_cp(By), _ensure_cp(Bz)
    Nx, Ny, Nz = Bxg.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = cp.fft.fftn(Bxg), cp.fft.fftn(Byg), cp.fft.fftn(Bzg)
    k2 = kxv*kxv + kyv*kyv + kzv*kzv
    pos = (kzv>0) | ((kzv==0)&(kyv>0)) | ((kzv==0)&(kyv==0)&(kxv>0))
    # Basis in ⟂k (vectorized)
    kn = cp.sqrt(k2) + 1e-15
    kxh, kyh, kzh = kxv/kn, kyv/kn, kzv/kn
    use_x = (cp.abs(kxh) < 0.9)
    ax = cp.where(use_x, 1.0, 0.0); ay = cp.where(use_x, 0.0, 1.0); az = cp.zeros_like(kxh)
    e1x = ay*kzh - az*kyh; e1y = az*kxh - ax*kzh; e1z = ax*kyh - ay*kxh
    n1 = cp.sqrt(e1x*e1x + e1y*e1y + e1z*e1z) + 1e-15
    e1x/=n1; e1y/=n1; e1z/=n1
    e2x = kyh*e1z - kzh*e1y; e2y = kzh*e1x - kxh*e1z; e2z = kxh*e1y - kyh*e1x
    # Project & rotate
    c1 = e1x*Bxh + e1y*Byh + e1z*Bzh
    c2 = e2x*Bxh + e2y*Byh + e2z*Bzh
    theta = cp.zeros_like(k2, dtype=_DTYPE)
    # Only give random angles on positive half
    theta[pos] = (cp.random.random(pos.sum(), dtype=_DTYPE) * (2*cp.pi)).astype(_DTYPE)
    ct, st = cp.cos(theta), cp.sin(theta)
    c1p = ct*c1 - st*c2
    c2p = st*c1 + ct*c2
    Bxh_rot = e1x*c1p + e2x*c2p
    Byh_rot = e1y*c1p + e2y*c2p
    Bzh_rot = e1z*c1p + e2z*c2p
    # Compose Hermitian spectrum: rotated pos + conj of flipped
    Bxh_new = cp.where(pos, Bxh_rot, 0.0+0.0j)
    Byh_new = cp.where(pos, Byh_rot, 0.0+0.0j)
    Bzh_new = cp.where(pos, Bzh_rot, 0.0+0.0j)
    Bxh_new = Bxh_new + cp.conj(Bxh_new[::-1, ::-1, ::-1])
    Byh_new = Byh_new + cp.conj(Byh_new[::-1, ::-1, ::-1])
    Bzh_new = Bzh_new + cp.conj(Bzh_new[::-1, ::-1, ::-1])
    # Keep self-conjugate planes from original (DC/Nyquist)
    zero = (k2==0)
    Bxh_new[zero] = Bxh[zero]; Byh_new[zero] = Byh[zero]; Bzh_new[zero] = Bzh[zero]
    # Back to real space + projection
    Brx = cp.fft.ifftn(Bxh_new).real.astype(_DTYPE)
    Bry = cp.fft.ifftn(Byh_new).real.astype(_DTYPE)
    Brz = cp.fft.ifftn(Bzh_new).real.astype(_DTYPE)
    Brx, Bry, Brz = _project_solenoidal(_ensure_np(Brx), _ensure_np(Bry), _ensure_np(Brz), spacings)
    return Brx, Bry, Brz

print("✅ GPU bootstrap loaded. Heavy ops are now cuFFT-accelerated when available.")


[GPU] NVIDIA GeForce RTX 4070 • SMs=46 • mem=12.9 GB  • CuPy=13.6.0
✅ GPU bootstrap loaded. Heavy ops are now cuFFT-accelerated when available.


In [16]:
# CNT Physical Glyph Hunt — Single Cell v0.10 (Beltrami-Linked Cert)
# Filters loop-for-loop by local Beltrami coherence to suppress spurious linking in nulls.
# Saves: link_beltrami_v010.json, link_beltrami_nulls_v010.csv next to your latest run.

import numpy as np, os, json, csv
from numpy.fft import fftn, ifftn
from pathlib import Path

print("CNT Physical Glyph Hunt — v0.10 (Beltrami-Linked Cert)")

# ---------- FFT + projection ----------
def _fftvec(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
    ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
    kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
    return kx, ky, kz

def _project_solenoidal(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    k2  = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    with np.errstate(divide='ignore', invalid='ignore'):
        Bxh = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    return np.real(ifftn(Bxh)), np.real(ifftn(Byh)), np.real(ifftn(Bzh))

def helicity_density(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
    i = 1j
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    k2 = kx*kx + ky*ky + kz*kz
    with np.errstate(divide='ignore', invalid='ignore'):
        Axh = i*np.where(k2!=0, cx/k2, 0.0)
        Ayh = i*np.where(k2!=0, cy/k2, 0.0)
        Azh = i*np.where(k2!=0, cz/k2, 0.0)
    Ax, Ay, Az = np.real(ifftn(Axh)), np.real(ifftn(Ayh)), np.real(ifftn(Azh))
    H = np.sum(Ax*Bx + Ay*By + Az*Bz)*dx*dy*dz
    V = (dx*Nx)*(dy*Ny)*(dz*Nz)
    return float(H/V)

# ---------- Derivatives / curl / Beltrami ----------
def _grad(f, spacings):
    dx, dy, dz = spacings
    return (np.gradient(f, dx, axis=0),
            np.gradient(f, dy, axis=1),
            np.gradient(f, dz, axis=2))

def curl_real(Fx, Fy, Fz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(Fy, dz, axis=2)
    dFz_dy = np.gradient(Fz, dy, axis=1)
    dFz_dx = np.gradient(Fz, dx, axis=0)
    dFx_dz = np.gradient(Fx, dz, axis=2)
    dFx_dy = np.gradient(Fx, dy, axis=1)
    dFy_dx = np.gradient(Fy, dx, axis=0)
    return (dFz_dy - dFy_dz, dFx_dz - dFz_dx, dFy_dx - dFx_dy)

def beltrami_alpha(Bx, By, Bz, spacings):
    Cx, Cy, Cz = curl_real(Bx, By, Bz, spacings)
    Bmag = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    Cmag = np.sqrt(Cx*Cx + Cy*Cy + Cz*Cz) + 1e-12
    cos = (Bx*Cx + By*Cy + Bz*Cz) / (Bmag*Cmag)
    return np.clip(cos, -1.0, 1.0)  # α(x) = cos angle(B, curl B)

# ---------- Interp + streamline loops ----------
def _tri_interp_periodic(arr, pos):
    N = arr.shape[0]
    x, y, z = pos
    i0 = int(np.floor(x)) % N; j0 = int(np.floor(y)) % N; k0 = int(np.floor(z)) % N
    i1 = (i0+1) % N; j1 = (j0+1) % N; k1 = (k0+1) % N
    tx = x - np.floor(x); ty = y - np.floor(y); tz = z - np.floor(z)
    c000 = arr[i0,j0,k0]; c100 = arr[i1,j0,k0]; c010 = arr[i0,j1,k0]; c110 = arr[i1,j1,k0]
    c001 = arr[i0,j0,k1]; c101 = arr[i1,j0,k1]; c011 = arr[i0,j1,k1]; c111 = arr[i1,j1,k1]
    c00 = c000*(1-tx) + c100*tx
    c01 = c001*(1-tx) + c101*tx
    c10 = c010*(1-tx) + c110*tx
    c11 = c011*(1-tx) + c111*tx
    c0 = c00*(1-ty) + c10*ty
    c1 = c01*(1-ty) + c11*ty
    return c0*(1-tz) + c1*tz

def _B_dir(Bx, By, Bz, pos):
    v = np.array([_tri_interp_periodic(Bx,pos),
                  _tri_interp_periodic(By,pos),
                  _tri_interp_periodic(Bz,pos)], float)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rk4_step(pos, dt, Bx, By, Bz):
    N = Bx.shape[0]; wrap = lambda p: np.mod(p, N)
    k1 = _B_dir(Bx,By,Bz,pos)
    k2 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k1))
    k3 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k2))
    k4 = _B_dir(Bx,By,Bz,wrap(pos + dt*k3))
    return wrap(pos + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4))

def _min_image_delta(a, b, box):
    d = a - b
    return (d + box/2.0) % box - box/2.0

def _trace_loop(seed, Bx, By, Bz, dt=0.3, steps=6000, min_close=600, close_tol=0.9):
    N = Bx.shape[0]; box = np.array([N,N,N], float)
    pos = np.array(seed, float); start = pos.copy()
    pts = [pos.copy()]
    for t in range(1, steps+1):
        pos = _rk4_step(pos, dt, Bx, By, Bz)
        if t % 2 == 0: pts.append(pos.copy())
        if t > min_close:
            d = np.linalg.norm(_min_image_delta(pos, start, box))
            if d < close_tol:
                pts.append(start.copy())
                return np.asarray(pts)
    return None

def _find_closed_loops(Bx, By, Bz, TARGET=12, grid_div=6, jitter=0.2, dt_list=(0.2,0.3,0.4), steps=6000, min_close=600, close_tol=0.9):
    N = Bx.shape[0]; box = np.array([N,N,N], float)
    coords = np.linspace(0, N, grid_div, endpoint=False) + N/(2*grid_div)
    seeds = [[xi,yi,zi] for xi in coords for yi in coords for zi in coords]
    rng = np.random.default_rng(101); rng.shuffle(seeds)
    loops, centers = [], []
    for sign in (1.0, -1.0):
        for dt in dt_list:
            for s in seeds:
                if len(loops) >= TARGET: break
                seed = (np.array(s) + rng.normal(0, jitter, 3)) % N
                L = _trace_loop(seed, Bx, By, Bz, dt=sign*dt, steps=steps, min_close=min_close, close_tol=close_tol)
                if L is None: continue
                c = np.mean(L[:-1], axis=0)
                if centers:
                    dd = [np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers]
                    if np.min(dd) < 1.5:  # avoid near-duplicates
                        continue
                loops.append(L); centers.append(c)
            if len(loops) >= TARGET: break
        if len(loops) >= TARGET: break
    return loops

# ---------- Loop Beltrami coherence ----------
def loop_beltrami_fraction(L, alpha_field):
    N = alpha_field.shape[0]
    vals = []
    for p in L[:-1]:
        vals.append(_tri_interp_periodic(alpha_field, p))
    vals = np.array(vals, float)
    return float(np.mean(vals > 0.7))  # fraction of samples where α>0.7

def filter_loops_beltrami(loops, alpha_field, min_frac=0.6):
    keep = []
    for L in loops:
        f = loop_beltrami_fraction(L, alpha_field)
        if f >= min_frac:
            keep.append(L)
    return keep

# ---------- Gauss linking ----------
def _gauss_link(A, B, box):
    A = np.asarray(A); B = np.asarray(B)
    a_mid = 0.5*(A[1:] + A[:-1]);  b_mid = 0.5*(B[1:] + B[:-1])
    da = A[1:] - A[:-1];           db = B[1:] - B[:-1]
    Da = da[:,None,:]; Db = db[None,:,:]
    Ra = a_mid[:,None,:]; Rb = b_mid[None,:,:]
    dR = _min_image_delta(Ra, Rb, box)
    cross = np.cross(Da, Db)
    num = np.einsum('ijk,ijk->ij', cross, dR)
    denom = (np.linalg.norm(dR, axis=2)**3 + 1e-12)
    return float(np.sum(num/denom)/(4*np.pi))

def total_linking_metric(loops, box):
    if len(loops) < 2: return 0.0, 0, []
    Lks = []
    for i in range(len(loops)):
        for j in range(i+1, len(loops)):
            Lks.append(_gauss_link(loops[i], loops[j], box))
    Lks = np.array(Lks, float)
    Lk_int = np.rint(Lks)
    total_abs = float(np.sum(np.abs(Lk_int)))
    nonzero = int(np.sum(np.abs(Lk_int) >= 1))
    return total_abs, nonzero, Lks.tolist()

# ---------- Spectral-rotation nulls ----------
def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
    Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)
    def pos_half(ix, iy, iz):
        kz = kzv[iz]; ky = kyv[iy]; kx = kxv[ix]
        if kz>0:return True
        if kz<0:return False
        if ky>0:return True
        if ky<0:return False
        return kx>0
    for ix in range(Nx):
        kx = kxv[ix]
        for iy in range(Ny):
            ky = kyv[iy]
            for iz in range(Nz):
                kz = kzv[iz]
                if kx==0 and ky==0 and kz==0:
                    Bxh_new[ix,iy,iz]=Bxh[ix,iy,iz]; Byh_new[ix,iy,iz]=Byh[ix,iy,iz]; Bzh_new[ix,iy,iz]=Bzh[ix,iy,iz]; continue
                if not pos_half(ix,iy,iz): continue
                k = np.array([kx,ky,kz], float); kn = np.linalg.norm(k)
                if kn==0: continue
                k_hat = k/kn
                ref = np.array([0,0,1.0]) if abs(k_hat[2])<0.99 else np.array([1.0,0,0])
                e1 = np.cross(ref, k_hat); e1 /= (np.linalg.norm(e1)+1e-15)
                e2 = np.cross(k_hat, e1);   e2 /= (np.linalg.norm(e2)+1e-15)
                Bk = np.array([Bxh[ix,iy,iz], Byh[ix,iy,iz], Bzh[ix,iy,iz]])
                c1, c2 = e1 @ Bk, e2 @ Bk
                th = rng.uniform(0, 2*np.pi); ct, st = np.cos(th), np.sin(th)
                Bkp = e1*(ct*c1 - st*c2) + e2*(st*c1 + ct*c2)
                Bxh_new[ix,iy,iz]=Bkp[0]; Byh_new[ix,iy,iz]=Bkp[1]; Bzh_new[ix,iy,iz]=Bkp[2]
                ixn,iyn,izn = (-ix)%Nx, (-iy)%Ny, (-iz)%Nz
                Bxh_new[ixn,iyn,izn]=np.conjugate(Bkp[0]); Byh_new[ixn,iyn,izn]=np.conjugate(Bkp[1]); Bzh_new[ixn,iyn,izn]=np.conjugate(Bkp[2])
    mask=(Bxh_new==0)&(Byh_new==0)&(Bzh_new==0)
    Bxh_new[mask]=Bxh[mask]; Byh_new[mask]=Byh[mask]; Bzh_new[mask]=Bzh[mask]
    Brx = np.real(ifftn(Bxh_new)); Bry = np.real(ifftn(Byh_new)); Brz = np.real(ifftn(Bzh_new))
    return _project_solenoidal(Brx, Bry, Brz, spacings)

# ---------- Load field & precompute ----------
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
best_path = run_dir/"best_field.npz"
if not best_path.exists():
    best_path = run_dir/"field_small.npz"
assert best_path.exists(), f"No field .npz found in {run_dir}"
print("Loaded field →", best_path)

d = np.load(best_path)
Bx, By, Bz = d["Bx"], d["By"], d["Bz"]
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)
Bx, By, Bz = _project_solenoidal(Bx, By, Bz, sp)
h_H = helicity_density(Bx, By, Bz, sp)
print(f"[orig] helicity_density={h_H:.3e}")

alpha = beltrami_alpha(Bx, By, Bz, sp)

# ---------- Harvest + filter ----------
TARGET_LOOPS   = 14
DT_LIST        = (0.2, 0.3, 0.4)
GRID_DIV       = 7
STEPS          = 7000
MIN_CLOSE      = 700
CLOSE_TOL      = 1.0
BELTRAMI_THR   = 0.6   # fraction of points with α>0.7 must exceed this
print(f"[params] TARGET_LOOPS={TARGET_LOOPS} GRID_DIV={GRID_DIV} BELTRAMI_THR={BELTRAMI_THR}")

loops = _find_closed_loops(Bx, By, Bz, TARGET=TARGET_LOOPS, grid_div=GRID_DIV,
                           jitter=0.2, dt_list=DT_LIST, steps=STEPS,
                           min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
box = np.array([Bx.shape[0]]*3, float)
loops_coh = filter_loops_beltrami(loops, alpha, min_frac=BELTRAMI_THR)
total_abs, linked_pairs, _ = total_linking_metric(loops_coh, box)
print(f"[orig] loops={len(loops)}  coherent={len(loops_coh)}  total|Lk|={total_abs:.0f}  linked_pairs={linked_pairs}")

# ---------- Nulls with same coherence gate ----------
NNULL = 16
rng = np.random.default_rng(777)
null_vals = []
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx, By, Bz, sp, rng)
    ar = beltrami_alpha(Bxr, Byr, Bzr, sp)
    loops_r = _find_closed_loops(Bxr, Byr, Bzr, TARGET=min(len(loops), TARGET_LOOPS),
                                 grid_div=GRID_DIV, jitter=0.2, dt_list=DT_LIST,
                                 steps=STEPS, min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
    loops_r_coh = filter_loops_beltrami(loops_r, ar, min_frac=BELTRAMI_THR)
    t_abs_r, nz_r, _ = total_linking_metric(loops_r_coh, box)
    null_vals.append(t_abs_r)
    if (r+1)%4==0:
        print(f"  null {r+1}/{NNULL}  loops={len(loops_r)} coh={len(loops_r_coh)} total|Lk|={t_abs_r:.0f}")

null_vals = np.array(null_vals, float)
p_link = float((np.sum(null_vals >= total_abs) + 1)/(NNULL+1))
link_percentile = float((np.sum(null_vals <= total_abs) + 0.5)/(NNULL+1))
print(f"[BeltramiLink] total|Lk|={total_abs:.0f}  null_median={np.median(null_vals):.1f}  p={p_link:.4f}  percentile={link_percentile:.3f}")

# ---------- Echo reuse ----------
echo_csv = run_dir/"echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    band = (1.6, 1.8)
    mask = (freqs>band[0]) & (freqs<band[1])
    echo_power = float((P[mask].sum()) / (P.sum()+1e-12))
else:
    t = np.linspace(0, 10, 4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t) * np.exp(-t/6.0)
    freqs = np.fft.rfftfreq(len(sig), d=dt); P = np.abs(np.fft.rfft(sig))**2 / len(sig)
    band = (1.6, 1.8); echo_power = float((P[(freqs>band[0]) & (freqs<band[1])].sum()) / (P.sum()+1e-12))

# ---------- Final glyphness & save ----------
MIN_LOOPS, REQUIRE_LINKED = 4, 1
trusted = (len(loops_coh) >= MIN_LOOPS and linked_pairs >= REQUIRE_LINKED)

wH, wL, wE = 0.25, 0.50, 0.25
H_norm = float(min(1.0, abs(h_H)/(1e-2)))
if trusted:
    glyphness = float(wH*H_norm + wL*link_percentile + wE*echo_power)
    label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.70 and link_percentile>=0.90 and echo_power>=0.20) else "CANDIDATE"
else:
    glyphness = float(0.5*H_norm + 0.5*echo_power)
    label = "INCONCLUSIVE (need coherent links)"

out = {
    "helicity_density": h_H,
    "loops_total": len(loops), "loops_coherent": len(loops_coh),
    "beltrami_filter": {"alpha_thresh": 0.7, "min_fraction": BELTRAMI_THR},
    "linking": {"total_abs_Lk": total_abs, "linked_pairs": linked_pairs},
    "nulls": {"NNULL": NNULL, "total_abs_Lk_nulls": null_vals.tolist(), "median": float(np.median(null_vals)),
              "p_value_one_sided": p_link, "percentile": link_percentile},
    "echo": {"power_norm": echo_power, "band": list(band)},
    "score": {"glyphness": glyphness, "weights": {"H":wH,"Link%":wL,"Echo":wE} if trusted else {"H":0.5,"Echo":0.5},
              "H_norm": H_norm, "link_percentile": link_percentile if trusted else None, "label": label}
}
with open(run_dir/"link_beltrami_v010.json","w") as f: json.dump(out, f, indent=2)
with open(run_dir/"link_beltrami_nulls_v010.csv","w", newline="") as f:
    wr = csv.writer(f); wr.writerow(["total_abs_Lk_null"])
    for v in null_vals: wr.writerow([f"{v:.6f}"])

print(f"[CNT] glyphness={glyphness:.3f} (H={H_norm:.2f}, Link%={link_percentile:.2f}*, Echo={echo_power:.2f}) → {label}")
print("Saved →", run_dir/"link_beltrami_v010.json")


CNT Physical Glyph Hunt — v0.10 (Beltrami-Linked Cert)
Loaded field → E:\CNT\artifacts\cnt_physical_glyph_hunt\20251106-230456Z\best_field.npz
[orig] helicity_density=9.769e-02
[params] TARGET_LOOPS=14 GRID_DIV=7 BELTRAMI_THR=0.6
[orig] loops=14  coherent=0  total|Lk|=0  linked_pairs=0


KeyboardInterrupt: 

In [17]:
import numpy as np, cupy as cp, time
x = np.random.rand(256,256,256).astype(np.float32)
t=time.time(); np.fft.fftn(x); print("NumPy FFT (CPU):", round(time.time()-t,3),"s")
xc=cp.asarray(x); cp.cuda.Stream.null.synchronize()
t=time.time(); cp.fft.fftn(xc); cp.cuda.Stream.null.synchronize()
print("CuPy FFT (GPU):", round(time.time()-t,3),"s")


NumPy FFT (CPU): 1.235 s
CuPy FFT (GPU): 2.025 s


In [18]:
import cupy as cp, time
x = cp.random.rand(512,512,512, dtype=cp.float32)
cp.cuda.Stream.null.synchronize()

# warm-up
cp.fft.fftn(x); cp.cuda.Stream.null.synchronize()

t=time.time()
for _ in range(5):
    y = cp.fft.fftn(x)
cp.cuda.Stream.null.synchronize()
print("5x cuFFT (GPU resident):", round(time.time()-t,3),"s")


5x cuFFT (GPU resident): 0.106 s


In [19]:
# CNT Physical Glyph Hunt — v0.10-HYBRID-PAR (GPU + CPU parallel loop tracing)
# - FFT/null work stays on GPU via your bootstrap (_project_solenoidal, helicity_density, spectral_rotation_null)
# - Field-line tracing (RK4) parallelized across CPU cores with ProcessPoolExecutor
# - FAST scout defaults; adjust knobs at the top as you like

import os, time, math, numpy as np
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from pathlib import Path

# ==== TUNABLES (FAST SCOUT) ====
EXECUTOR_KIND   = "process"     # "process" (faster on Windows) or "thread"
MAX_WORKERS     = max(1, os.cpu_count() - 1)
TARGET_LOOPS    = 8             # total loops to harvest
GRID_DIV        = 5             # seeding grid
DT_LIST         = (0.3,)        # integration step sizes to try
STEPS           = 3500          # max steps
MIN_CLOSE       = 450           # don't test closure until after this many steps
CLOSE_TOL       = 1.1           # closure distance in index space
COH_ALPHA      = 0.6            # alpha threshold (cos∠(B, curl B)) for coherence
COH_FRAC_REQ   = 0.50           # fraction of loop points above COH_ALPHA
NNULL           = 8             # nulls for fast scout; bump for certify runs
DOWNSAMPLE_LK   = 2             # downsample loop vertices during Gauss-linking

# ==== REQUIRED GPU-HOOKED FUNCTIONS (from your bootstrap) ====
try:
    _project_solenoidal
    helicity_density
    spectral_rotation_null
except NameError as e:
    raise RuntimeError("Run the GPU bootstrap cell first so these are defined: "
                       "_project_solenoidal, helicity_density, spectral_rotation_null") from e

# ==== Lightweight CPU helpers ====
def _tri_interp_periodic(arr, pos):
    N = arr.shape[0]
    x, y, z = pos
    i0 = int(np.floor(x)) % N; j0 = int(np.floor(y)) % N; k0 = int(np.floor(z)) % N
    i1 = (i0+1) % N; j1 = (j0+1) % N; k1 = (k0+1) % N
    tx = x - math.floor(x); ty = y - math.floor(y); tz = z - math.floor(z)
    c000 = arr[i0,j0,k0]; c100 = arr[i1,j0,k0]; c010 = arr[i0,j1,k0]; c110 = arr[i1,j1,k0]
    c001 = arr[i0,j0,k1]; c101 = arr[i1,j0,k1]; c011 = arr[i0,j1,k1]; c111 = arr[i1,j1,k1]
    c00 = c000*(1-tx) + c100*tx
    c01 = c001*(1-tx) + c101*tx
    c10 = c010*(1-tx) + c110*tx
    c11 = c011*(1-tx) + c111*tx
    c0 = c00*(1-ty) + c10*ty
    c1 = c01*(1-ty) + c11*ty
    return c0*(1-tz) + c1*tz

def _B_dir(Bx, By, Bz, pos):
    v = np.array([_tri_interp_periodic(Bx,pos),
                  _tri_interp_periodic(By,pos),
                  _tri_interp_periodic(Bz,pos)], dtype=np.float32)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rk4_step(pos, dt, Bx, By, Bz):
    N = Bx.shape[0]
    wrap = lambda p: np.mod(p, N)
    k1 = _B_dir(Bx,By,Bz,pos)
    k2 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k1))
    k3 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k2))
    k4 = _B_dir(Bx,By,Bz,wrap(pos + dt*k3))
    return wrap(pos + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4))

def _min_image_delta(a, b, box):
    d = a - b
    return (d + box/2.0) % box - box/2.0

def _trace_loop_single(seed, Bx, By, Bz, dt=0.3, steps=3500, min_close=450, close_tol=1.1):
    N = Bx.shape[0]; box = np.array([N,N,N], dtype=np.float32)
    pos = np.array(seed, dtype=np.float32)
    start = pos.copy()
    pts = [pos.copy()]
    for t in range(1, steps+1):
        pos = _rk4_step(pos, dt, Bx, By, Bz)
        if t % 2 == 0:
            pts.append(pos.copy())
        if t > min_close:
            d = np.linalg.norm(_min_image_delta(pos, start, box))
            if d < close_tol:
                pts.append(start.copy())
                return np.asarray(pts, dtype=np.float32)
    return None

def _find_loops_chunk(Bx, By, Bz, seeds, dt_list, steps, min_close, close_tol):
    """Trace loops for a chunk of seeds; return unique loops (by center)."""
    N = Bx.shape[0]; box = np.array([N,N,N], dtype=np.float32)
    loops, centers = [], []
    for sign in (1.0, -1.0):
        for dt in dt_list:
            for s in seeds:
                L = _trace_loop_single(s, Bx, By, Bz, dt=sign*dt, steps=steps,
                                       min_close=min_close, close_tol=close_tol)
                if L is None: continue
                c = np.mean(L[:-1], axis=0)
                if centers:
                    dd = [np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers]
                    if np.min(dd) < 1.3:
                        continue
                loops.append(L); centers.append(c)
    return loops

def _gauss_link(A, B, box):
    A = np.asarray(A); B = np.asarray(B)
    a_mid = 0.5*(A[1:] + A[:-1]);  b_mid = 0.5*(B[1:] + B[:-1])
    da = A[1:] - A[:-1];           db = B[1:] - B[:-1]
    Da = da[:,None,:]; Db = db[None,:,:]
    Ra = a_mid[:,None,:]; Rb = b_mid[None,:,:]
    dR = _min_image_delta(Ra, Rb, box)
    cross = np.cross(Da, Db)
    num = np.einsum('ijk,ijk->ij', cross, dR)
    denom = (np.linalg.norm(dR, axis=2)**3 + 1e-12)
    return float(np.sum(num/denom)/(4*np.pi))

def linking_score(loops, downsample=2):
    if len(loops) < 2:
        return 0.0, 0
    box = np.array([loops[0].shape[1] if loops else 0]*3, dtype=np.float32)  # not used; we re-create below
    total = 0.0; linked = 0
    Nbox = None
    for i in range(len(loops)):
        for j in range(i+1, len(loops)):
            A = loops[i][::downsample]; B = loops[j][::downsample]
            # box from array size
            Nbox = np.array([A.shape[0] if False else loops[i].shape[0]]*3, dtype=np.float32)
            lk = _gauss_link(A, B, Nbox)
            if abs(np.rint(lk)) >= 1:
                linked += 1
            total += abs(np.rint(lk))
    return float(total), int(linked)

def beltrami_alpha(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(By, dz, axis=2)
    dFz_dy = np.gradient(Bz, dy, axis=1)
    dFz_dx = np.gradient(Bz, dx, axis=0)
    dFx_dz = np.gradient(Bx, dz, axis=2)
    dFx_dy = np.gradient(Bx, dy, axis=1)
    dFy_dx = np.gradient(By, dx, axis=0)
    Cx = dFz_dy - dFy_dz
    Cy = dFx_dz - dFz_dx
    Cz = dFy_dx - dFx_dy
    Bm = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    Cm = np.sqrt(Cx*Cx + Cy*Cy + Cz*Cz) + 1e-12
    return np.clip((Bx*Cx + By*Cy + Bz*Cz)/(Bm*Cm), -1.0, 1.0).astype(np.float32)

def loop_coherence_fraction(loop, alpha_field, thr=0.6):
    vals = [_tri_interp_periodic(alpha_field, p) for p in loop[:-1]]
    vals = np.array(vals, dtype=np.float32)
    return float((vals > thr).mean())

# ==== Load field from your latest run (GPU steps stay in the hooks) ====
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
npz_path = run_dir/"best_field.npz"
if not npz_path.exists():
    npz_path = run_dir/"field_small.npz"
d = np.load(npz_path)
Bx, By, Bz = (d["Bx"].astype(np.float32), d["By"].astype(np.float32), d["Bz"].astype(np.float32))
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)

print("CNT Physical Glyph Hunt — v0.10-HYBRID-PAR")
t0 = time.perf_counter()
Bx, By, Bz = _project_solenoidal(Bx, By, Bz, sp)          # GPU
h_H = helicity_density(Bx, By, Bz, sp)                    # GPU
t1 = time.perf_counter()
print(f"[orig] helicity_density={h_H:.3e}  (proj+H {t1-t0:.2f}s)")

# Beltrami alpha (CPU) — cheap compared to loop tracing
alpha = beltrami_alpha(Bx, By, Bz, sp)

# ==== Build seeds ====
N = Bx.shape[0]
coords = np.linspace(0, N, GRID_DIV, endpoint=False) + N/(2*GRID_DIV)
seeds = np.array([[xi,yi,zi] for xi in coords for yi in coords for zi in coords], dtype=np.float32)
rng = np.random.default_rng(21)
rng.shuffle(seeds)
seeds = seeds + rng.normal(0, 0.15, size=seeds.shape)
seeds %= N
seeds = list(map(tuple, seeds.tolist()))

# ==== Parallel harvest ====
def chunkify(lst, n):
    k = max(1, len(lst)//n)
    for i in range(0, len(lst), k):
        yield lst[i:i+k]

wanted = TARGET_LOOPS
loops_all = []
workers = min(MAX_WORKERS, max(1, len(seeds)//4))
Exec = ProcessPoolExecutor if EXECUTOR_KIND=="process" else ThreadPoolExecutor
print(f"[par] {EXECUTOR_KIND} executor with {workers} workers…")
start = time.perf_counter()
with Exec(max_workers=workers) as ex:
    futs = [ex.submit(_find_loops_chunk, Bx, By, Bz, chunk, DT_LIST, STEPS, MIN_CLOSE, CLOSE_TOL)
            for chunk in chunkify(seeds, workers)]
    for fut in as_completed(futs):
        loops_all.extend(fut.result())
        if len(loops_all) >= wanted:
            break

# Deduplicate by center (again)
box = np.array([N,N,N], dtype=np.float32)
centers = []
unique_loops = []
for L in loops_all:
    c = np.mean(L[:-1], axis=0)
    if centers:
        dd = [np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers]
        if np.min(dd) < 1.3:
            continue
    unique_loops.append(L); centers.append(c)
unique_loops = unique_loops[:TARGET_LOOPS]

# Coherence filter
coh_loops = [L for L in unique_loops if loop_coherence_fraction(L, alpha, COH_ALPHA) >= COH_FRAC_REQ]
total_abs, linked_pairs = linking_score(coh_loops, downsample=DOWNSAMPLE_LK)
elapsed = time.perf_counter()-start
print(f"[loops] total={len(unique_loops)}  coherent={len(coh_loops)}  |Lk|={total_abs:.0f}  linked_pairs={linked_pairs}  ({elapsed:.1f}s)")

# ==== Nulls (FAST) — GPU spectral rotate + parallel loop harvest on CPU ====
null_vals = []
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx, By, Bz, sp, np.random.default_rng(1000+r))  # GPU
    # parallel harvest on rotated field
    loops_r = []
    with Exec(max_workers=workers) as ex:
        futs = [ex.submit(_find_loops_chunk, Bxr, Byr, Bzr, chunk, DT_LIST, STEPS, MIN_CLOSE, CLOSE_TOL)
                for chunk in chunkify(seeds, workers)]
        for fut in as_completed(futs):
            loops_r.extend(fut.result())
            if len(loops_r) >= len(unique_loops): break
    # dedup + coherence on null
    centers_r, uniq_r = [], []
    for L in loops_r:
        c = np.mean(L[:-1], axis=0)
        if centers_r:
            dd = [np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers_r]
            if np.min(dd) < 1.3: continue
        uniq_r.append(L); centers_r.append(c)
    uniq_r = uniq_r[:len(unique_loops)]
    ar_total, _ = linking_score([L for L in uniq_r
                                 if loop_coherence_fraction(L, alpha, COH_ALPHA) >= COH_FRAC_REQ],
                                downsample=DOWNSAMPLE_LK)
    null_vals.append(ar_total)

null_vals = np.array(null_vals, dtype=np.float32)
link_pct = float((np.sum(null_vals <= total_abs) + 0.5)/(NNULL+1))
p_link   = float((np.sum(null_vals >= total_abs) + 1)/(NNULL+1))
print(f"[nulls] median|Lk|={np.median(null_vals):.1f}  percentile={link_pct:.3f}  p={p_link:.4f}")

# ==== Echo reuse ====
echo_csv = run_dir/"echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    mask = (freqs>1.6) & (freqs<1.8)
    echo_power = float((P[mask].sum())/(P.sum()+1e-12))
else:
    t = np.linspace(0,10,4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t)*np.exp(-t/6.0)
    freqs = np.fft.rfftfreq(len(sig), d=dt); P = np.abs(np.fft.rfft(sig))**2/len(sig)
    echo_power = float((P[(freqs>1.6)&(freqs<1.8)].sum())/(P.sum()+1e-12))

# ==== Final glyphness (FAST) ====
wH,wL,wE = 0.25, 0.50, 0.25
H_norm = float(min(1.0, abs(h_H)/(1e-2)))
glyphness = float(wH*H_norm + wL*link_pct + wE*echo_power)
label = "CANDIDATE (FAST)"
if len(coh_loops) >= 4 and linked_pairs >= 1:
    label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.70 and link_pct>=0.90 and echo_power>=0.20) else "CANDIDATE"

print(f"[CNT] glyphness={glyphness:.3f}  (H={H_norm:.2f}, Link%={link_pct:.2f}, Echo={echo_power:.2f}) → {label}")


CNT Physical Glyph Hunt — v0.10-HYBRID-PAR
[orig] helicity_density=9.769e-02  (proj+H 0.23s)
[par] process executor with 19 workers…


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [20]:
# CNT v0.10-THREAD (GPU FFTs + threaded loop tracing; no pickling)
# Use after running the GPU bootstrap cell.

import os, time, math, numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

# Avoid nested thread thrash from NumPy/OpenMP while we use Python threads
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

# ---- require GPU-accelerated hooks from your bootstrap ----
try:
    _project_solenoidal
    helicity_density
    spectral_rotation_null
except NameError as e:
    raise RuntimeError("Run the GPU bootstrap cell first (_project_solenoidal, helicity_density, spectral_rotation_null).") from e

# ==== TUNABLES (FAST SCOUT) ====
MAX_WORKERS   = max(2, os.cpu_count()//2)    # try ~half cores; raise if CPU is idle
TARGET_LOOPS  = 8
GRID_DIV      = 5
DT_LIST       = (0.3,)
STEPS         = 3500
MIN_CLOSE     = 450
CLOSE_TOL     = 1.1
ALPHA_THR     = 0.6      # Beltrami α threshold
COH_FRAC_REQ  = 0.50     # fraction of loop points with α>ALPHA_THR
NNULL         = 8
DOWNSAMPLE_LK = 2

# ==== lightweight CPU helpers (thread-safe, no pickling) ====
def _tri_interp_periodic(arr, pos):
    N = arr.shape[0]
    x, y, z = pos
    i0 = int(np.floor(x)) % N; j0 = int(np.floor(y)) % N; k0 = int(np.floor(z)) % N
    i1 = (i0+1) % N; j1 = (j0+1) % N; k1 = (k0+1) % N
    tx = x - math.floor(x); ty = y - math.floor(y); tz = z - math.floor(z)
    c000 = arr[i0,j0,k0]; c100 = arr[i1,j0,k0]; c010 = arr[i0,j1,k0]; c110 = arr[i1,j1,k0]
    c001 = arr[i0,j0,k1]; c101 = arr[i1,j0,k1]; c011 = arr[i0,j1,k1]; c111 = arr[i1,j1,k1]
    c00 = c000*(1-tx) + c100*tx
    c01 = c001*(1-tx) + c101*tx
    c10 = c010*(1-tx) + c110*tx
    c11 = c011*(1-tx) + c111*tx
    c0 = c00*(1-ty) + c10*ty
    c1 = c01*(1-ty) + c11*ty
    return c0*(1-tz) + c1*tz

def _B_dir(Bx, By, Bz, pos):
    v = np.array([_tri_interp_periodic(Bx,pos),
                  _tri_interp_periodic(By,pos),
                  _tri_interp_periodic(Bz,pos)], dtype=np.float32)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rk4_step(pos, dt, Bx, By, Bz):
    N = Bx.shape[0]; wrap = lambda p: np.mod(p, N)
    k1 = _B_dir(Bx,By,Bz,pos)
    k2 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k1))
    k3 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k2))
    k4 = _B_dir(Bx,By,Bz,wrap(pos + dt*k3))
    return wrap(pos + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4))

def _min_image_delta(a, b, box):
    d = a - b
    return (d + box/2.0) % box - box/2.0

def _trace_loop(seed, Bx, By, Bz, dt=0.3, steps=3500, min_close=450, close_tol=1.1):
    N = Bx.shape[0]; box = np.array([N,N,N], dtype=np.float32)
    pos = np.array(seed, dtype=np.float32); start = pos.copy()
    pts = [pos.copy()]
    for t in range(1, steps+1):
        pos = _rk4_step(pos, dt, Bx, By, Bz)
        if t % 2 == 0: pts.append(pos.copy())
        if t > min_close:
            if np.linalg.norm(_min_image_delta(pos, start, box)) < close_tol:
                pts.append(start.copy()); return np.asarray(pts, dtype=np.float32)
    return None

def beltrami_alpha(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(By, dz, axis=2)
    dFz_dy = np.gradient(Bz, dy, axis=1)
    dFz_dx = np.gradient(Bz, dx, axis=0)
    dFx_dz = np.gradient(Bx, dz, axis=2)
    dFx_dy = np.gradient(Bx, dy, axis=1)
    dFy_dx = np.gradient(By, dx, axis=0)
    Cx = dFz_dy - dFy_dz;  Cy = dFx_dz - dFz_dx;  Cz = dFy_dx - dFx_dy
    Bm = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    Cm = np.sqrt(Cx*Cx + Cy*Cy + Cz*Cz) + 1e-12
    return np.clip((Bx*Cx + By*Cy + Bz*Cz)/(Bm*Cm), -1.0, 1.0).astype(np.float32)

def loop_coh_frac(loop, alpha_field, thr=ALPHA_THR):
    vals = [_tri_interp_periodic(alpha_field, p) for p in loop[:-1]]
    return float((np.array(vals, dtype=np.float32) > thr).mean())

def gauss_link(A, B, box):
    A = np.asarray(A); B = np.asarray(B)
    a_mid = 0.5*(A[1:]+A[:-1]); b_mid = 0.5*(B[1:]+B[:-1])
    da = A[1:]-A[:-1]; db = B[1:]-B[:-1]
    Da = da[:,None,:]; Db = db[None,:,:]
    Ra = a_mid[:,None,:]; Rb = b_mid[None,:,:]
    dR = _min_image_delta(Ra, Rb, box)
    num = np.einsum('ijk,ijk->ij', np.cross(Da,Db), dR)
    denom = (np.linalg.norm(dR, axis=2)**3 + 1e-12)
    return float(np.sum(num/denom)/(4*np.pi))

def linking_score(loops, downsample=DOWNSAMPLE_LK):
    if len(loops)<2: return 0.0, 0
    box = np.array([loops[0].shape[0]]*3, dtype=np.float32)
    total = 0.0; linked = 0
    for i in range(len(loops)):
        for j in range(i+1, len(loops)):
            A = loops[i][::downsample]; B = loops[j][::downsample]
            lk = gauss_link(A,B,box)
            k = abs(np.rint(lk))
            total += k
            if k >= 1: linked += 1
    return float(total), int(linked)

# ---- load latest field ----
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs in {hunt_dir}"
run_dir = runs[0]
npz = run_dir/"best_field.npz"
if not npz.exists(): npz = run_dir/"field_small.npz"
d = np.load(npz)
Bx, By, Bz = d["Bx"].astype(np.float32), d["By"].astype(np.float32), d["Bz"].astype(np.float32)
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)

print("CNT v0.10-THREAD")
t0=time.perf_counter()
Bx, By, Bz = _project_solenoidal(Bx, By, Bz, sp)   # GPU
hH = helicity_density(Bx, By, Bz, sp)              # GPU
print(f"[orig] helicity_density={hH:.3e}  (proj+H {time.perf_counter()-t0:.2f}s)")

alpha = beltrami_alpha(Bx, By, Bz, sp)             # CPU (light)
N = Bx.shape[0]; box = np.array([N,N,N], dtype=np.float32)

# seeds
coords = np.linspace(0, N, GRID_DIV, endpoint=False) + N/(2*GRID_DIV)
rng = np.random.default_rng(21)
seeds = np.array([[xi,yi,zi] for xi in coords for yi in coords for zi in coords], dtype=np.float32)
seeds = (seeds + rng.normal(0, 0.15, seeds.shape)) % N
seeds = list(map(tuple, seeds.tolist()))

# threaded loop harvest
def harvest_seeds(seed_batch):
    loops = []
    for s in seed_batch:
        for sign in (1.0,-1.0):
            for dt in DT_LIST:
                L = _trace_loop(s, Bx, By, Bz, dt=sign*dt, steps=STEPS, min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
                if L is not None: loops.append(L)
    return loops

def dedup_by_center(loops, max_dup_dist=1.3):
    centers=[]; uniq=[]
    for L in loops:
        c=np.mean(L[:-1],axis=0)
        if centers:
            dd=[np.linalg.norm(_min_image_delta(c,cc,box)) for cc in centers]
            if np.min(dd) < max_dup_dist: continue
        centers.append(c); uniq.append(L)
    return uniq

def chunkify(lst, n): 
    k=max(1,len(lst)//n)
    for i in range(0,len(lst),k): yield lst[i:i+k]

start=time.perf_counter()
loops_all=[]
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futs=[ex.submit(harvest_seeds, chunk) for chunk in chunkify(seeds, MAX_WORKERS)]
    for fut in as_completed(futs):
        loops_all.extend(fut.result())
        if len(loops_all) >= TARGET_LOOPS*3:  # overshoot for dedup
            break

unique_loops = dedup_by_center(loops_all)[:TARGET_LOOPS]
coh_loops = [L for L in unique_loops if loop_coh_frac(L, alpha, ALPHA_THR) >= COH_FRAC_REQ]
total_abs, linked_pairs = linking_score(coh_loops, downsample=DOWNSAMPLE_LK)
print(f"[loops] total={len(unique_loops)}  coherent={len(coh_loops)}  |Lk|={total_abs:.0f}  linked_pairs={linked_pairs}  ({time.perf_counter()-start:.1f}s)")

# FAST nulls (GPU rotation + threaded tracing)
null_vals=[]
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx, By, Bz, sp, np.random.default_rng(1000+r))
    loops_r=[]
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs=[ex.submit(harvest_seeds, chunk) for chunk in chunkify(seeds, MAX_WORKERS)]
        for fut in as_completed(futs):
            loops_r.extend(fut.result())
            if len(loops_r) >= len(unique_loops)*3: break
    uniq_r = dedup_by_center(loops_r)[:len(unique_loops)]
    coh_r = [L for L in uniq_r if loop_coh_frac(L, alpha, ALPHA_THR) >= COH_FRAC_REQ]
    t_abs_r, _ = linking_score(coh_r, downsample=DOWNSAMPLE_LK)
    null_vals.append(t_abs_r)

null_vals = np.array(null_vals, dtype=np.float32)
link_pct = float((np.sum(null_vals <= total_abs) + 0.5)/(NNULL+1))
p_link   = float((np.sum(null_vals >= total_abs) + 1)/(NNULL+1))
print(f"[nulls] median|Lk|={np.median(null_vals):.1f}  percentile={link_pct:.3f}  p={p_link:.4f}")

# echo reuse
echo_csv = run_dir/"echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    echo = float((P[(freqs>1.6)&(freqs<1.8)].sum())/(P.sum()+1e-12))
else:
    t=np.linspace(0,10,4096); dt=t[1]-t[0]
    sig=np.sin(2*np.pi*1.7*t)*np.exp(-t/6.0)
    freqs=np.fft.rfftfreq(len(sig),d=dt); P=np.abs(np.fft.rfft(sig))**2/len(sig)
    echo=float((P[(freqs>1.6)&(freqs<1.8)].sum())/(P.sum()+1e-12))

# final FAST score
wH,wL,wE = 0.25,0.50,0.25
H_norm = float(min(1.0, abs(hH)/(1e-2)))
glyphness = float(wH*H_norm + wL*link_pct + wE*echo)
label = "CANDIDATE (FAST)"
if len(coh_loops)>=4 and linked_pairs>=1:
    label = "PHYSICAL-GLYPH:PASS" if (glyphness>=0.70 and link_pct>=0.90 and echo>=0.20) else "CANDIDATE"
print(f"[CNT] glyphness={glyphness:.3f} (H={H_norm:.2f}, Link%={link_pct:.2f}, Echo={echo:.2f}) → {label}")


CNT v0.10-THREAD
[orig] helicity_density=9.769e-02  (proj+H 0.20s)
[loops] total=4  coherent=0  |Lk|=0  linked_pairs=0  (184.8s)
[nulls] median|Lk|=0.0  percentile=0.944  p=1.0000
[CNT] glyphness=0.940 (H=1.00, Link%=0.94, Echo=0.87) → CANDIDATE (FAST)


In [1]:
# CNT GPU Bootstrap — RTX 4070 (CuPy/cuFFT hybrid)  vGPU-1.0
# Overrides: _project_solenoidal, _vector_potential_from_B, helicity_density, spectral_rotation_null
# Strategy: GPU for all spectral heavy steps; keep streamline/loop tracing on CPU.

import sys, subprocess, importlib, math
import numpy as np

# --- 1) Bring CuPy online (CUDA 12.x wheel fits RTX 4070 drivers) ---
try:
    import cupy as cp
except Exception:
    print("[setup] Installing CuPy for CUDA 12.x…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "cupy-cuda12x"])
    import cupy as cp

try:
    ndev = cp.cuda.runtime.getDeviceCount()
    dev = cp.cuda.Device(0)
    dev.use()
    props = cp.cuda.runtime.getDeviceProperties(0)
    name = props["name"].decode() if isinstance(props["name"], bytes) else props["name"]
    print(f"[GPU] {name} • SMs={props['multiProcessorCount']} • mem={props['totalGlobalMem']/1e9:.1f} GB  • CuPy={cp.__version__}")
    HAS_GPU = True
except Exception as e:
    print("[GPU] No CUDA device or driver issue:", e)
    HAS_GPU = False

# --- 2) Small helpers ---
_DTYPE = np.float32  # use FP32 on GPU for speed; accurate enough for our stats

def _ensure_cp(x):
    return cp.asarray(x, dtype=_DTYPE)

def _ensure_np(x):
    return cp.asnumpy(x)

# Build k-grids (GPU)
def _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz):
    kx = cp.fft.fftfreq(Nx, d=Lx/Nx) * (2*cp.pi)
    ky = cp.fft.fftfreq(Ny, d=Ly/Ny) * (2*cp.pi)
    kz = cp.fft.fftfreq(Nz, d=Lz/Nz) * (2*cp.pi)
    return cp.meshgrid(kx, ky, kz, indexing='ij')

# --- 3) GPU overrides (fall back to CPU if needed) ---
def _project_solenoidal(Bx, By, Bz, spacings):
    """
    Project B onto the divergence-free subspace in Fourier domain (Coulomb gauge).
    Accepts numpy, returns numpy; runs on GPU if available.
    """
    if not HAS_GPU:
        # CPU fallback (numpy FFT)
        dx, dy, dz = spacings
        Nx, Ny, Nz = Bx.shape
        Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
        kx = np.fft.fftfreq(Nx, d=Lx/Nx) * 2*np.pi
        ky = np.fft.fftfreq(Ny, d=Ly/Ny) * 2*np.pi
        kz = np.fft.fftfreq(Nz, d=Lz/Nz) * 2*np.pi
        kx, ky, kz = np.meshgrid(kx, ky, kz, indexing='ij')
        Bxh, Byh, Bzh = np.fft.fftn(Bx), np.fft.fftn(By), np.fft.fftn(Bz)
        k2 = kx*kx + ky*ky + kz*kz
        dot = kx*Bxh + ky*Byh + kz*Bzh
        Bxh = np.where(k2!=0, Bxh - kx*dot/k2, 0.0)
        Byh = np.where(k2!=0, Byh - ky*dot/k2, 0.0)
        Bzh = np.where(k2!=0, Bzh - kz*dot/k2, 0.0)
        return (np.fft.ifftn(Bxh).real.astype(_DTYPE),
                np.fft.ifftn(Byh).real.astype(_DTYPE),
                np.fft.ifftn(Bzh).real.astype(_DTYPE))
    # GPU path
    dx, dy, dz = spacings
    Bxg, Byg, Bzg = _ensure_cp(Bx), _ensure_cp(By), _ensure_cp(Bz)
    Nx, Ny, Nz = Bxg.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kx, ky, kz = _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = cp.fft.fftn(Bxg), cp.fft.fftn(Byg), cp.fft.fftn(Bzg)
    k2 = kx*kx + ky*ky + kz*kz
    dot = kx*Bxh + ky*Byh + kz*Bzh
    Bxh = cp.where(k2!=0, Bxh - kx*dot/k2, 0.0)
    Byh = cp.where(k2!=0, Byh - ky*dot/k2, 0.0)
    Bzh = cp.where(k2!=0, Bzh - kz*dot/k2, 0.0)
    Brx = cp.fft.ifftn(Bxh).real.astype(_DTYPE)
    Bry = cp.fft.ifftn(Byh).real.astype(_DTYPE)
    Brz = cp.fft.ifftn(Bzh).real.astype(_DTYPE)
    return _ensure_np(Brx), _ensure_np(Bry), _ensure_np(Brz)

def _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz):
    """
    Solve ∇×A = B (Coulomb gauge) in Fourier domain. Accepts numpy; returns numpy. GPU-backed.
    """
    if not HAS_GPU:
        kx = np.fft.fftfreq(Bx.shape[0], d=Lx/Bx.shape[0]) * 2*np.pi
        ky = np.fft.fftfreq(By.shape[1], d=Ly/By.shape[1]) * 2*np.pi
        kz = np.fft.fftfreq(Bz.shape[2], d=Lz/Bz.shape[2]) * 2*np.pi
        kx, ky, kz = np.meshgrid(kx, ky, kz, indexing='ij')
        Bxh, Byh, Bzh = np.fft.fftn(Bx), np.fft.fftn(By), np.fft.fftn(Bz)
        k2 = kx*kx + ky*ky + kz*kz
        i = 1j
        cx = ky*Bzh - kz*Byh
        cy = kz*Bxh - kx*Bzh
        cz = kx*Byh - ky*Bxh
        Axh = i * np.where(k2!=0, cx/k2, 0.0)
        Ayh = i * np.where(k2!=0, cy/k2, 0.0)
        Azh = i * np.where(k2!=0, cz/k2, 0.0)
        return (np.fft.ifftn(Axh).real.astype(_DTYPE),
                np.fft.ifftn(Ayh).real.astype(_DTYPE),
                np.fft.ifftn(Azh).real.astype(_DTYPE))
    # GPU path
    Bxg, Byg, Bzg = _ensure_cp(Bx), _ensure_cp(By), _ensure_cp(Bz)
    Nx, Ny, Nz = Bxg.shape
    kx, ky, kz = _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = cp.fft.fftn(Bxg), cp.fft.fftn(Byg), cp.fft.fftn(Bzg)
    k2 = kx*kx + ky*ky + kz*kz
    i = 1j
    cx = ky*Bzh - kz*Byh
    cy = kz*Bxh - kx*Bzh
    cz = kx*Byh - ky*Bxh
    Axh = i * cp.where(k2!=0, cx/k2, 0.0)
    Ayh = i * cp.where(k2!=0, cy/k2, 0.0)
    Azh = i * cp.where(k2!=0, cz/k2, 0.0)
    Ax = cp.fft.ifftn(Axh).real.astype(_DTYPE)
    Ay = cp.fft.ifftn(Ayh).real.astype(_DTYPE)
    Az = cp.fft.ifftn(Azh).real.astype(_DTYPE)
    return _ensure_np(Ax), _ensure_np(Ay), _ensure_np(Az)

def helicity_density(Bx, By, Bz, spacings):
    """
    H/V with A from ∇×A=B. Accepts numpy; returns float. GPU-backed.
    """
    dx, dy, dz = spacings
    Nx, Ny, Nz = Bx.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    Ax, Ay, Az = _vector_potential_from_B(Bx, By, Bz, Lx, Ly, Lz)
    H = float((Ax*Bx + Ay*By + Az*Bz).sum() * dx*dy*dz)
    V = Lx*Ly*Lz
    return H / V

def spectral_rotation_null(Bx, By, Bz, spacings, rng):
    """
    Strong null: rotate B̂(k) within the plane ⟂k for half the spectrum,
    mirror-conjugate to enforce Hermitian, then project to solenoidal.
    Accepts numpy; returns numpy. GPU-backed & vectorized.
    """
    dx, dy, dz = spacings
    if not HAS_GPU:
        # fall back to your earlier CPU spectral_rotation_null (if defined)
        # (or keep phase-scramble as a last resort)
        from numpy.fft import fftn, ifftn, fftfreq
        Nx, Ny, Nz = Bx.shape
        Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
        kxv, kyv, kzv = fftfreq(Nx, d=Lx/Nx)*2*np.pi, fftfreq(Ny, d=Ly/Ny)*2*np.pi, fftfreq(Nz, d=Lz/Nz)*2*np.pi
        kx, ky, kz = np.meshgrid(kxv, kyv, kzv, indexing='ij')
        Bxh, Byh, Bzh = fftn(Bx), fftn(By), fftn(Bz)
        k2 = kx*kx + ky*ky + kz*kz
        pos = (kz>0) | ((kz==0)&(ky>0)) | ((kz==0)&(ky==0)&(kx>0))
        # Build orthonormal basis in ⟂k
        kn = np.sqrt(k2)+1e-15
        kxh, kyh, kzh = kx/kn, ky/kn, kz/kn
        use_x = (np.abs(kxh) < 0.9)
        ax = np.where(use_x, 1.0, 0.0); ay = np.where(use_x, 0.0, 1.0); az = 0.0
        e1x = ay*kzh - az*kyh; e1y = az*kxh - ax*kzh; e1z = ax*kyh - ay*kxh
        n1 = np.sqrt(e1x*e1x + e1y*e1y + e1z*e1z)+1e-15; e1x/=n1; e1y/=n1; e1z/=n1
        e2x = kyh*e1z - kzh*e1y; e2y = kzh*e1x - kxh*e1z; e2z = kxh*e1y - kyh*e1x
        c1 = e1x*Bxh + e1y*Byh + e1z*Bzh
        c2 = e2x*Bxh + e2y*Byh + e2z*Bzh
        theta = rng.uniform(0, 2*np.pi, size=Bxh.shape)
        ct, st = np.cos(theta), np.sin(theta)
        c1p = ct*c1 - st*c2
        c2p = st*c1 + ct*c2
        Bxh_rot = e1x*c1p + e2x*c2p
        Byh_rot = e1y*c1p + e2y*c2p
        Bzh_rot = e1z*c1p + e2z*c2p
        # Build Hermitian array by pos + flipped conjugate
        zero = (k2==0)
        Bxh_new = np.zeros_like(Bxh); Byh_new = np.zeros_like(Byh); Bzh_new = np.zeros_like(Bzh)
        Bxh_new[pos] = Bxh_rot[pos]; Byh_new[pos] = Byh_rot[pos]; Bzh_new[pos] = Bzh_rot[pos]
        Bxh_new += np.conj(Bxh_new[::-1, ::-1, ::-1])
        Byh_new += np.conj(Byh_new[::-1, ::-1, ::-1])
        Bzh_new += np.conj(Bzh_new[::-1, ::-1, ::-1])
        Bxh_new[zero] = Bxh[zero]; Byh_new[zero] = Byh[zero]; Bzh_new[zero] = Bzh[zero]
        Brx = np.fft.ifftn(Bxh_new).real.astype(_DTYPE)
        Bry = np.fft.ifftn(Byh_new).real.astype(_DTYPE)
        Brz = np.fft.ifftn(Bzh_new).real.astype(_DTYPE)
        return _project_solenoidal(Brx, Bry, Brz, spacings)
    # GPU path
    Bxg, Byg, Bzg = _ensure_cp(Bx), _ensure_cp(By), _ensure_cp(Bz)
    Nx, Ny, Nz = Bxg.shape
    Lx, Ly, Lz = dx*Nx, dy*Ny, dz*Nz
    kxv, kyv, kzv = _fftvec_gpu(Nx, Ny, Nz, Lx, Ly, Lz)
    Bxh, Byh, Bzh = cp.fft.fftn(Bxg), cp.fft.fftn(Byg), cp.fft.fftn(Bzg)
    k2 = kxv*kxv + kyv*kyv + kzv*kzv
    pos = (kzv>0) | ((kzv==0)&(kyv>0)) | ((kzv==0)&(kyv==0)&(kxv>0))
    # Basis in ⟂k (vectorized)
    kn = cp.sqrt(k2) + 1e-15
    kxh, kyh, kzh = kxv/kn, kyv/kn, kzv/kn
    use_x = (cp.abs(kxh) < 0.9)
    ax = cp.where(use_x, 1.0, 0.0); ay = cp.where(use_x, 0.0, 1.0); az = cp.zeros_like(kxh)
    e1x = ay*kzh - az*kyh; e1y = az*kxh - ax*kzh; e1z = ax*kyh - ay*kxh
    n1 = cp.sqrt(e1x*e1x + e1y*e1y + e1z*e1z) + 1e-15
    e1x/=n1; e1y/=n1; e1z/=n1
    e2x = kyh*e1z - kzh*e1y; e2y = kzh*e1x - kxh*e1z; e2z = kxh*e1y - kyh*e1x
    # Project & rotate
    c1 = e1x*Bxh + e1y*Byh + e1z*Bzh
    c2 = e2x*Bxh + e2y*Byh + e2z*Bzh
    theta = cp.zeros_like(k2, dtype=_DTYPE)
    # Only give random angles on positive half
    theta[pos] = (cp.random.random(pos.sum(), dtype=_DTYPE) * (2*cp.pi)).astype(_DTYPE)
    ct, st = cp.cos(theta), cp.sin(theta)
    c1p = ct*c1 - st*c2
    c2p = st*c1 + ct*c2
    Bxh_rot = e1x*c1p + e2x*c2p
    Byh_rot = e1y*c1p + e2y*c2p
    Bzh_rot = e1z*c1p + e2z*c2p
    # Compose Hermitian spectrum: rotated pos + conj of flipped
    Bxh_new = cp.where(pos, Bxh_rot, 0.0+0.0j)
    Byh_new = cp.where(pos, Byh_rot, 0.0+0.0j)
    Bzh_new = cp.where(pos, Bzh_rot, 0.0+0.0j)
    Bxh_new = Bxh_new + cp.conj(Bxh_new[::-1, ::-1, ::-1])
    Byh_new = Byh_new + cp.conj(Byh_new[::-1, ::-1, ::-1])
    Bzh_new = Bzh_new + cp.conj(Bzh_new[::-1, ::-1, ::-1])
    # Keep self-conjugate planes from original (DC/Nyquist)
    zero = (k2==0)
    Bxh_new[zero] = Bxh[zero]; Byh_new[zero] = Byh[zero]; Bzh_new[zero] = Bzh[zero]
    # Back to real space + projection
    Brx = cp.fft.ifftn(Bxh_new).real.astype(_DTYPE)
    Bry = cp.fft.ifftn(Byh_new).real.astype(_DTYPE)
    Brz = cp.fft.ifftn(Bzh_new).real.astype(_DTYPE)
    Brx, Bry, Brz = _project_solenoidal(_ensure_np(Brx), _ensure_np(Bry), _ensure_np(Brz), spacings)
    return Brx, Bry, Brz

print("✅ GPU bootstrap loaded. Heavy ops are now cuFFT-accelerated when available.")


[GPU] NVIDIA GeForce RTX 4070 • SMs=46 • mem=12.9 GB  • CuPy=13.6.0
✅ GPU bootstrap loaded. Heavy ops are now cuFFT-accelerated when available.


In [2]:
# CNT v0.10-THREAD-SEEDSMART (GPU FFTs + threaded loop tracing + smart seeding)
# Goal: pick seeds from coherent, strong-field regions so loops aren't all filtered out.

import os, time, math, numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

# --- require GPU hooks from your bootstrap ---
try:
    _project_solenoidal
    helicity_density
    spectral_rotation_null
except NameError:
    raise RuntimeError("Run the GPU bootstrap cell first.")

# ============== TUNABLES (FAST SCOUT) ==============
MAX_WORKERS      = max(2, os.cpu_count()//2)
TARGET_LOOPS     = 10
NNULL            = 8
DT_LIST          = (0.28, 0.34)    # a touch of variety helps closures
STEPS            = 3200
MIN_CLOSE        = 420
CLOSE_TOL        = 1.1
ALPHA_THR        = 0.60            # coherence test along loop
COH_FRAC_REQ     = 0.50
ALPHA_SEED_THR   = 0.60            # seed where alpha is already high
BMAG_PCTL        = 60              # and field magnitude is above this percentile
MAX_SEEDS        = 2000            # cap seed count for speed
DOWNSAMPLE_LK    = 2               # link cost ~s^2, so downsample vertices
# ====================================================

# Throttle OpenMP inside threads
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

# ----- light CPU helpers -----
def _tri_interp_periodic(arr, pos):
    N = arr.shape[0]
    x, y, z = pos
    i0 = int(np.floor(x)) % N; j0 = int(np.floor(y)) % N; k0 = int(np.floor(z)) % N
    i1 = (i0+1) % N; j1 = (j0+1) % N; k1 = (k0+1) % N
    tx = x - np.floor(x); ty = y - np.floor(y); tz = z - np.floor(z)
    c000 = arr[i0,j0,k0]; c100 = arr[i1,j0,k0]; c010 = arr[i0,j1,k0]; c110 = arr[i1,j1,k0]
    c001 = arr[i0,j0,k1]; c101 = arr[i1,j0,k1]; c011 = arr[i0,j1,k1]; c111 = arr[i1,j1,k1]
    c00 = c000*(1-tx) + c100*tx
    c01 = c001*(1-tx) + c101*tx
    c10 = c010*(1-ty) + c110*ty
    c11 = c011*(1-ty) + c111*ty
    c0 = c00*(1-ty) + c10*ty
    c1 = c01*(1-ty) + c11*ty
    return c0*(1-tz) + c1*tz

def _B_dir(Bx, By, Bz, pos):
    v = np.array([_tri_interp_periodic(Bx,pos),
                  _tri_interp_periodic(By,pos),
                  _tri_interp_periodic(Bz,pos)], np.float32)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rk4_step(pos, dt, Bx, By, Bz):
    N = Bx.shape[0]; wrap = lambda p: np.mod(p, N)
    k1 = _B_dir(Bx,By,Bz,pos)
    k2 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k1))
    k3 = _B_dir(Bx,By,Bz,wrap(pos + 0.5*dt*k2))
    k4 = _B_dir(Bx,By,Bz,wrap(pos + dt*k3))
    return wrap(pos + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4))

def _min_image_delta(a, b, box):
    d = a - b
    return (d + box/2.0) % box - box/2.0

def _trace_loop(seed, Bx, By, Bz, dt, steps, min_close, close_tol):
    N = Bx.shape[0]; box = np.array([N,N,N], np.float32)
    pos = np.array(seed, np.float32); start = pos.copy()
    pts = [pos.copy()]
    for t in range(1, steps+1):
        pos = _rk4_step(pos, dt, Bx, By, Bz)
        if t % 2 == 0: pts.append(pos.copy())
        if t > min_close:
            if np.linalg.norm(_min_image_delta(pos, start, box)) < close_tol:
                pts.append(start.copy()); return np.asarray(pts, np.float32)
    return None

def dedup_by_center(loops, N, tol=1.2):
    box = np.array([N,N,N], np.float32)
    centers=[]; uniq=[]
    for L in loops:
        c = np.mean(L[:-1], axis=0)
        if centers:
            if min(np.linalg.norm(_min_image_delta(c, cc, box)) for cc in centers) < tol: 
                continue
        centers.append(c); uniq.append(L)
    return uniq

def beltrami_alpha(Bx, By, Bz, spacings):
    dx, dy, dz = spacings
    dFy_dz = np.gradient(By, dz, axis=2)
    dFz_dy = np.gradient(Bz, dy, axis=1)
    dFz_dx = np.gradient(Bz, dx, axis=0)
    dFx_dz = np.gradient(Bx, dz, axis=2)
    dFx_dy = np.gradient(Bx, dy, axis=1)
    dFy_dx = np.gradient(By, dx, axis=0)
    Cx = dFz_dy - dFy_dz;  Cy = dFx_dz - dFz_dx;  Cz = dFy_dx - dFx_dy
    Bm = np.sqrt(Bx*Bx + By*By + Bz*Bz) + 1e-12
    Cm = np.sqrt(Cx*Cx + Cy*Cy + Cz*Cz) + 1e-12
    return np.clip((Bx*Cx + By*Cy + Bz*Cz)/(Bm*Cm), -1.0, 1.0).astype(np.float32), Bm.astype(np.float32)

def linking_score(loops, N, downsample=DOWNSAMPLE_LK):
    if len(loops) < 2: return 0.0, 0
    box = np.array([N,N,N], np.float32)
    total = 0.0; linked = 0
    for i in range(len(loops)):
        for j in range(i+1, len(loops)):
            A = loops[i][::downsample]; B = loops[j][::downsample]
            a_mid = 0.5*(A[1:]+A[:-1]); b_mid = 0.5*(B[1:]+B[:-1])
            da = A[1:]-A[:-1]; db = B[1:]-B[:-1]
            Da = da[:,None,:]; Db = db[None,:,:]
            Ra = a_mid[:,None,:]; Rb = b_mid[None,:,:]
            dR = _min_image_delta(Ra, Rb, box)
            num = np.einsum('ijk,ijk->ij', np.cross(Da,Db), dR)
            denom = (np.linalg.norm(dR, axis=2)**3 + 1e-12)
            lk = float(np.sum(num/denom)/(4*np.pi))
            k = abs(np.rint(lk))
            total += k
            if k >= 1: linked += 1
    return float(total), int(linked)

# ----- load latest field -----
root = Path(os.getenv('CNT_LAB_DIR') or ('E:\\CNT' if Path('E:\\CNT').exists() else os.getcwd()))
hunt_dir = root / "artifacts" / "cnt_physical_glyph_hunt"
runs = sorted([p for p in hunt_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {hunt_dir}"
run_dir = runs[0]
npz = run_dir/"best_field.npz"
if not npz.exists(): npz = run_dir/"field_small.npz"
d = np.load(npz)
Bx, By, Bz = d["Bx"].astype(np.float32), d["By"].astype(np.float32), d["Bz"].astype(np.float32)
dx = float(d.get("dx", 2*np.pi/Bx.shape[0])); dy = float(d.get("dy", 2*np.pi/By.shape[1])); dz = float(d.get("dz", 2*np.pi/Bz.shape[2]))
sp = (dx,dy,dz)
N = Bx.shape[0]

print("CNT v0.10-THREAD-SEEDSMART")
t0=time.perf_counter()
Bx, By, Bz = _project_solenoidal(Bx, By, Bz, sp)   # GPU
hH = helicity_density(Bx, By, Bz, sp)              # GPU
print(f"[orig] helicity_density={hH:.3e}  (proj+H {time.perf_counter()-t0:.2f}s)")

# α-map + |B| map
alpha, Bmag = beltrami_alpha(Bx, By, Bz, sp)
bcut = np.percentile(Bmag, BMAG_PCTL)
mask = (alpha > ALPHA_SEED_THR) & (Bmag > bcut)
idx = np.argwhere(mask)
if idx.size == 0:
    # relax once if too strict
    alpha_thr_relax = max(0.5, ALPHA_SEED_THR - 0.05)
    bcut_relax = np.percentile(Bmag, 50)
    mask = (alpha > alpha_thr_relax) & (Bmag > bcut_relax)
    idx = np.argwhere(mask)
K = min(MAX_SEEDS, len(idx))
if K == 0:
    print("[seeds] no coherent regions found; falling back to uniform grid.")
    coords = np.linspace(0, N, 5, endpoint=False) + N/10
    seeds = np.array([[xi,yi,zi] for xi in coords for yi in coords for zi in coords], np.float32)
else:
    sel = idx[np.random.default_rng(42).choice(len(idx), size=K, replace=False)]
    # jitter inside voxels
    jitter = np.random.default_rng(43).random((K,3)).astype(np.float32) - 0.5
    seeds = (sel.astype(np.float32) + jitter).astype(np.float32)

# threaded harvest
def trace_batch(batch):
    out=[]
    for s in batch:
        for dt in DT_LIST:
            for sign in (1.0,-1.0):
                L = _trace_loop(tuple(s), Bx, By, Bz, dt=sign*dt, steps=STEPS, min_close=MIN_CLOSE, close_tol=CLOSE_TOL)
                if L is not None: out.append(L)
    return out

def chunkify(arr, n):
    n = max(1, n)
    k = max(1, len(arr)//n)
    for i in range(0, len(arr), k):
        yield arr[i:i+k]

loops=[]
start=time.perf_counter()
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futs=[ex.submit(trace_batch, chunk) for chunk in chunkify(list(seeds), MAX_WORKERS)]
    for fut in as_completed(futs):
        loops.extend(fut.result())
        if len(loops) >= TARGET_LOOPS*3: break

loops = dedup_by_center(loops, N)[:TARGET_LOOPS]
# coherence along loops
def coh_frac(L): 
    vals = [_tri_interp_periodic(alpha, p) for p in L[:-1]]
    return float((np.array(vals)>ALPHA_THR).mean())
coh_loops = [L for L in loops if coh_frac(L) >= COH_FRAC_REQ]
total_abs, linked_pairs = linking_score(coh_loops, N, downsample=DOWNSAMPLE_LK)
print(f"[loops] total={len(loops)}  coherent={len(coh_loops)}  |Lk|={total_abs:.0f}  linked_pairs={linked_pairs}  ({time.perf_counter()-start:.1f}s)")

# --- FAST nulls with same seeding footprint ---
null_vals=[]
for r in range(NNULL):
    Bxr, Byr, Bzr = spectral_rotation_null(Bx, By, Bz, sp, np.random.default_rng(1000+r))  # GPU
    loops_r=[]
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs=[ex.submit(trace_batch, chunk) for chunk in chunkify(list(seeds), MAX_WORKERS)]
        for fut in as_completed(futs):
            loops_r.extend(fut.result())
            if len(loops_r) >= len(loops)*3: break
    loops_r = dedup_by_center(loops_r, N)[:len(loops)]
    # reuse alpha from original for gate? No—compute quick α_r for fairness
    alpha_r, _ = beltrami_alpha(Bxr, Byr, Bzr, sp)
    def coh_r(L):
        vals=[_tri_interp_periodic(alpha_r, p) for p in L[:-1]]
        return float((np.array(vals)>ALPHA_THR).mean())
    coh_r_loops = [L for L in loops_r if coh_r(L) >= COH_FRAC_REQ]
    t_abs_r, _ = linking_score(coh_r_loops, N, downsample=DOWNSAMPLE_LK)
    null_vals.append(t_abs_r)

null_vals = np.array(null_vals, np.float32)
if (linked_pairs==0) or (len(coh_loops)<2):
    link_pct = 0.50   # degenerate guard: don't let 0 linking look "great"
    p_link   = 1.0
else:
    link_pct = float((np.sum(null_vals <= total_abs) + 0.5)/(NNULL+1))
    p_link   = float((np.sum(null_vals >= total_abs) + 1)/(NNULL+1))
print(f"[nulls] median|Lk|={np.median(null_vals):.1f}  percentile={link_pct:.3f}  p={p_link:.4f}")

# echo reuse
echo_csv = run_dir/"echo_spectrum.csv"
if echo_csv.exists():
    arr = np.loadtxt(echo_csv, delimiter=",", skiprows=1)
    freqs, P = arr[:,0], arr[:,1]
    echo = float((P[(freqs>1.6)&(freqs<1.8)].sum())/(P.sum()+1e-12))
else:
    t = np.linspace(0,10,4096); dt = t[1]-t[0]
    sig = np.sin(2*np.pi*1.7*t)*np.exp(-t/6.0)
    freqs = np.fft.rfftfreq(len(sig), d=dt); P = np.abs(np.fft.rfft(sig))**2/len(sig)
    echo = float((P[(freqs>1.6)&(freqs<1.8)].sum())/(P.sum()+1e-12))

# final FAST score (guarded)
wH,wL,wE = 0.25,0.50,0.25
H_norm = float(min(1.0, abs(hH)/(1e-2)))
glyphness = float(wH*H_norm + wL*link_pct + wE*echo)
label = "CANDIDATE (FAST)"
if (len(coh_loops)>=4 and linked_pairs>=1 and link_pct>=0.90 and echo>=0.20 and glyphness>=0.70):
    label = "PHYSICAL-GLYPH:PASS"
print(f"[CNT] glyphness={glyphness:.3f}  (H={H_norm:.2f}, Link%={link_pct:.2f}, Echo={echo:.2f}) → {label}")


CNT v0.10-THREAD-SEEDSMART
[orig] helicity_density=9.769e-02  (proj+H 5.47s)
[loops] total=10  coherent=1  |Lk|=0  linked_pairs=0  (4020.5s)


TypeError: 'ndarray' object cannot be interpreted as an integer